In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os.path
import glob
import string
import statistics
from os.path import exists
from scipy.stats import norm
from scipy.stats import zscore
from scipy.stats import pearsonr  
from scipy.stats import chisquare
from sklearn import linear_model
import statsmodels.api as sm
from statsmodels.stats.anova import AnovaRM
import math
import scipy.stats

In [3]:
def snodgrass(num,denom):
    return (float(num)+.5) / (float(denom) +1)

def dprime(hit,fa):
    return norm.ppf(hit,loc=0,scale=1) - norm.ppf(fa,loc=0,scale=1)

In [4]:
def clean_pidn(filename): 
    
    #find pidn
    if not filename.startswith('.'):
        file_temp = filename.strip().split('_')
        file_temp2 = file_temp[1].strip().split('.')
        if file_temp2[0][0]=='0':
            file_PIDN= file_temp2[0].strip().split('0', 1)
            pidn=file_PIDN[1]
        else:
            pidn=int(file_temp2[0])
            
    return (pidn)

In [5]:
## Code taken from Craig Stark to find lines for the beginning of the study and test phases

def findlines(filename):
    s_starts=[]
    t_starts=[]
    with open(filename) as file:
        for linenum, linetxt in enumerate(file,1):
            if "Study phase started at" in linetxt:
                s_starts.append(linenum)
            elif "Test phase started at" in linetxt:
                t_starts.append(linenum)
    return (s_starts, t_starts)


In [6]:
def targ_trials(data):
    ## calculate number of Target Trials
    #initialize number of target trials to zero
    num_targ_trials = 0
    #loop through the data and add one if the condition is T (target)
    for d in range(len(data)):
        if data.loc[d, 'Cond'] == 'T':
            num_targ_trials +=1
    return num_targ_trials

In [7]:
def foil_trials(data):
        ## calculate number of Foil Trials
    #initialize number of foil trials to zero
    num_foil_trials = 0
    #loop through the data and add one to the number of new trials if the condition is F (foil)
    for g in range(len(data)):
        if data.loc[g, 'Cond'] == 'F':
            num_foil_trials +=1
    return num_foil_trials

In [8]:
def lure_trials(data):
        ## calculate number of Lure Trials
    #initialize number of lure trials to zero
    num_lure_trials = 0
    #loop through the data and add one to the number of lure trials if the condition is L (Lure)
    for f in range(len(data)):
        if data.loc[f, 'Cond'] == 'L':
            num_lure_trials +=1
    return num_lure_trials 

In [9]:
def no_resp(data):
     ## calculate total number of no response

    #initialize nuumber of no resposnses to zero
    num_nr = 0
    #for loop through the data and add one if the response is 0 (meaning no response); print the result 
    for n in range(len(data)):
        if (data.loc[n, 'Resp'] == 0):
            num_nr +=1  
    
     ## calculate number of no responses to Targets

    #initialize number of no responses to targets to zero
    num_nr_targ = 0
    #loop through the data and add one to number of no responses to targets
    #if the response is 0 (no response) and the condition is T (target); print result
    for a in range(len(data)):
        if (data.loc[a, 'Resp'] ==0) & (data.loc[a, 'Cond'] == 'T'):
            num_nr_targ +=1
         ## calculate number of no responses to Lures

    #initialize number of no responses to lures to zero
    num_nr_lure = 0
    #loop through the data and add one to the number of no responses to lures 
    #if the response is 0 (no response) and the condition is L (Lure)
    for b in range(len(data)):
        if (data.loc[b, 'Resp'] == 0) & (data.loc[b, 'Cond'] =='L'):
            num_nr_lure+=1

    ## calculate number of no responses to Foils

    #initialize number of no responses to foils to zero
    num_nr_foil = 0
    #loop through the data and add one to the number of no responses to foils
    #if the response is 0 (no response) and the condition is F (foil)
    for c in range(len(data)):
        if (data.loc[c, 'Resp'] == 0) & (data.loc[c,'Cond'] == 'F'):
            num_nr_foil+=1
            
    prop_nr = num_nr/test_len

    return num_nr, num_nr_targ, num_nr_lure, num_nr_foil, prop_nr

In [10]:
def targ_resp(data):
    ## calculate number of target trials
    num_targ_trials = targ_trials(data)
    ## calculate number of Old|Target

    #iitialize the number of Target|Old to zero
    num_old_targ = 0
    #loop through the data and add one to the number of of Old|Target 
    #if the condition is T (target) and if the response is 1 (Old); print result
    for j in range(len(data)):
        if ((data.loc[j, 'Cond'] == 'T') & (data.loc[j, 'Resp'] == 1)):
            num_old_targ +=1

    ## Calculate proportion of Old|Target for ALL Target Trials

    #Divide the number of old|target by the total number of target trials
    prop_old_targ = num_old_targ/num_targ_trials
    
    ## Calculate number of Similar|Target

    #initialize the number of Similar|Target to zero
    num_sim_targ = 0

    #loop through the data and add one to the number of Similar|Target 
    #if the condition is T (target) and if the participant responsed 2 (Similar); print result
    for k in range(len(data)):
        if ((data.loc[k, 'Cond']=='T') & (data.loc[k, 'Resp']==2)):
            num_sim_targ+=1

    ## calculate proportion of Similar|target for all target trials

    #divide the number of Similar|target by the total number of target trials
    prop_sim_targ= num_sim_targ/num_targ_trials

    ## calculate number of New|Target

    # initialize the number of New|Target to zero
    num_new_targ = 0
    #loop through the data and add one to the number of New|Target
    #if the condition is T(Target) and if the participant responded 3(New); print result
    for l in range(len(data)):
        if ((data.loc[l, 'Cond'] == 'T') & (data.loc[l, 'Resp'] == 3)):
            num_new_targ+=1
    
    ## calculate the proportion of New|Target for all target trials

    #divide number of New|Target by the total number of target trials
    prop_new_targ = num_new_targ/num_targ_trials
   

    return num_old_targ, prop_old_targ, num_sim_targ, prop_sim_targ, num_new_targ, prop_new_targ

In [11]:
def lure_resp(data):
    num_lure_trials = lure_trials(data)
    ## calculate number of Old|Lure

    #Initialize the number of Old|Lure to zero
    num_old_lure = 0
    #loop through the data and add one to the number of Old|Lure 
    # if the condition is L (Lure) and the participant responded 1 (Old); print result
    for m in range(len(data)):
        if ((data.loc[m, 'Cond']=='L') & data.loc[m, 'Resp'] == 1):
            num_old_lure+=1
   
    ## calculate the proportion of Old|Lure for all lure trials

    #divide number of Old|Lure by the total number of lure trials
    prop_old_lure = num_old_lure/num_lure_trials
    
       ## calculate number of Similar|Lure

    #Initialize the number of Similar|Lure to zero
    num_sim_lure = 0
    #loop through the data and add one to the number of Similar|Lure
    #if the condition is L (Lure) and the participant responded 2 (Similar); print result
    for o in range(len(data)):
        if ((data.loc[o, 'Cond']=="L") & (data.loc[o, 'Resp'] == 2)):
            num_sim_lure+=1

    ## Calculate the proportion of Similar|Lure for all lure trials

    #divide number of Similar|Lure by the total number of lure trials
    prop_sim_lure = num_sim_lure/num_lure_trials
    
    ## calculate number of New|Lure

    #Initialize the number of New|Lure to zero
    num_new_lure = 0
    #loop through the data and add one to the number of New|Lure
    #if the condition is L(Lure) and the participant responded 3 (New); print result
    for p in range(len(data)):
        if ((data.loc[p, 'Cond']=='L') & (data.loc[p, 'Resp']== 3)):
            num_new_lure+=1

    ## calculate proportion of new|lure for all lure trials

    #divide number of new|lure by the total number of lure trials
    prop_new_lure = num_new_lure/num_lure_trials

    return num_old_lure, prop_old_lure, num_sim_lure, prop_sim_lure, num_new_lure, prop_new_lure

In [12]:
def foil_resp(data):
    ## calculate number of foil trials
    num_foil_trials = foil_trials(data)
      ## calculate number of Old|Foil
     #Initialize the number of Old|Foil
    num_old_foil = 0
    #loop throught the data and add one to the number of Old|Foil
    #if the condition is F (Foil) and the participant responded 1 (Old); print result
    for q in range(len(data)):
        if ((data.loc[q, 'Cond'] == 'F') & (data.loc[q, 'Resp'] == 1)):
            num_old_foil+=1

    ## calculate proportion of old|foil for all foil trials

    #divide number of old|foil by the total number of foil trials
    prop_old_foil = num_old_foil/num_foil_trials
    
    ##calculate number of Similar|Foil

    #Initialize the number of Similar|Foil to zero
    num_sim_foil = 0
    #loop through the data and add one to the number of Similar|Foil
    #if the condition is F(Foil) and the participant responded 2(similar); print result
    for r in range(len(data)):
        if ((data.loc[r, 'Cond'] == 'F') & (data.loc[r, 'Resp'] == 2)):
            num_sim_foil+=1

      ## calculate proportion of similar|foil for all foil trials

    #divide the number of similar|foil by the number of foil trials
    prop_sim_foil = num_sim_foil/num_foil_trials
    
    ##calculate nummber of New|Foil

    #Initialize the number of New|Foil to zero
    num_new_foil = 0
    #Loop through the data and add one to the number of New|Foil
    #if the condition is F(Foil) and the participant responded 3(New)
    for s in range(len(data)):
        if ((data.loc[s, 'Cond'] == 'F') & (data.loc[s, 'Resp'] == 3)):
            num_new_foil+=1

   
    
    
    ## calculate proportion of new|foil for all foil trials 

    #divide the number of new|foil by the total number of foil trials
    prop_new_foil = num_new_foil/num_foil_trials

    return num_old_foil, prop_old_foil, num_sim_foil, prop_sim_foil, num_new_foil, prop_new_foil

In [13]:
def corr_ans(data):
    num_targ_trials = targ_trials(data)
    num_lure_trials = lure_trials(data)
    num_foil_trials = foil_trials(data)
    num_new_foil = 0
    num_old_targ = 0
    num_sim_lure = 0 
    for s in range(len(data)):
        if ((data.loc[s, 'Cond'] == 'F') & (data.loc[s, 'Resp'] == 3)):
            num_new_foil+=1
    
    for j in range(len(data)):
        if ((data.loc[j, 'Cond'] == 'T') & (data.loc[j, 'Resp'] == 1)):
            num_old_targ +=1
     
    for o in range(len(data)):
        if ((data.loc[o, 'Cond']=="L") & (data.loc[o, 'Resp'] == 2)):
            num_sim_lure+=1

    ## Calculate number of correct responses 
    #add the number of correct responses (New|Foil, Old|Target, Sim|Lure)
    corr_resp = num_new_foil + num_old_targ + num_sim_lure

    ## calculate proportion of correct responses

    prop_corr_resp = corr_resp/(num_targ_trials + num_lure_trials + num_foil_trials)
    
    return corr_resp, prop_corr_resp

In [14]:
def calc_bps(prop_sim_lure, prop_sim_foil):
    ## calculate BPS Score

    # subtract the proportion of sim|foil from the proportion of sim|lure
    bps_score = prop_sim_lure - prop_sim_foil
    return bps_score


In [15]:
def recog_scores(prop_old_targ, prop_old_foil, prop_old_lure, prop_sim_lure):
    ## calculate traditional recognition score

    #subtract the proportion of old|foil from the proportion of old|targ
    recog_targ_foil= prop_old_targ-prop_old_foil
    
    ## calculate other recognition score

    ## subtract old|lure from old|target
    recog_targ_lure = prop_old_targ - prop_old_lure
    
    ## calculate the rate of similar items correctly identified 
    ##minus the rate of similar items misidentified as old (S|S-O|S)
    #Aldi, G. A., Lange, I., Gigli, C., Goossens, L., Schruers, K. R., & 
    #Cosci, F. (2018). Validation of the Mnemonic Similarity Task–Context
    #Version. Revista Brasileira de Psiquiatria, 40(4), 432-440.

    recog_lure = prop_old_lure - prop_sim_lure 
    
    return recog_targ_foil, recog_targ_lure, recog_lure

In [16]:
demo_data = pd.read_csv('/Users/madisonhunt/Desktop/Oak/stark_task/stark_demographics.csv')
pidn_list = demo_data['pidn']
file_list=pd.DataFrame()

files= os.listdir('/Users/madisonhunt/Desktop/Oak/stark_task/Raw/Data')
row=0
for name in files:
    if not name.startswith('.'):
        file_temp = name.strip().split('_')
        file_temp2 = file_temp[1].strip().split('.')
        if file_temp2[0][0]=='0':
            file_PIDN= file_temp2[0].strip().split('0', 1)
            file_list.loc[row,'PIDN']=file_PIDN[1]
        else:
            file_list.loc[row,'PIDN']=file_temp2[0]  
        file_list.loc[row, 'PIDN'] = int(file_list.loc[row, 'PIDN'])
    row+=1
file_list = file_list.sort_values(by = 'PIDN')

row = 0
for n in file_list['PIDN']:
    if n not in pidn_list:
        print(n)
    row+=1   


FileNotFoundError: File b'/Users/madisonhunt/Desktop/Oak/stark_task/stark_demographics.csv' does not exist

In [17]:
all_data=pd.DataFrame()
np_data=pd.DataFrame()
row=0
missing=0
has_data=0
n_stim_per=64
study_len = n_stim_per*2
test_len = n_stim_per*3
demo_data = pd.read_csv('/Users/madisonhunt/Desktop/Oak/stark_task/stark_demographics.csv')
pidn_list = demo_data['pidn']
age_list = demo_data['np_age']
ed_list = demo_data['yrs_educ']
gender_list = demo_data['gender']
for pidn in pidn_list:

    if pidn < 685:
        filename = '/Users/madisonhunt/Desktop/Oak/stark_task/Data/BPSOlog_0{}.txt'.format(pidn)
#         print(filename)
    elif pidn > 685:
        filename = '/Users/madisonhunt/Desktop/Oak/stark_task/Data/MSTlog_0{}.txt'.format(pidn)
#         print(filename)
        if pidn > 1000: 
            filename = '/Users/madisonhunt/Desktop/Oak/stark_task/Data/MSTlog_{}.txt'.format(pidn)
    #         print(filename)

    if exists(filename):

        has_data +=1
        np_data.loc[row, 'pidn'] = demo_data.loc[row, 'pidn']
        np_data.loc[row, 'mmse'] = demo_data.loc[row, 'mmse']
        np_data.loc[row, 'hvltr4'] = demo_data.loc[row, 'hvltr4']
        np_data.loc[row, 'hvltpr'] = demo_data.loc[row, 'hvltrpr']
        np_data.loc[row, 'bvmtr4'] = demo_data.loc[row, 'bvmtr4']
        np_data.loc[row, 'bvmtpr'] = demo_data.loc[row, 'bvmtrpr']
        np_data.loc[row, 'lm2'] = demo_data.loc[row, 'lm2']
        np_data.loc[row, 'lmpr'] = demo_data.loc[row, 'lmpr']
        np_data.loc[row, 'mmse_norm'] = demo_data.loc[row, 'mmse_norm']
        np_data.loc[row, 'hvltr4_norm'] = demo_data.loc[row, 'hvltr4_norm']
        np_data.loc[row, 'bvmtr4_norm'] = demo_data.loc[row, 'bvmtr4_norm']
        np_data.loc[row, 'lm2_norm'] = demo_data.loc[row, 'lm2_norm']
        np_data.loc[row, 'ir_comp'] = demo_data.loc[row, 'immediate_recall_comp']
        np_data.loc[row, 'dr_comp'] = demo_data.loc[row, 'delayed_recall_comp']
        np_data.loc[row, 'recog_comp'] = demo_data.loc[row, 'recognition_comp']
        

        s_starts, t_starts = findlines(filename)
        if len(s_starts) & len(t_starts):
            study_starts = s_starts[-1]
            test_starts = t_starts[-1]
            pt_data =  pd.read_csv(filename, delimiter="\t", skiprows= test_starts, nrows=test_len, header = 0)     

            all_data.loc[row, 'pidn'] = pidn 
            all_data.loc[row, 'age'] = age_list[row]
            all_data.loc[row, 'education'] = ed_list[row]
            if gender_list[row] == 0:
                all_data.loc[row, 'gender'] = 'F'
            else:
                all_data.loc[row,'gender'] = 'M'
            all_data.loc[row, 'num_targ_tr']  = targ_trials(pt_data)             
            all_data.loc[row, 'num_foil_tr'] = foil_trials(pt_data)
            all_data.loc[row, 'num_lure_tr'] = lure_trials(pt_data)
            [all_data.loc[row, 'num_nr'], all_data.loc[row, 'num_nr_t'], 
             all_data.loc[row, 'num_nr_l'], all_data.loc[row, 'num_nr_f'],
             all_data.loc[row, 'prop_nr']] = no_resp(pt_data)
            [all_data.loc[row, 'n_old_targ'], all_data.loc[row, 'p_old_targ'], 
             all_data.loc[row, 'n_sim_targ'], all_data.loc[row, 'p_sim_targ'],
             all_data.loc[row, 'n_new_targ'], all_data.loc[row, 'p_new_targ']] = targ_resp(pt_data)
            [all_data.loc[row, 'n_old_lure'], all_data.loc[row, 'p_old_lure'],
            all_data.loc[row, 'n_sim_lure'], all_data.loc[row, 'p_sim_lure'],
            all_data.loc[row, 'n_new_lure'], all_data.loc[row, 'p_new_lure']] = lure_resp(pt_data)
            [all_data.loc[row, 'n_old_foil'], all_data.loc[row, 'p_old_foil'],
            all_data.loc[row, 'n_sim_foil'], all_data.loc[row, 'p_sim_foil'], 
            all_data.loc[row, 'n_new_foil'], all_data.loc[row, 'p_new_foil']] = foil_resp(pt_data)
            [all_data.loc[row, 'n_correct'], all_data.loc[row, 'p_correct']] = corr_ans(pt_data)
            all_data.loc[row, 'bps'] = calc_bps(all_data.loc[row, 'p_sim_lure'], all_data.loc[row, 'p_sim_foil'])
            [all_data.loc[row, 'old_targ_old_foil'], all_data.loc[row, 'old_targ_old_lure'],
             all_data.loc[row,'old_lure_sim_lure']] = recog_scores(all_data.loc[row, 'p_old_targ'], all_data.loc[row, 'p_old_foil'], all_data.loc[row,'p_old_lure'],  all_data.loc[row, 'p_sim_lure'])
            all_data.loc[row, 'dprime_old_new'] = dprime(snodgrass(all_data.loc[row, 'n_old_targ'], all_data.loc[row, 'num_targ_tr']), snodgrass(all_data.loc[row, 'n_old_foil'], all_data.loc[row, 'num_foil_tr']))
            all_data.loc[row,'dprime_old_lure'] = dprime(snodgrass(all_data.loc[row, 'n_old_targ'], all_data.loc[row, 'num_targ_tr']), snodgrass(all_data.loc[row, 'n_old_lure'], all_data.loc[row, 'num_lure_tr']))
            all_data.loc[row, 'dprime_bps'] = dprime(snodgrass(all_data.loc[row, 'p_sim_lure'], all_data.loc[row, 'num_lure_tr']), snodgrass(all_data.loc[row, 'p_sim_foil'], all_data.loc[row, 'num_foil_tr']))
            row +=1
        else: print(pidn)
    else:
#         print(pidn)
        missing +=1
# print(all_data['dprime_old_new'])



FileNotFoundError: File b'/Users/madisonhunt/Desktop/Oak/stark_task/stark_demographics.csv' does not exist

In [18]:
print(all_data[all_data['pidn']==1105])

KeyError: 'pidn'

In [ ]:
all_data.to_csv('/Users/madisonhunt/Desktop/mst_data.csv', index=False)

In [ ]:
len(all_data)

In [ ]:
has_data

In [ ]:
missing

In [ ]:
has_data+missing

In [ ]:
all_data.head()

In [ ]:
all_data.columns.values

In [ ]:
from scipy import stats 
stats.ttest_rel(all_data['num_nr_t'], all_data['num_nr_l'], axis=0, nan_policy='propagate')

In [ ]:
stats.ttest_rel(all_data['num_nr_t'], all_data['num_nr_f'], axis=0, nan_policy='propagate')

In [ ]:
stats.ttest_rel(all_data['num_nr_l'], all_data['num_nr_f'], axis=0, nan_policy='propagate')

In [ ]:
anov_data = pd.DataFrame()

data_temp=all_data.copy()
data_temp['TrialType'] = 'Target'
data_temp['Pr_NR'] = (data_temp['num_nr_t']/64)

anov_data = data_temp




In [ ]:

x= all_data[['age', 'education']]
y=all_data['bps']
regr=linear_model.LinearRegression()
regr.fit(x,y)
print('Intercept: ', regr.intercept_)
print('Coefficients: ', regr.coef_)

In [ ]:
max(all_data['dprime_old_lure'])

In [18]:
data_long=pd.DataFrame()
row=0
missing=0
n_stim_per=64
study_len = n_stim_per*2
test_len = n_stim_per*3
demo_data = pd.read_csv('/Users/madisonhunt/Desktop/Oak/stark_task/stark_demographics.csv')
pidn_list = demo_data['pidn']
for pidn in pidn_list:
    if pidn != 999 and pidn !=685:
        if pidn < 685:
            filename = '/Users/madisonhunt/Desktop/Oak/stark_task/Data/BPSOlog_0{}.txt'.format(pidn)
           
        if pidn > 685:
            if pidn < 1000:
#         elif pidn > 685 and pidn <1000:
                filename = '/Users/madisonhunt/Desktop/Oak/stark_task/Data/MSTlog_0{}.txt'.format(pidn)
                
            else:
                filename = '/Users/madisonhunt/Desktop/Oak/stark_task/Data/MSTlog_{}.txt'.format(pidn)
                

        if exists(filename):

            s_starts, t_starts = findlines(filename)
            if len(s_starts) & len(t_starts):
                study_starts = s_starts[-1]
                test_starts = t_starts[-1]
                pt_data =  pd.read_csv(filename, delimiter="\t", skiprows= test_starts, nrows=test_len, header = 0)     

                pt_data['pidn'] = pidn
                
                data_long = data_long.append(pt_data,ignore_index=True)
            

In [19]:
data_long.loc[data_long['Resp']==1,'Resp'] = 'Old'
data_long.loc[data_long['Resp']==2,'Resp'] = 'Similar'
data_long.loc[data_long['Resp']==3,'Resp'] = 'New'
data_long.loc[data_long['Resp']==0,'Resp'] = 'No Resp'

data_long.loc[data_long['Cond'] == 'T', 'Cond'] = 'Target'
data_long.loc[data_long['Cond'] == 'L', 'Cond'] = 'Lure'
data_long.loc[data_long['Cond'] == 'F', 'Cond'] = 'Foil'

data_long.loc[data_long['Acc'] ==1, 'Acc'] = 'Correct'
data_long.loc[data_long['Acc'] == 0, 'Acc'] ='Incorrect'




In [ ]:
data_adj = pd.DataFrame()
row=0
row1=0
for n in pd.unique(data_long['pidn']):
    
    data_adj.loc[row1, 'pidn'] = n
    data_adj.loc[row1, 'Cond'] ='Target'
    data_adj.loc[row1, 'nr'] = len(data_long[(data_long['pidn'] == n) & (data_long['RT'] == 0) &(data_long['Cond']=='Target')])
    data_adj.loc[row1, 'trials'] = len(data_long[(data_long['pidn'] == n) & (data_long['RT'] > 400) & (data_long['Cond'] =='Target')])
    data_adj.loc[row1, 'correct'] = len((data_long[(data_long['pidn'] == n) & (data_long['RT'] > 400) & (data_long['Cond'] =='Target') & (data_long['Resp'] == 'Old')]))/(data_adj.loc[row1, 'trials'])
   
    data_adj.loc[row1+1, 'pidn'] = n
    data_adj.loc[row1+1, 'Cond'] ='Lure'
    data_adj.loc[row1+1, 'nr'] = len(data_long[(data_long['pidn'] == n) & (data_long['RT'] == 0) &(data_long['Cond']=='Lure')])
    data_adj.loc[row1+1, 'trials'] = len(data_long[(data_long['pidn'] == n) & (data_long['RT'] > 400) & (data_long['Cond'] =='Lure')])
    data_adj.loc[row1+1, 'correct'] = len((data_long[(data_long['pidn'] == n) & (data_long['RT'] > 400) & (data_long['Cond'] =='Lure') & (data_long['Resp'] == 'Similar')]))/(data_adj.loc[row1+1, 'trials'])
    
    data_adj.loc[row1+2, 'pidn'] = n
    data_adj.loc[row1+2, 'Cond'] ='Foil'
    data_adj.loc[row1+2, 'nr'] = len(data_long[(data_long['pidn'] == n) & (data_long['RT'] == 0) &(data_long['Cond']=='Foil')])
    data_adj.loc[row1+2, 'trials'] = len(data_long[(data_long['pidn'] == n) & (data_long['RT'] > 400) & (data_long['Cond'] =='Foil')])
    data_adj.loc[row1+2, 'correct'] = len((data_long[(data_long['pidn'] == n) & (data_long['RT'] > 400) & (data_long['Cond'] =='Foil') & (data_long['Resp'] == 'New')]))/(data_adj.loc[row1+2, 'trials'])
    
    
    row1+=3
print(data_adj)

In [ ]:
sns.lmplot(x='nr', y ='correct', hue='Cond', data=data_adj)

In [ ]:
pearsonr(x=data_adj[data_adj['Cond']=='Foil']['nr'], y =data_adj[data_adj['Cond']=='Foil']['correct'] )

In [ ]:
print(data_adj_wide[data_adj_wide['pidn']==1045])

In [ ]:
data_adj_wide = pd.DataFrame()
row=0

for n in pd.unique(data_adj['pidn']):
    data_adj_wide.loc[row, 'pidn'] = n
    data_adj_wide.loc[row, 't_trials'] = data_adj.loc[row, 'trials']
    data_adj_wide.loc[row, 'l_trials'] = data_adj.loc[row+1, 'trials']
    data_adj_wide.loc[row, 'f_trials'] = data_adj.loc[row+2,'trials']
    row+=3
    
print(data_adj_wide)

In [ ]:

print(len(data_adj_wide[(data_adj_wide['t_trials'] > 48) & (data_adj_wide['l_trials'] > 48)& (data_adj_wide['f_trials'] > 48)]['pidn']))


In [ ]:
lure_data_adj = pd.DataFrame()
row=0
for n in pd.unique(data_long['pidn']):
    lure_data_adj.loc[row, 'pidn'] = n
    lure_data_adj.loc[row, 'nr'] = len(data_long[(data_long['pidn'] == n) & (data_long['RT'] == 0) &(data_long['Cond']=='Lure')])
    lure_data_adj.loc[row, 'RT'] = data_long.loc[(data_long['pidn'] == n)&(data_long['Cond']=='Lure')&(data_long['RT']>400),'RT'].median()
    lure_data_adj.loc[row, 'trials'] = len(data_long[(data_long['pidn'] == n) & (data_long['RT'] > 400) & (data_long['Cond'] =='Lure') & (data_long['Resp'] != 'Old')])
    lure_data_adj.loc[row, 'correct'] = len((data_long[(data_long['pidn'] == n) & (data_long['RT'] > 400) & (data_long['Cond'] =='Lure') & (data_long['Resp'] == 'Similar')]))/(lure_data_adj.loc[row, 'trials'])
    row+=1
    

In [ ]:
g = sns.lmplot(x='RT', y='correct', data=lure_data_adj)

In [ ]:
pearsonr(x=lure_data_adj['RT'], y=lure_data_adj['correct'])

In [20]:
# used the summary df you sent me
mst_data = pd.read_csv('/Users/madisonhunt/Desktop/mst_data.csv') 

# exclude by old/new d'
mst_clean = mst_data[mst_data['dprime_old_new']>.5]
len(mst_clean) # 3 fewer subjects

#confirmed now absent

# create factor variable reflecting nr status for a given person


mst_clean.loc[mst_clean['prop_nr']>=.20, 'nr_group'] = 'nr_.20'
mst_clean.loc[(mst_clean['prop_nr']>=.15)&(mst_clean['prop_nr']<.20), 'nr_group'] = 'nr_.15'
mst_clean.loc[(mst_clean['prop_nr']>=.10)&(mst_clean['prop_nr']<.15), 'nr_group'] = 'nr_.10'
mst_clean.loc[(mst_clean['prop_nr']>=.05)&(mst_clean['prop_nr']<.10), 'nr_group'] = 'nr_.05'
mst_clean.loc[(mst_clean['prop_nr']<.05), 'nr_group'] = 'nr_low'

# plot the data with nr group in different colours, but plot the regression line over the full dataset




/Users/madisonhunt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/madisonhunt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [ ]:
print(mst_data[mst_data['prop_nr'] >.20]['pidn'])

In [ ]:
print(np_data[np_data['pidn']== 617])

In [ ]:
g=sns.lmplot(x='age', y='dprime_old_new', hue='nr_group', data=mst_clean, fit_reg= False, hue_order = ['nr_low', 'nr_.05','nr_.10', 'nr_.15', 'nr_.20'])
sns.regplot(x="age", y="dprime_old_new", data=mst_clean, scatter=False, ax=g.axes[0, 0])

In [ ]:
g=sns.lmplot(x='education', y='dprime_old_new', hue='nr_group', data=mst_clean, fit_reg= False, hue_order = ['nr_low', 'nr_.05','nr_.10', 'nr_.15', 'nr_.20'])
sns.regplot(x="education", y="dprime_old_new", data=mst_clean, scatter=False, ax=g.axes[0, 0])

In [ ]:
g=sns.lmplot(x='age', y='dprime_old_lure', hue='nr_group', data=mst_clean, fit_reg= False, hue_order = ['nr_low', 'nr_.05','nr_.10', 'nr_.15', 'nr_.20'])
sns.regplot(x="age", y="dprime_old_lure", data=mst_clean, scatter=False, ax=g.axes[0, 0])

In [ ]:
g=sns.lmplot(x='education', y='dprime_old_lure', hue='nr_group', data=mst_clean, fit_reg= False, hue_order = ['nr_low', 'nr_.05','nr_.10', 'nr_.15', 'nr_.20'])
sns.regplot(x="education", y="dprime_old_lure", data=mst_clean, scatter=False, ax=g.axes[0, 0])

In [ ]:
g=sns.lmplot(x='age', y='dprime_bps', hue='nr_group', data=mst_clean, fit_reg= False, hue_order = ['nr_low', 'nr_.05','nr_.10', 'nr_.15', 'nr_.20'])
sns.regplot(x="age", y="dprime_bps", data=mst_clean, scatter=False, ax=g.axes[0, 0])

In [ ]:
g=sns.lmplot(x='education', y='dprime_bps', hue='nr_group', data=mst_clean, fit_reg= False, hue_order = ['nr_low', 'nr_.05','nr_.10', 'nr_.15', 'nr_.20'])
sns.regplot(x="education", y="dprime_bps", data=mst_clean, scatter=False, ax=g.axes[0, 0])

In [ ]:
g=sns.lmplot(x="dprime_old_new",y="dprime_old_lure",hue="nr_group",data=mst_clean,fit_reg=False, hue_order = ['nr_low', 'nr_.05','nr_.10', 'nr_.15', 'nr_.20'])
sns.regplot(x="dprime_old_new", y="dprime_old_lure", data=mst_clean, scatter=False, ax=g.axes[0, 0])

In [ ]:
g=sns.lmplot(x="dprime_old_new",y="dprime_old_lure",hue="nr_group",data=mst_clean[mst_clean['nr_group']!='nr_.20'],fit_reg=False, hue_order = ['nr_low', 'nr_.05','nr_.10', 'nr_.15'])
sns.regplot(x="dprime_old_new", y="dprime_old_lure", data=mst_clean[mst_clean['nr_group']!='nr_.20'], scatter=False, ax=g.axes[0, 0])

In [ ]:
pearsonr(x=mst_clean[mst_clean['nr_group']!='nr_.20']['dprime_bps'], y =mst_clean[mst_clean['nr_group']!='nr_.20']['dprime_old_lure'] )

In [ ]:
g=sns.lmplot(x="dprime_old_new",y="dprime_old_lure",hue="nr_group",data=mst_clean[(mst_clean['nr_group']!='nr_.20') & (mst_clean['nr_group']!='nr_.15')],fit_reg=False, hue_order = ['nr_low', 'nr_.05','nr_.10'])
sns.regplot(x="dprime_old_new", y="dprime_old_lure", data=mst_clean[(mst_clean['nr_group']!='nr_.20') & (mst_clean['nr_group']!='nr_.15')], scatter=False, ax=g.axes[0, 0])

In [ ]:
pearsonr(x=mst_clean['dprime_old_new'], y = mst_clean['dprime_bps'])


In [ ]:
g=sns.lmplot(x="dprime_old_new",y="dprime_old_lure",hue="nr_group",data=mst_clean[(mst_clean['nr_group']!='nr_.20') & (mst_clean['nr_group']!='nr_.15') & (mst_clean['nr_group']!='nr_.10')],fit_reg=False, hue_order = ['nr_low', 'nr_.05'])
sns.regplot(x="dprime_old_new", y="dprime_old_lure", data=mst_clean[(mst_clean['nr_group']!='nr_.20') & (mst_clean['nr_group']!='nr_.15') & (mst_clean['nr_group']!='nr_.10')], scatter=False, ax=g.axes[0, 0])

In [ ]:
g=sns.lmplot(x="dprime_old_new",y="dprime_bps",hue="nr_group",data=mst_clean,fit_reg=False, hue_order = ['nr_low', 'nr_.05','nr_.10', 'nr_.15', 'nr_.20'])
sns.regplot(x="dprime_old_new", y="dprime_bps", data=mst_clean, scatter=False, ax=g.axes[0, 0])

In [ ]:
g=sns.lmplot(x="dprime_old_new",y="dprime_bps",hue="nr_group",data=mst_clean[mst_clean['nr_group']!='nr_.20'],fit_reg=False, hue_order = ['nr_low', 'nr_.05','nr_.10', 'nr_.15'])
sns.regplot(x="dprime_old_new", y="dprime_bps", data=mst_clean[mst_clean['nr_group']!='nr_.20'], scatter=False, ax=g.axes[0, 0])

In [ ]:
g=sns.lmplot(x="dprime_old_new",y="dprime_bps",hue="nr_group",data=mst_clean[(mst_clean['nr_group']!='nr_.20') & (mst_clean['nr_group']!='nr_.15')],fit_reg=False, hue_order = ['nr_low', 'nr_.05','nr_.10'])
sns.regplot(x="dprime_old_new", y="dprime_bps", data=mst_clean[(mst_clean['nr_group']!='nr_.20') & (mst_clean['nr_group']!='nr_.15')], scatter=False, ax=g.axes[0, 0])

In [ ]:
g=sns.lmplot(x="dprime_old_new",y="dprime_bps",hue="nr_group",data=mst_clean[(mst_clean['nr_group']!='nr_.20') & (mst_clean['nr_group']!='nr_.15') & (mst_clean['nr_group']!='nr_.10')],fit_reg=False, hue_order = ['nr_low', 'nr_.05'])
sns.regplot(x="dprime_old_new", y="dprime_bps", data=mst_clean[(mst_clean['nr_group']!='nr_.20') & (mst_clean['nr_group']!='nr_.15') & (mst_clean['nr_group']!='nr_.10')], scatter=False, ax=g.axes[0, 0])

In [ ]:
g=sns.lmplot(x="dprime_old_new",y="dprime_bps",hue="nr_group",data=mst_clean[(mst_clean['nr_group']!='nr_.20') & (mst_clean['nr_group']!='nr_.15') & (mst_clean['nr_group']!='nr_.10')& (mst_clean['nr_group']!='nr_.05')],fit_reg=False)
sns.regplot(x="dprime_old_new", y="dprime_bps", data=mst_clean[(mst_clean['nr_group']!='nr_.20') & (mst_clean['nr_group']!='nr_.15') & (mst_clean['nr_group']!='nr_.10')& (mst_clean['nr_group']!='nr_.05')], scatter=False, ax=g.axes[0, 0])

In [ ]:
g=sns.lmplot(x="dprime_bps",y="dprime_old_lure",hue="nr_group",data=mst_clean,fit_reg=False, hue_order = ['nr_low', 'nr_.05','nr_.10', 'nr_.15', 'nr_.20'])
sns.regplot(x="dprime_bps", y="dprime_old_lure", data=mst_clean, scatter=False, ax=g.axes[0, 0])

In [ ]:
g=sns.lmplot(x="dprime_bps",y="dprime_old_lure",hue="nr_group",data=mst_clean[mst_clean['nr_group']!='nr_.20'],fit_reg=False, hue_order = ['nr_low', 'nr_.05','nr_.10', 'nr_.15'])
sns.regplot(x="dprime_bps", y="dprime_old_lure", data=mst_clean[mst_clean['nr_group']!='nr_.20'], scatter=False, ax=g.axes[0, 0])

In [ ]:
g=sns.lmplot(x="dprime_bps",y="dprime_old_lure",hue="nr_group",data=mst_clean[(mst_clean['nr_group']!='nr_.20') & (mst_clean['nr_group']!='nr_.15')],fit_reg=False, hue_order = ['nr_low', 'nr_.05','nr_.10'])
sns.regplot(x="dprime_bps", y="dprime_old_lure", data=mst_clean[(mst_clean['nr_group']!='nr_.20') & (mst_clean['nr_group']!='nr_.15')], scatter=False, ax=g.axes[0, 0])

In [ ]:
g=sns.lmplot(x="dprime_bps",y="dprime_old_lure",hue="nr_group",data=mst_clean[(mst_clean['nr_group']!='nr_.20') & (mst_clean['nr_group']!='nr_.15') & (mst_clean['nr_group']!='nr_.10')],fit_reg=False, hue_order = ['nr_low', 'nr_.05'])
sns.regplot(x="dprime_bps", y="dprime_old_lure", data=mst_clean[(mst_clean['nr_group']!='nr_.20') & (mst_clean['nr_group']!='nr_.15') & (mst_clean['nr_group']!='nr_.10')], scatter=False, ax=g.axes[0, 0])

In [ ]:
g=sns.lmplot(x="dprime_bps",y="dprime_old_lure",hue="nr_group",data=mst_clean[(mst_clean['nr_group']!='nr_.20') & (mst_clean['nr_group']!='nr_.15') & (mst_clean['nr_group']!='nr_.10')& (mst_clean['nr_group']!='nr_.05')],fit_reg=False)
sns.regplot(x="dprime_bps", y="dprime_old_lure", data=mst_clean[(mst_clean['nr_group']!='nr_.20') & (mst_clean['nr_group']!='nr_.15') & (mst_clean['nr_group']!='nr_.10')& (mst_clean['nr_group']!='nr_.05')], scatter=False, ax=g.axes[0, 0])

In [ ]:
print(all_data[all_data['dprime_old_new']<.3]['pidn'])

In [ ]:
temp1= data_long[data_long['RT']>400].reset_index(drop=True)
temp1.head()

subs2skip = [528, 866, 1230, 468, 883]
temp1 = temp1[(temp1['pidn']!=528) & (temp1['pidn']!=866) &(temp1['pidn']!=1230)]

row=0
temp2=pd.DataFrame()
for n in pd.unique(temp1['pidn']):
        temp2.loc[row, 'pidn'] = n
        temp2.loc[row, 'NR'] = all_data.loc[row, 'prop_nr']
        temp2.loc[row, 't_trial'] = len(temp1[(temp1['pidn']==n) & (temp1['Cond'] == 'Target')])
        temp2.loc[row, 'l_trial'] = len(temp1[(temp1['pidn']==n) & (temp1['Cond'] == 'Lure')])
        temp2.loc[row, 'f_trial'] = len(temp1[(temp1['pidn']==n) & (temp1['Cond'] == 'Foil')])
        temp2.loc[row, 'p_old_targ'] = all_data.loc[row, 'n_old_targ']/temp2.loc[row, 't_trial']
        temp2.loc[row, 'p_sim_targ'] = all_data.loc[row, 'n_sim_targ']/temp2.loc[row, 't_trial']
        temp2.loc[row, 'p_new_targ'] = all_data.loc[row, 'n_new_targ']/temp2.loc[row, 't_trial']
        temp2.loc[row, 'p_old_lure'] = all_data.loc[row, 'n_old_lure']/temp2.loc[row, 'l_trial']
        temp2.loc[row, 'p_sim_lure'] = all_data.loc[row, 'n_sim_lure']/temp2.loc[row, 'l_trial']
        temp2.loc[row, 'p_new_lure'] = all_data.loc[row, 'n_new_lure']/temp2.loc[row, 'l_trial']
        temp2.loc[row, 'p_old_foil'] = all_data.loc[row, 'n_old_foil']/temp2.loc[row, 'f_trial']
        temp2.loc[row, 'p_sim_foil'] = all_data.loc[row, 'n_sim_foil']/temp2.loc[row, 'f_trial']
        temp2.loc[row, 'p_new_foil'] = all_data.loc[row, 'n_new_foil']/temp2.loc[row, 'f_trial']
        temp2.loc[row, 'dprime_old_new'] = dprime(snodgrass(all_data.loc[row, 'n_old_targ'], temp2.loc[row, 't_trial']), snodgrass(all_data.loc[row, 'n_old_foil'], temp2.loc[row, 'f_trial']))
        temp2.loc[row,'dprime_old_sim'] = dprime(snodgrass(all_data.loc[row, 'n_old_targ'], temp2.loc[row, 't_trial']), snodgrass(all_data.loc[row, 'n_old_lure'], temp2.loc[row, 'l_trial']))
        temp2.loc[row, 'dprime_bps'] = dprime(snodgrass(all_data.loc[row, 'p_sim_lure'], temp2.loc[row, 'l_trial']), snodgrass(all_data.loc[row, 'p_sim_foil'], temp2.loc[row, 'f_trial']))
        row+=1

       

In [ ]:
nr_20 = temp2[temp2['NR']<.2]
nr_15 = temp2[temp2['NR']<.15]
nr_10 = temp2[temp2['NR']<.10]
nr_5 = temp2[temp2['NR']<.05]

temp2= temp2.dropna()

nr_20 = nr_20.dropna()
nr_15 = nr_15.dropna()
nr_10 = nr_10.dropna()
nr_5 = nr_5.dropna()

In [ ]:
sns.lmplot(x='dprime_old_new',y='dprime_bps', data=nr_5 )

In [ ]:
sns.lmplot(x='dprime_bps',y='dprime_old_sim', data=nr_5)

In [ ]:
pearsonr(x=temp2['dprime_old_new'], y=temp2['dprime_bps'])

In [ ]:
pearsonr(x=temp2['dprime_old_sim'], y=temp2['dprime_bps'])

In [ ]:
pearsonr(x=temp2['dprime_old_new'], y=temp2['dprime_old_sim'])

In [ ]:
sns.lmplot(x='dprime_old_new',y='dprime_old_sim', data=nr_5 )

In [ ]:
pearsonr(x=temp3['dprime_old_new'], y = temp3['dprime_bps'])

In [ ]:
pearsonr(x=temp3['dprime_old_new'], y = temp3['dprime_old_sim'])

In [ ]:
pearsonr(x=temp3['dprime_old_sim'], y = temp3['dprime_bps'])

In [ ]:
sns.lmplot(x='dprime_bps',y='dprime_old_sim', data=temp2 )

In [ ]:
temp_data = data_long[data_long['RT'] == 0]


temp2 = temp_data.groupby(['pidn', 'Cond']).count().reset_index()
# nr=pd.DataFrame()
# row = 0
# row1=0
# for n in pd.unique(temp2['pidn']):
#     nr.loc[row, 'f_nr'] = temp2.loc[row1, 'RT']
#     nr.loc[row, 'l_nr'] = temp2.loc[row1+1, 'RT']
#     nr.loc[row, 't_nr'] = temp2.loc[row1+2, 'RT']
#     row+=1
#     row1+=3
print(temp2)

In [ ]:
pearsonr(x=temp2['dprime_old_new'], y= temp2['dprime_bps'])

In [ ]:
trial_data = pd.DataFrame()
row=0
for subject in pd.unique(data_long['pidn']):
    trial_data.loc[row,'trials_exl_nr']=len(data_long[(data_long['pidn']==subject)&(data_long['RT']>0)])
    trial_data.loc[row,'trials_exl_nr_fast']=len(data_long[(data_long['pidn']==subject)&(data_long['RT']>400)])
    trial_data.loc[row,'trials_exl_fast']=len(data_long[(data_long['pidn']==subject)&((data_long['RT']>400)| (data_long['RT'] ==0))])
    
    trial_data.loc[row,'targ_trials_exl_nr']=len(data_long[(data_long['pidn']==subject)&(data_long['RT']>0)&(data_long['Cond']=='Target')])
    trial_data.loc[row,'targ_trials_exl_nr_fast']=len(data_long[(data_long['pidn']==subject)&(data_long['RT']>400)&(data_long['Cond']=='Target')])
    trial_data.loc[row,'targ_trials_exl_fast']=len(data_long[(data_long['pidn']==subject)&((data_long['RT']>400)| (data_long['RT'] ==0))&(data_long['Cond']=='Target')])
    
    trial_data.loc[row,'lure_trials_exl_nr']=len(data_long[(data_long['pidn']==subject)&(data_long['RT']>0)&(data_long['Cond']=='Lure')])
    trial_data.loc[row,'lure_trials_exl_nr_fast']=len(data_long[(data_long['pidn']==subject)&(data_long['RT']>400)&(data_long['Cond']=='Lure')])
    trial_data.loc[row,'lure_trials_exl_fast']=len(data_long[(data_long['pidn']==subject)&((data_long['RT']>400)| (data_long['RT'] ==0))&(data_long['Cond']=='Lure')])
    
    trial_data.loc[row,'foil_trials_exl_nr']=len(data_long[(data_long['pidn']==subject)&(data_long['RT']>0)&(data_long['Cond']=='Foil')])
    trial_data.loc[row,'foil_trials_exl_nr_fast']=len(data_long[(data_long['pidn']==subject)&(data_long['RT']>400)&(data_long['Cond']=='Foil')])
    trial_data.loc[row,'foil_trials_exl_fast']=len(data_long[(data_long['pidn']==subject)&((data_long['RT']>400)| (data_long['RT'] ==0))&(data_long['Cond']=='Foil')])
    
    trial_data.loc[row,'pidn']=subject
    
    row+=1
    
print(trial_data)

In [ ]:
print(trial_data[trial_data['pidn']==1045])

In [ ]:
trial_data = trial_data.merge(all_data, on='pidn')

In [ ]:

g=sns.boxplot(x='Cond', y='RT', data=temp2,  palette='pastel', order=['Target', 'Lure', 'Foil'], fliersize=0)
g= sns.stripplot(x='Cond', y='RT', data= temp2, jitter=.2,split=True,  order=['Target', 'Lure', 'Foil'], linewidth = .5)

plt.xlabel('Condition')
plt.ylabel('NR')

In [ ]:
sns.lmplot(x=nr[nr['Cond'] =='Target']['RT'], y =nr[nr['Cond'] =='Target']['RT'], data=nr)

In [ ]:
sns.barplot(x='Cond', y = 'RT', data=nr_data, order =['Target', 'Lure', 'Foil'])
plt.ylabel('Num NR')

In [ ]:
row = 0
foil_data=pd.DataFrame()
for pt in (data_long['pidn']):
    if data_long.loc[row, 'Cond'] == 'Foil':
        foil_data.loc[row, 'pidn'] = data_long.loc[row, 'pidn']
        foil_data.loc[row, 'RT'] = data_long.loc[row, 'RT']
        foil_data.loc[row, 'Resp'] = data_long.loc[row, 'Resp']
    row+=1
    
foil_data = foil_data[foil_data['RT']>400]
foil_data.reset_index(drop=True)  

In [ ]:
g = sns.distplot(foil_data[foil_data['Resp'] == 'Old']['RT'], kde=0, bins = 25)
g=sns.distplot(foil_data[foil_data['Resp']=='Similar']['RT'], kde=0, bins=25)
g= sns.distplot(foil_data[foil_data['Resp'] == 'New']['RT'], kde=0, bins = 25)
plt.legend(['Old', 'Similar', 'New'])
g.set(xlim=[400, 2050])
g.set(ylim=[0, 950])

In [ ]:
scipy.stats.ttest_ind(foil_data[foil_data['Resp'] == 'Old']['RT'], foil_data[foil_data['Resp']=='Similar']['RT'], nan_policy='propagate', equal_var = False)

In [ ]:
scipy.stats.ttest_ind(foil_data[foil_data['Resp'] == 'Old']['RT'], foil_data[foil_data['Resp']=='New']['RT'], nan_policy='propagate' equal_var=False)

In [ ]:
scipy.stats.ttest_ind(foil_data[foil_data['Resp'] == 'Similar']['RT'], foil_data[foil_data['Resp']=='New']['RT'], nan_policy='propagate', equal_var=False)

In [ ]:
foil_grouped = foil_data.groupby(['pidn', 'Resp']).median().reset_index()

In [ ]:

g = sns.distplot(foil_grouped[foil_grouped['Resp'] == 'Old']['RT'], kde =0, bins=25)
g=sns.distplot(foil_grouped[foil_grouped['Resp']=='Similar']['RT'], kde=0, bins=25)
g = sns.distplot(foil_grouped[foil_grouped['Resp'] == 'New']['RT'], kde =0, bins=25)
plt.legend(['Old', 'Similar','New'])
g.set(xlim=[400, 2050])
g.set(ylim=[0, 35])

In [ ]:
print(foil_grouped[(foil_grouped['Resp']=='Old') & (foil_grouped['RT']<600)])

In [ ]:
print(all_data[all_data['pidn']==814]['n_old_foil'])

In [ ]:
stats.ttest_ind(foil_grouped[foil_grouped['Resp'] == 'Old']['RT'], foil_grouped[foil_grouped['Resp']=='Similar']['RT'], nan_policy='propagate', equal_var=False)

In [ ]:
stats.ttest_ind(foil_grouped[foil_grouped['Resp'] == 'Old']['RT'], foil_grouped[foil_grouped['Resp']=='New']['RT'], nan_policy='propagate', equal_var=False)

In [ ]:
stats.ttest_ind(foil_grouped[foil_grouped['Resp'] == 'Similar']['RT'], foil_grouped[foil_grouped['Resp']=='New']['RT'], nan_policy='propagate', equal_var=False)

In [ ]:
row = 0
targ_data=pd.DataFrame()
for pt in (data_long['pidn']):
    if data_long.loc[row, 'Cond'] == 'Target':
        targ_data.loc[row, 'pidn'] = data_long.loc[row, 'pidn']
        targ_data.loc[row, 'RT'] = data_long.loc[row, 'RT']
        targ_data.loc[row, 'Resp'] = data_long.loc[row, 'Resp']
    row+=1
    
targ_data = targ_data[targ_data['RT']>400]
targ_data.reset_index(drop=True)    


In [ ]:
row = 0
targ_data=pd.DataFrame()
for pt in (data_long['pidn']):
    if data_long.loc[row, 'Cond'] == 'Target':
        targ_data.loc[row, 'pidn'] = data_long.loc[row, 'pidn']
        targ_data.loc[row, 'RT'] = data_long.loc[row, 'RT']
        targ_data.loc[row, 'Resp'] = data_long.loc[row, 'Resp']
    row+=1
    
targ_data = targ_data[targ_data['RT']>400]
targ_data.reset_index(drop=True)  

In [ ]:
scipy.stats.ttest_ind(targ_data[targ_data['Resp'] == 'Old']['RT'], targ_data[targ_data['Resp']=='Similar']['RT'], equal_var=False)

In [ ]:
scipy.stats.ttest_ind(targ_data[targ_data['Resp'] == 'Old']['RT'], targ_data[targ_data['Resp']=='New']['RT'], equal_var=False)

In [ ]:
scipy.stats.ttest_ind(targ_data[targ_data['Resp'] == 'Similar']['RT'], targ_data[targ_data['Resp']=='New']['RT'], equal_var=False)

In [ ]:
g = sns.distplot(targ_data[targ_data['Resp'] == 'Old']['RT'], kde=0, bins = 25)
g=sns.distplot(targ_data[targ_data['Resp']=='Similar']['RT'], kde=0, bins=25)
g= sns.distplot(targ_data[targ_data['Resp'] == 'New']['RT'], kde=0, bins = 25)
plt.legend(['Old', 'Similar', 'New'])
g.set(xlim=[400, 2050])
g.set(ylim=[0, 950])

In [ ]:
targ_grouped = targ_data.groupby(['pidn', 'Resp']).median().reset_index()

In [ ]:

g = sns.distplot(targ_grouped[targ_grouped['Resp'] == 'Old']['RT'], kde =0, bins=25)
g=sns.distplot(targ_grouped[targ_grouped['Resp']=='Similar']['RT'], kde=0, bins=25)
g = sns.distplot(targ_grouped[targ_grouped['Resp'] == 'New']['RT'], kde =0, bins=25)
plt.legend(['Old', 'Similar','New'])
g.set(xlim=[400, 2050])
g.set(ylim=[0, 35])

In [ ]:
row = 0
lure_data=pd.DataFrame()
for pt in (data_long['pidn']):
    if data_long.loc[row, 'Cond'] == 'Lure':
        lure_data.loc[row, 'pidn'] = data_long.loc[row, 'pidn']
        lure_data.loc[row, 'RT'] = data_long.loc[row, 'RT']
        lure_data.loc[row, 'Resp'] = data_long.loc[row, 'Resp']
    row+=1
    
lure_data = lure_data[lure_data['RT']>400]
lure_data.reset_index(drop=True)    


In [ ]:
scipy.stats.ttest_ind(lure_data[lure_data['Resp'] == 'Old']['RT'], lure_data[lure_data['Resp']=='Similar']['RT'], equal_var=False)

In [ ]:
scipy.stats.ttest_ind(lure_data[lure_data['Resp'] == 'Old']['RT'], lure_data[lure_data['Resp']=='New']['RT'], equal_var=False)

In [ ]:
scipy.stats.ttest_ind(lure_data[lure_data['Resp'] == 'Similar']['RT'], lure_data[lure_data['Resp']=='New']['RT'], equal_var=False)

In [ ]:
scipy.stats.ttest_ind(lure_grouped[lure_grouped['Resp'] == 'Old']['RT'], lure_grouped[lure_grouped['Resp']=='Similar']['RT'], equal_var=False)

In [ ]:
scipy.stats.ttest_ind(lure_grouped[lure_grouped['Resp'] == 'Old']['RT'], lure_grouped[lure_grouped['Resp']=='New']['RT'], equal_var=False)

In [ ]:
scipy.stats.ttest_ind(lure_grouped[lure_grouped['Resp'] == 'Similar']['RT'], lure_grouped[lure_grouped['Resp']=='New']['RT'], equal_var=False)

In [ ]:
g = sns.distplot(lure_data[lure_data['Resp'] == 'Old']['RT'], kde=0, bins = 25)
g=sns.distplot(lure_data[lure_data['Resp']=='Similar']['RT'], kde=0, bins=25)
g= sns.distplot(lure_data[lure_data['Resp'] == 'New']['RT'], kde=0, bins = 25)
plt.legend(['Old', 'Similar', 'New'])
g.set(xlim=[400, 2050])
g.set(ylim = [0, 950])

In [ ]:
scipy.stats.ttest_ind(foil_grouped[foil_grouped['Resp'] == 'Old']['RT'], foil_grouped[foil_grouped['Resp']=='Similar']['RT'], equal_var=False)

In [ ]:
scipy.stats.ttest_ind(foil_grouped[foil_grouped['Resp'] == 'Old']['RT'], foil_grouped[foil_grouped['Resp']=='New']['RT'], equal_var=False)

In [ ]:
scipy.stats.ttest_ind(foil_grouped[foil_grouped['Resp'] == 'Similar']['RT'], foil_grouped[foil_grouped['Resp']=='New']['RT'], equal_var=False)

In [ ]:
scipy.stats.ttest_ind(targ_grouped[targ_grouped['Resp'] == 'Old']['RT'], targ_grouped[targ_grouped['Resp']=='Similar']['RT'], equal_var=False)

In [ ]:
scipy.stats.ttest_ind(targ_grouped[targ_grouped['Resp'] == 'Old']['RT'], targ_grouped[targ_grouped['Resp']=='New']['RT'], equal_var=False)

In [ ]:
scipy.stats.ttest_ind(targ_grouped[targ_grouped['Resp'] == 'Similar']['RT'], targ_grouped[targ_grouped['Resp']=='New']['RT'], equal_var=False)

In [ ]:
lure_grouped = lure_data.groupby(['pidn', 'Resp']).median().reset_index()

In [ ]:

g = sns.distplot(lure_grouped[lure_grouped['Resp'] == 'Old']['RT'], kde =0, bins=25)
g=sns.distplot(lure_grouped[lure_grouped['Resp']=='Similar']['RT'], kde=0, bins=25)
g = sns.distplot(lure_grouped[lure_grouped['Resp'] == 'New']['RT'], kde =0, bins=25)
plt.legend(['Old', 'Similar','New'])
g.set(xlim=[400, 2050])
g.set(ylim=[0, 35])

In [ ]:
print([foil_grouped[foil_grouped['RT']>1950]['pidn']])

In [ ]:
print(all_data[all_data['pidn'] == 693]['n_old_foil'])

In [ ]:
trial_data.hear

In [ ]:
row = 0 
noresp_data=pd.DataFrame()
for n in trial_data:
    noresp_data.loc[row, 'target'] = ((trial_data.loc[row, 'num_nr_t'])/(trial_data.loc[row, 'targ_trials_exl_fast']))
    noresp_data.loc[row, 'lure'] =((trial_data.loc[row, 'num_nr_l'])/(trial_data.loc[row, 'lure_trials_exl_fast']))
    noresp_data.loc[row, 'foil'] = ((trial_data.loc[row, 'num_nr_f'])/(trial_data.loc[row, 'foil_trials_exl_fast']))  
    row+=1
    
print(noresp_data)


In [ ]:
sns.lmplot(x='target', y = 'lure', data=noresp_data)

In [ ]:
pearsonr(x=noresp_data['target'], y=noresp_data['lure'])

In [ ]:
pearsonr(x=noresp_data['target'], y=noresp_data['foil'])

In [ ]:
pearsonr(x=noresp_data['foil'], y=noresp_data['lure'])

In [ ]:
g=sns.distplot(noresp_data['target'], kde = 0, bins=25)
g=sns.distplot(noresp_data['lure'], kde=0, bins=25)
g=sns.distplot(noresp_data['foil'], kde=0, bins = 25)
plt.xlabel('Prop NR')


In [ ]:
g=sns.lmplot(x='t_prop_nr', y='l_prop_nr', data=trial_data)
g.set(xlim=[0, .3])

In [ ]:
pearsonr(x=trial_data['t_prop_nr'], y=trial_data['l_prop_nr'])

In [ ]:
g=sns.lmplot(x='t_prop_nr', y='f_prop_nr', data=trial_data)
g.set(xlim=[0, .35])

In [ ]:
g=sns.lmplot(x='l_prop_nr', y='f_prop_nr', data=trial_data)

In [ ]:
stats.ttest_rel(trial_data['t_prop_nr'], trial_data['l_prop_nr'], nan_policy='propagate')

In [ ]:
g=sns.distplot(trial_data['t_prop_nr'], kde = 0, bins=25)
g.set(xlim=[0, .5])
plt.xlabel('Prop NR')

In [ ]:
g=sns.distplot(trial_data['l_prop_nr'], kde = 0, bins=25, color='orange')
g.set(xlim=[0, .5])
plt.xlabel('Prop NR')

In [ ]:
g=sns.distplot(trial_data['f_prop_nr'], kde = 0, bins=25, color='green')
g.set(xlim=[0, .5])
plt.xlabel('Prop NR')

In [ ]:
chisquare([trial_data['t_prop_nr'], trial_data['l_prop_nr'], trial_data['f_prop_nr']])

In [ ]:
cond_data = pd.DataFrame()
row = 0
row1 = 0
for x in trial_data:
    cond_data.loc[row, 'pidn'] == trial_data.loc[row1, 'pidn']
    cond_data.loc[row+1, 'pidn'] == trial_data.loc[row1, 'pidn']
    cond_data.loc[row+2, 'pidn'] == trial_data.loc[row1, 'pidn']
    cond_data.loc[row, 'Cond'] = 'T'
    cond_data.loc[row, 'NR'] = (all_data.loc[row1, 'num_nr_t']/64)
    cond_data.loc[row, 'corr'] = all_data.loc[row1, 'n_old_targ']/trial_data.loc[row1, 'targ_trials_exl_nr_fast']
    cond_data.loc[row+1, 'Cond'] = 'L'
    cond_data.loc[row+2, 'NR'] = (all_data.loc[row1, 'num_nr_l']/64)
    cond_data.loc[row+1, 'corr'] = all_data.loc[row1, 'n_sim_lure']/trial_data.loc[row1, 'lure_trials_exl_nr_fast']
    cond_data.loc[row+2, 'Cond'] = 'F'
    cond_data.loc[row+2, 'NR'] = (all_data.loc[row1, 'num_nr_f']/64)
   
    cond_data.loc[row+2, 'corr'] = all_data.loc[row1, 'n_new_foil']/trial_data.loc[row1, 'foil_trials_exl_nr_fast']
    row=row+3
    row1+=1
cond_data.head()

In [ ]:
g=sns.lmplot(x='NR', y='corr', data= cond_data, hue='Cond', height=7)
plt.ylabel('Percent Correct')

In [ ]:
t_cond = cond_data[(cond_data['Cond']=='T')]
l_cond = cond_data[cond_data['Cond']=='L']
f_cond= cond_data[cond_data['Cond']=='F']

t_cond= t_cond[:-3]
l_cond= l_cond[:-3]
f_cond= f_cond[:-3]

In [ ]:
pearsonr(x=t_cond['NR'], y =t_cond['corr'])

In [ ]:
pearsonr(x=l_cond['NR'], y =l_cond['corr'])

In [ ]:
pearsonr(x=f_cond['NR'], y =f_cond['corr'])

In [ ]:
pearsonr(x=cond_data[cond_data['Cond']=='L']['NR'], y =cond_data[cond_data['Cond']=='L']['corr'])

In [ ]:
pearsonr(x=cond_data[cond_data['Cond']=='F']['NR'], y =cond_data[cond_data['Cond']=='F']['corr'])

In [ ]:
g = sns.lmplot( x='NR', y = 'd_o_n', data=cond_data, hue = 'Cond', height=7)
plt.ylabel('d\' Old/New')


In [ ]:
pearsonr(x=t_cond['NR'], y=t_cond['d_o_n'])

In [ ]:
pearsonr(x=l_cond['NR'], y=l_cond['d_o_n'])

In [ ]:
pearsonr(x=f_cond['NR'], y=f_cond['d_o_n'])

In [ ]:
sns.lmplot( x='NR', y = 'd_o_s', data=cond_data, hue = 'Cond', height=7)
plt.ylabel('d\' Old/Similar')

In [ ]:
sns.lmplot( x='NR', y = 'd_bps', data=cond_data, hue = 'Cond', height=7)
plt.ylabel('d\' BPS')

In [21]:
rt_data = data_long

In [ ]:
print(np_data[np_data['pidn']==1230]['ir_comp'])

In [ ]:
g = sns.distplot(rt_data[(rt_data['pidn'] == 528) & (rt_data['Cond']=='Target')]['RT'], kde = 0, bins = 25)
g = sns.distplot(rt_data[(rt_data['pidn'] == 528) & (rt_data['Cond']=='Lure')]['RT'], kde = 0, bins =25)
g = sns.distplot(rt_data[(rt_data['pidn'] == 528) & (rt_data['Cond']=='Foil')]['RT'], kde = 0, bins = 25)
g.set(ylim=[0,15])

In [ ]:
g = sns.distplot(rt_data[(rt_data['pidn'] ==673) & (rt_data['Cond']=='Target')]['RT'], kde = 0, bins = 25)
g = sns.distplot(rt_data[(rt_data['pidn'] == 673) & (rt_data['Cond']=='Lure')]['RT'], kde = 0, bins =25)
g = sns.distplot(rt_data[(rt_data['pidn'] == 673) & (rt_data['Cond']=='Foil')]['RT'], kde = 0, bins = 25)
g.set(ylim=[0,15])

In [ ]:
g = sns.distplot(rt_data[(rt_data['pidn'] == 1185) & (rt_data['Cond']=='Target')]['RT'], kde = 0, bins = 25)
g = sns.distplot(rt_data[(rt_data['pidn'] ==1185) & (rt_data['Cond']=='Lure')]['RT'], kde = 0, bins =25)
g = sns.distplot(rt_data[(rt_data['pidn'] == 1185) & (rt_data['Cond']=='Foil')]['RT'], kde = 0, bins = 25)
g.set(ylim=[0,15])

In [ ]:
sns.distplot(rt_data[rt_data['Cond'] == 'Target']['RT'], kde = 0, bins = 50)
sns.distplot(rt_data[rt_data['Cond'] == 'Lure']['RT'], kde = 0, bins = 50)
sns.distplot(rt_data[rt_data['Cond'] == 'Foil']['RT'], kde = 0, bins = 50)

In [ ]:
print(len(rt_data[rt_data['RT']<500]['pidn']))

In [ ]:
g=sns.distplot(rt_data[(rt_data['Cond'] == 'Target') & (rt_data['Acc']=='Correct')]['RT'], kde = 0, bins = 50)
g=sns.distplot(rt_data[(rt_data['Cond'] == 'Lure')& (rt_data['Acc']=='Correct')]['RT'], kde = 0, bins = 50)
g=sns.distplot(rt_data[(rt_data['Cond'] == 'Foil') & (rt_data['Acc']=='Correct')]['RT'], kde = 0,bins = 50)
g.set(ylim = (0, 20))
g.set(xlim = (0, 700))

In [ ]:
print(rt_data[(rt_data['Acc']=='Incorrect') &( rt_data['RT']<600)]['pidn'])

In [ ]:
scipy.stats.ttest_ind(rt_data[(rt_data['Cond'] == 'Target') & (rt_data['Acc']=='Correct')]['RT'], rt_data[(rt_data['Cond'] == 'Lure')& (rt_data['Acc']=='Correct')]['RT'], equal_var=False)

In [ ]:
scipy.stats.ttest_ind(rt_data[(rt_data['Cond'] == 'Target') & (rt_data['Acc']=='Correct')]['RT'], rt_data[(rt_data['Cond'] == 'Foil')& (rt_data['Acc']=='Correct')]['RT'], equal_var=False)

In [ ]:
scipy.stats.ttest_ind(rt_data[(rt_data['Cond'] == 'Lure') & (rt_data['Acc']=='Correct')]['RT'], rt_data[(rt_data['Cond'] == 'Foil')& (rt_data['Acc']=='Correct')]['RT'], equal_var=False)

In [ ]:
scipy.stats.ttest_ind(rt_data[(rt_data['Cond'] == 'Target') & (rt_data['Acc']=='Incorrect')]['RT'], rt_data[(rt_data['Cond'] == 'Lure')& (rt_data['Acc']=='Incorrect')]['RT'], equal_var=False)

In [ ]:
scipy.stats.ttest_ind(rt_data[(rt_data['Cond'] == 'Target') & (rt_data['Acc']=='Incorrect')]['RT'], rt_data[(rt_data['Cond'] == 'Foil')& (rt_data['Acc']=='Incorrect')]['RT'], equal_var=False)

In [ ]:
scipy.stats.ttest_ind(rt_data[(rt_data['Cond'] == 'Lure') & (rt_data['Acc']=='Incorrect')]['RT'], rt_data[(rt_data['Cond'] == 'Foil')& (rt_data['Acc']=='Incorrect')]['RT'], equal_var=False)

In [ ]:
g = sns.distplot(rt_data[(rt_data['Cond'] == 'Target') & (rt_data['Acc']=='Incorrect')]['RT'], kde = 0, bins = 50)
g= sns.distplot(rt_data[(rt_data['Cond'] == 'Lure')& (rt_data['Acc']=='Incorrect')]['RT'], kde = 0, bins = 50)
g= sns.distplot(rt_data[(rt_data['Cond'] == 'Foil') & (rt_data['Acc']=='Incorrect')]['RT'], kde = 0,  bins = 50)
g.set(ylim = (0, 20))
g.set(xlim = (0, 700))

In [ ]:
avg_data = rt_data.groupby(['pidn', 'Cond']).mean().reset_index()
avg_data = avg_data.merge(all_data, on = 'pidn')
print(avg_data)

In [ ]:
sns.distplot(avg_data[(avg_data['Cond'] == 'Target')]['RT'], kde= 0, bins = 50)
sns.distplot(avg_data[(avg_data['Cond'] == 'Lure')]['RT'], kde= 0, bins = 50)
sns.distplot(avg_data[(avg_data['Cond'] == 'Foil')]['RT'], kde= 0, bins = 50)

In [ ]:
sns.distplot(avg_data[(avg_data['Cond'] == 'Target') & (avg_data['Acc'] == 'Correct')]['RT'], kde= 0, bins = 50)
sns.distplot(avg_data[(avg_data['Cond'] == 'Lure')& (avg_data['Acc'] == 'Correct')]['RT'], kde= 0, bins = 50)
sns.distplot(avg_data[(avg_data['Cond'] == 'Foil')& (avg_data['Acc'] == 'Correct')]['RT'], kde= 0, bins = 50)

In [ ]:
g = sns.distplot(avg_data[(avg_data['Cond'] == 'Target') & (avg_data['Acc'] == 'Incorrect')]['RT'], kde= 0, bins = 50)
g = sns.distplot(avg_data[(avg_data['Cond'] == 'Lure')& (avg_data['Acc'] == 'Incorrect')]['RT'], kde= 0, bins = 50)
g= sns.distplot(avg_data[(avg_data['Cond'] == 'Foil')& (avg_data['Acc'] == 'Incorrect')]['RT'], kde= 0, bins = 50)
g.set(ylim = [0, 20])

In [22]:
med_data = rt_data.groupby(['pidn','Cond', 'Resp']).median().reset_index()
med_data=med_data.merge(data_adj, on=['pidn', 'Cond'])
med_data=med_data.merge(all_data, on=['pidn'])
print(med_data)

NameError: name 'data_adj' is not defined

In [ ]:
sns.lmplot(x= 'RT', y='correct', data=med_data, hue='Cond', hue_order=['Target', 'Lure', 'Foil'], height=7)

In [ ]:
pearsonr(x=med_data[med_data['Cond']=='Lure']['correct'], y = med_data[med_data['Cond']=='Lure']['RT'])

In [ ]:
print(med_data[med_data['pidn']==1185])

In [ ]:
sns.distplot(med_data['RT'], kde=0, bins=50)
g.set(ylim=[0, 60])

In [ ]:
g = sns.distplot(med_data[(med_data['Acc'] == 'Correct')]['RT'], kde= 0, bins = 50)
g = sns.distplot(med_data[(med_data['Acc'] == 'Incorrect')]['RT'], kde= 0, bins = 50)
plt.legend(['Correct', 'Incorrect'])
g.set(ylim=[0, 60])

In [ ]:
sns.distplot(med_data[(med_data['Cond'] == 'Target')]['RT'], kde= 0, bins = 50)
sns.distplot(med_data[(med_data['Cond'] == 'Lure')]['RT'], kde= 0, bins = 50)
sns.distplot(med_data[(med_data['Cond'] == 'Foil')]['RT'], kde= 0, bins = 50)

In [ ]:
scipy.stats.ttest_ind(med_data[(med_data['Cond'] == 'Target')]['RT'], med_data[(med_data['Cond'] == 'Lure')]['RT'], equal_var=False)

In [ ]:
scipy.stats.ttest_ind(med_data[(med_data['Cond'] == 'Target')]['RT'], med_data[(med_data['Cond'] == 'Foil')]['RT'], equal_var=False)

In [ ]:
scipy.stats.ttest_ind(med_data[(med_data['Cond'] == 'Lure')]['RT'], med_data[(med_data['Cond'] == 'Foil')]['RT'], equal_var=False)

In [ ]:
print(all_data[all_data['pidn']==1185])


In [ ]:
print(data_long[(data_long['pidn'] ==1185) & (data_long['RT']>400) & (data_long['Cond'] == 'Lure')])

In [ ]:
g = sns.distplot(med_data[(med_data['Cond'] == 'Target') & (med_data['Acc'] == 'Correct')]['RT'], kde= 0, bins = 50)
g = sns.distplot(med_data[(med_data['Cond'] == 'Lure')& (med_data['Acc'] == 'Correct')]['RT'], kde= 0, bins = 50)
g = sns.distplot(med_data[(med_data['Cond'] == 'Foil')& (med_data['Acc'] == 'Correct')]['RT'], kde= 0, bins = 50)
g.set(ylim=[0, 20])

In [ ]:
g = sns.distplot(med_data[(med_data['Cond'] == 'Target') & (med_data['Acc'] == 'Incorrect')]['RT'], kde= 0, bins = 50)
g = sns.distplot(med_data[(med_data['Cond'] == 'Lure')& (med_data['Acc'] == 'Incorrect')]['RT'], kde= 0, bins = 50)
g = sns.distplot(med_data[(med_data['Cond'] == 'Foil')& (med_data['Acc'] == 'Incorrect')]['RT'], kde= 0, bins = 50)
g.set(ylim=[0, 20])
g.set(xlim=[400, 2050])

In [ ]:

lure_med_nr = med_data[med_data['Cond']=='Lure']
print(lure_med_nr)

In [ ]:
sns.lmplot(x='RT', y = 'dprime_bps', data= lure_med_nr)

In [ ]:
pearsonr(x=lure_med_nr['RT'], y=lure_med_nr['dprime_old_lure'])

In [ ]:
print(all_data[all_data['dprime_old_new']<1])

In [ ]:
sns.lmplot(x= 'prop_nr', y='RT', data=high_nr_med, hue='Cond', hue_order=['Target', 'Lure', 'Foil'], height=7)

In [ ]:
t_med = high_nr_med[high_nr_med['Cond']=='Target']
l_med = high_nr_med[high_nr_med['Cond']=='Lure']
f_med = high_nr_med[high_nr_med['Cond']=='Foil']

In [ ]:
pearsonr(x=t_med['prop_nr'], y = t_med['RT'])

In [ ]:
pearsonr(x=f_med['prop_nr'], y = f_med['RT'])

In [ ]:
pearsonr(x=l_med['prop_nr'], y = l_med['RT'])

In [ ]:
pearsonr (x = high_nr_med['prop_nr'], y = high_nr_med['RT']) 

In [ ]:
high_nr_avg = avg_data[avg_data['prop_nr'] > .05]
print(high_nr_avg)

In [ ]:
sns.lmplot(x= 'prop_nr', y='RT', data=high_nr_avg, hue='Cond', height=7)

In [ ]:
g= sns.lmplot(x = 'prop_nr', y = 'dprime_old_new', data= avg_data)

In [ ]:
pearsonr(x=avg_data['prop_nr'], y=avg_data['dprime_old_new'])

In [ ]:
pearsonr(x=avg_data['prop_nr'], y=avg_data['dprime_bps'])

In [ ]:
pearsonr(x=avg_data['prop_nr'], y=avg_data['dprime_old_lure'])

In [ ]:
g= sns.lmplot(x = 'prop_nr', y = 'dprime_bps', data= high_nr_avg)

In [ ]:
g= sns.lmplot(x = 'prop_nr', y = 'dprime_old_lure', data= high_nr_avg)

In [ ]:
high_nr_med = med_data[med_data['prop_nr']>.05]

In [ ]:
d= pd.DataFrame()
row=0
for sub in pd.unique(all_data['pidn']):
    d.loc[row, 'pidn'] = sub
    d.loc[row, 'num_nr'] = len(data_long[(data_long['pidn']== sub) & (data_long['RT'] == 0)])
    d.loc[row, 'num_fast'] = len(data_long[(data_long['pidn']== sub) & (data_long['RT'] < 400) & (data_long['RT'] != 0)])
    d.loc[row, 'num_tr_no_fast'] = 64-d.loc[row, 'num_fast']
    d.loc[row, 'pr_nr'] = d.loc[row, 'num_nr']/d.loc[row, 'num_tr_no_fast']
    row+=1
    
d = d.merge(all_data, on= 'pidn')

In [ ]:
sns.lmplot(x='age', y = 'pr_nr', data=d)

In [ ]:
pearsonr(x=d['age'], y=d['pr_nr'])

In [ ]:
sns.lmplot(x='num_nr', y = 'num_fast', data=d)

In [ ]:
pearsonr(x=d['num_nr'], y=d['num_fast'])

In [ ]:
pidns = pd.unique(data_long['pidn'])
print(pidns)

In [ ]:
no_nr = data_long[data_long['RT']!=0]
no_nr.reset_index(inplace=True, drop=True)
#print(no_nr)

In [ ]:
sns.distplot(trial_data['foil_trials_exl_fast'], kde=False)

In [ ]:
sns.distplot(trial_data['foil_trials_exl_nr'], kde=False)

In [ ]:
sns.distplot(trial_data['foil_trials_exl_nr_fast'], kde=False)

In [ ]:
sns.distplot(trial_data['targ_trials_exl_fast'], kde=False)

In [ ]:
sns.distplot(trial_data['lure_trials_exl_nr'], kde=False)

In [ ]:
sns.distplot(trial_data['lure_trials_exl_nr_fast'], kde=False)

In [ ]:
sns.distplot(trial_data['lure_trials_exl_fast'], kde=False)

In [ ]:
sns.distplot(trial_data['targ_trials_exl_nr'], kde=False)

In [ ]:
sns.distplot(trial_data['targ_trials_exl_nr_fast'], kde=False)

In [ ]:
sns.distplot(trial_data['trials_exl_nr_fast'], kde=False)

In [ ]:
sns.distplot(trial_data['trials_exl_fast'], kde=False)

In [ ]:
sns.distplot(trial_data['trials_exl_nr'], kde=False)

In [ ]:
print(all_data[all_data['pidn'] == 1185])

In [ ]:
mid_nr_3 = data_long[(data_long['pidn']==929)]

In [ ]:
sns.distplot(mid_nr_3['RT'], kde = False)

In [ ]:
sns.distplot(mid_nr_3[(mid_nr_3['Acc'] == 'Correct' )]['RT'], kde = False)

In [ ]:
sns.distplot(mid_nr_3[(mid_nr_3['Acc'] == 'Incorrect' )]['RT'], kde = False)

In [ ]:
sns.distplot(mid_nr_3[(mid_nr_3['Cond'] == 'Target' )]['RT'], kde = False)

In [ ]:
sns.distplot(mid_nr_3[(mid_nr_3['Cond'] == 'Target' ) & (mid_nr_3['Acc'] == 'Correct')]['RT'], kde = False)

In [ ]:
sns.distplot(mid_nr_3[(mid_nr_3['Cond'] == 'Target' ) & (mid_nr_3['Acc'] == 'Incorrect')]['RT'], kde = False)

In [ ]:
sns.distplot(mid_nr_3[(mid_nr_3['Cond'] == 'Lure' )]['RT'], kde = False)

In [ ]:
sns.distplot(mid_nr_3[(mid_nr_3['Cond'] == 'Lure' ) & (mid_nr_3['Acc'] == 'Correct')]['RT'], kde = False)

In [ ]:
sns.distplot(mid_nr_3[(mid_nr_3['Cond'] == 'Lure' ) & (mid_nr_3['Acc'] == 'Incorrect')]['RT'], kde = False)


In [ ]:
sns.distplot(mid_nr_3[(mid_nr_3['Cond'] == 'Foil' )]['RT'], kde = False)

In [ ]:
sns.distplot(mid_nr_3[(mid_nr_3['Cond'] == 'Foil' ) & (mid_nr_3['Acc'] == 'Correct')]['RT'], kde = False)

In [ ]:
sns.distplot(mid_nr_3[(mid_nr_3['Cond'] == 'Foil' ) & (mid_nr_3['Acc'] == 'Incorrect')]['RT'], kde = False)

In [ ]:
mid_nr_2 = data_long[(data_long['pidn']==859)]

In [ ]:
sns.distplot(mid_nr_2['RT'], kde = False)

In [ ]:
sns.distplot(mid_nr_2[(mid_nr_2['Acc'] == 'Correct' )]['RT'], kde = False)

In [ ]:
sns.distplot(mid_nr_2[(mid_nr_2['Acc'] == 'Incorrect' )]['RT'], kde = False)

In [ ]:
sns.distplot(mid_nr_2[(mid_nr_2['Cond'] == 'Target' )]['RT'], kde = False)

In [ ]:
sns.distplot(mid_nr_2[(mid_nr_2['Cond'] == 'Target' ) & (mid_nr_2['Acc'] == 'Correct')]['RT'], kde = False)

In [ ]:
sns.distplot(mid_nr_2[(mid_nr_2['Cond'] == 'Target' ) & (mid_nr_2['Acc'] == 'Incorrect')]['RT'], kde = False)

In [ ]:
sns.distplot(mid_nr_2[(mid_nr_2['Cond'] == 'Lure' )]['RT'], kde = False)

In [ ]:
sns.distplot(mid_nr_2[(mid_nr_2['Cond'] == 'Lure' ) & (mid_nr_2['Acc'] == 'Correct')]['RT'], kde = False)

In [ ]:
sns.distplot(mid_nr_2[(mid_nr_2['Cond'] == 'Lure' ) & (mid_nr_2['Acc'] == 'Incorrect')]['RT'], kde = False)


In [ ]:
sns.distplot(mid_nr_2[(mid_nr_2['Cond'] == 'Foil' )]['RT'], kde = False)

In [ ]:
sns.distplot(mid_nr_2[(mid_nr_2['Cond'] == 'Foil' ) & (mid_nr_2['Acc'] == 'Correct')]['RT'], kde = False)

In [ ]:
sns.distplot(mid_nr_2[(mid_nr_2['Cond'] == 'Foil' ) & (mid_nr_2['Acc'] == 'Incorrect')]['RT'], kde = False)

In [ ]:
mid_nr_1 = data_long[(data_long['pidn']==721)]

In [ ]:
sns.distplot(mid_nr_1['RT'], kde = False)

In [ ]:
sns.distplot(mid_nr_1[(mid_nr_1['Acc'] == 'Correct' )]['RT'], kde = False)

In [ ]:
sns.distplot(mid_nr_1[(mid_nr_1['Acc'] == 'Incorrect' )]['RT'], kde = False)

In [ ]:
sns.distplot(mid_nr_1[(mid_nr_1['Cond'] == 'Target' )]['RT'], kde = False)

In [ ]:
sns.distplot(mid_nr_1[(mid_nr_1['Cond'] == 'Target' ) & (mid_nr_1['Acc'] == 'Correct')]['RT'], kde = False)

In [ ]:
sns.distplot(mid_nr_1[(mid_nr_1['Cond'] == 'Target' ) & (mid_nr_1['Acc'] == 'Incorrect')]['RT'], kde = False)

In [ ]:
sns.distplot(mid_nr_1[(mid_nr_1['Cond'] == 'Lure' )]['RT'], kde = False)

In [ ]:
sns.distplot(mid_nr_1[(mid_nr_1['Cond'] == 'Lure' ) & (mid_nr_1['Acc'] == 'Correct')]['RT'], kde = False)

In [ ]:
sns.distplot(mid_nr_1[(mid_nr_1['Cond'] == 'Lure' ) & (mid_nr_1['Acc'] == 'Incorrect')]['RT'], kde = False)


In [ ]:
sns.distplot(mid_nr_1[(mid_nr_1['Cond'] == 'Foil' )]['RT'], kde = False)

In [ ]:
sns.distplot(mid_nr_1[(mid_nr_1['Cond'] == 'Foil' ) & (mid_nr_1['Acc'] == 'Correct')]['RT'], kde = False)

In [ ]:
sns.distplot(mid_nr_1[(mid_nr_1['Cond'] == 'Foil' ) & (mid_nr_1['Acc'] == 'Incorrect')]['RT'], kde = False)

In [ ]:
high_nr_3 = data_long[(data_long['pidn']==1045)]

In [ ]:
sns.distplot(high_nr_3['RT'], kde = False)

In [ ]:
sns.distplot(high_nr_3[(high_nr_3['Acc'] == 'Correct' )]['RT'], kde = False)

In [ ]:
sns.distplot(high_nr_3[(high_nr_3['Acc'] == 'Incorrect' )]['RT'], kde = False)

In [ ]:
sns.distplot(high_nr_3[(high_nr_3['Cond'] == 'Target' )]['RT'], kde = False)

In [ ]:
sns.distplot(high_nr_3[(high_nr_3['Cond'] == 'Target' ) & (high_nr_3['Acc'] == 'Correct')]['RT'], kde = False)

In [ ]:
sns.distplot(high_nr_3[(high_nr_3['Cond'] == 'Target' ) & (high_nr_3['Acc'] == 'Incorrect')]['RT'], kde = False)

In [ ]:
sns.distplot(high_nr_3[(high_nr_3['Cond'] == 'Lure' )]['RT'], kde = False)

In [ ]:
sns.distplot(high_nr_3[(high_nr_3['Cond'] == 'Lure' ) & (high_nr_3['Acc'] == 'Correct')]['RT'], kde = False)

In [ ]:
sns.distplot(high_nr_3[(high_nr_3['Cond'] == 'Lure' ) & (high_nr_3['Acc'] == 'Incorrect')]['RT'], kde = False)

In [ ]:
sns.distplot(high_nr_3[(high_nr_3['Cond'] == 'Foil' )]['RT'], kde = False)

In [ ]:
sns.distplot(high_nr_3[(high_nr_3['Cond'] == 'Foil' ) & (high_nr_3['Acc'] == 'Correct')]['RT'], kde = False)

In [ ]:
sns.distplot(high_nr_3[(high_nr_3['Cond'] == 'Foil' ) & (high_nr_3['Acc'] == 'Incorrect')]['RT'], kde = False)

In [ ]:
high_nr_2 = data_long[(data_long['pidn']==987)]

In [ ]:
sns.distplot(high_nr_2['RT'], kde = False)

In [ ]:
sns.distplot(high_nr_2[(high_nr_2['Acc'] == 'Correct' )]['RT'], kde = False)

In [ ]:
sns.distplot(high_nr_2[(high_nr_2['Acc'] == 'Incorrect' )]['RT'], kde = False)

In [ ]:
sns.distplot(high_nr_2[(high_nr_2['Cond'] == 'Target' )]['RT'], kde = False)

In [ ]:
sns.distplot(high_nr_2[(high_nr_2['Cond'] == 'Target' ) & (high_nr_2['Acc'] == 'Correct')]['RT'], kde = False)

In [ ]:
sns.distplot(high_nr_2[(high_nr_2['Cond'] == 'Target' ) & (high_nr_2['Acc'] == 'Incorrect')]['RT'], kde = False)

In [ ]:
sns.distplot(high_nr_2[(high_nr_2['Cond'] == 'Lure' )]['RT'], kde = False)

In [ ]:
sns.distplot(high_nr_2[(high_nr_2['Cond'] == 'Lure' ) & (high_nr_2['Acc'] == 'Correct')]['RT'], kde = False)

In [ ]:
sns.distplot(high_nr_2[(high_nr_2['Cond'] == 'Lure' ) & (high_nr_2['Acc'] == 'Incorrect')]['RT'], kde = False)

In [ ]:
sns.distplot(high_nr_2[(high_nr_2['Cond'] == 'Foil' )]['RT'], kde = False)

In [ ]:
sns.distplot(high_nr_2[(high_nr_2['Cond'] == 'Foil' ) & (high_nr_2['Acc'] == 'Correct')]['RT'], kde = False)

In [ ]:
sns.distplot(high_nr_2[(high_nr_2['Cond'] == 'Foil' ) & (high_nr_2['Acc'] == 'Incorrect')]['RT'], kde = False)

In [ ]:
high_nr_1 = data_long[(data_long['pidn']==732)]

In [ ]:
sns.distplot(high_nr_1['RT'], kde = False)

In [ ]:
sns.distplot(high_nr_1[high_nr_1['Acc'] == 'Correct']['RT'], kde = False)

In [ ]:
sns.distplot(high_nr_1[high_nr_1['Acc'] == 'Incorrect']['RT'], kde = False)

In [ ]:
sns.distplot(high_nr_1[(high_nr_1['Cond'] == 'Target' )]['RT'], kde = False)

In [ ]:
sns.distplot(high_nr_1[(high_nr_1['Cond'] == 'Target' )& (high_nr_1['Acc'] == 'Correct')]['RT'], kde = False)

In [ ]:
sns.distplot(high_nr_1[(high_nr_1['Cond'] == 'Target' )& (high_nr_1['Acc'] == 'Incorrect')]['RT'], kde = False)

In [ ]:
sns.distplot(high_nr_1[(high_nr_1['Cond'] == 'Lure' )]['RT'], kde = False)

In [ ]:
sns.distplot(high_nr_1[(high_nr_1['Cond'] == 'Lure' )& (high_nr_1['Acc'] == 'Correct')]['RT'], kde = False)

In [ ]:
sns.distplot(high_nr_1[(high_nr_1['Cond'] == 'Lure' )& (high_nr_1['Acc'] == 'Incorrect')]['RT'], kde = False)

In [ ]:
sns.distplot(high_nr_1[(high_nr_1['Cond'] == 'Foil' )]['RT'], kde = False)

In [ ]:
sns.distplot(high_nr_1[(high_nr_1['Cond'] == 'Foil' )& (high_nr_1['Acc'] == 'Correct')]['RT'], kde = False)

In [ ]:
sns.distplot(high_nr_1[(high_nr_1['Cond'] == 'Foil' )& (high_nr_1['Acc'] == 'Incorrect')]['RT'], kde = False)

In [ ]:
no_nr_2 = data_long[(data_long['pidn'] == 534)]

In [ ]:
sns.distplot(no_nr_2['RT'], kde = False)

In [ ]:
sns.distplot(no_nr_2[no_nr_2['Acc'] == 'Correct']['RT'], kde = False)

In [ ]:
sns.distplot(no_nr_2[no_nr_2['Acc'] == 'Incorrect']['RT'], kde = False)

In [ ]:
sns.distplot(no_nr_2[(no_nr_2['Cond'] == 'Target' )]['RT'], kde = False)

In [ ]:
sns.distplot(no_nr_2[(no_nr_2['Cond'] == 'Target' )& (no_nr_2['Acc'] == 'Correct')]['RT'], kde = False)

In [ ]:
sns.distplot(no_nr_2[(no_nr_2['Cond'] == 'Target' )& (no_nr_2['Acc'] == 'Incorrect')]['RT'], kde = False)

In [ ]:
sns.distplot(no_nr_2[(no_nr_2['Cond'] == 'Lure' )]['RT'], kde = False)

In [ ]:
sns.distplot(no_nr_2[(no_nr_2['Cond'] == 'Lure' )& (no_nr_2['Acc'] == 'Correct')]['RT'], kde = False)

In [ ]:
sns.distplot(no_nr_2[(no_nr_2['Cond'] == 'Lure' )& (no_nr_2['Acc'] == 'Incorrect')]['RT'], kde = False)

In [ ]:
sns.distplot(no_nr_2[(no_nr_2['Cond'] == 'Foil' )]['RT'], kde = False)

In [ ]:
sns.distplot(no_nr_2[(no_nr_2['Cond'] == 'Foil' )& (no_nr_2['Acc'] == 'Correct')]['RT'], kde = False)

In [ ]:
sns.distplot(no_nr_2[(no_nr_2['Cond'] == 'Foil' )& (no_nr_2['Acc'] == 'Incorrect')]['RT'], kde = False)

In [ ]:
no_nr_1 = data_long[(data_long['pidn'] == 532)]

In [ ]:
sns.distplot(no_nr_1['RT'], kde = False)

In [ ]:
sns.distplot(no_nr_1[no_nr_1['Acc'] == 'Correct']['RT'], kde = False)

In [ ]:
sns.distplot(no_nr_1[no_nr_1['Acc'] == 'Incorrect']['RT'], kde = False)

In [ ]:
sns.distplot(no_nr_1[no_nr_1['Cond'] == 'Target']['RT'], kde = False)

In [ ]:
sns.distplot(no_nr_1[(no_nr_1['Cond'] == 'Target' )& (no_nr_1['Acc'] == 'Correct')]['RT'], kde = False)

In [ ]:
sns.distplot(no_nr_1[(no_nr_1['Cond'] == 'Target' )& (no_nr_1['Acc'] == 'Incorrect')]['RT'], kde = False)

In [ ]:
sns.distplot(no_nr_1[no_nr_1['Cond'] == 'Lure']['RT'], kde = False)

In [ ]:
sns.distplot(no_nr_1[(no_nr_1['Cond'] == 'Lure' )& (no_nr_1['Acc'] == 'Correct')]['RT'], kde = False)

In [ ]:
sns.distplot(no_nr_1[(no_nr_1['Cond'] == 'Lure' )& (no_nr_1['Acc'] == 'Incorrect')]['RT'], kde = False)

In [ ]:
sns.distplot(no_nr_1[no_nr_1['Cond'] == 'Foil']['RT'], kde = False)

In [ ]:
sns.distplot(no_nr_1[(no_nr_1['Cond'] == 'Foil' )& (no_nr_1['Acc'] == 'Correct')]['RT'], kde = False)

In [ ]:
sns.distplot(no_nr_1[(no_nr_1['Cond'] == 'Foil' )& (no_nr_1['Acc'] == 'Incorrect')]['RT'], kde = False)

In [ ]:
pd.unique(all_data[(all_data['prop_nr']>.1) & (all_data['prop_nr']<.12)]['pidn'])

In [ ]:
#truncate data for non-real reaction times
data_long.loc[data_long['RT'] > 2000, 'RT'] = 2000

data_long = data_long[(data_long.RT >400) | (data_long.RT ==0)]
data_long.reset_index(inplace=True)
print(data_long)

In [ ]:
data_med_RT = data_long.groupby(['pidn','Cond', 'Resp']).median().reset_index()
data_med_RT = data_med_RT.merge(all_data, on='pidn')
print(data_med_RT)

In [ ]:
print(data_med_RT[data_med_RT['pidn']==1045])

In [ ]:
sns.lmplot(x='RT', y= 'dprime_old_lure', data = data_med_RT[data_med_RT['Cond']=='Lure'], hue_order=['Target', 'Lure', 'Foil'])

In [ ]:
sns.distplot(data_med_RT['RT'], kde=False)

In [ ]:
sns.distplot(data_med_RT[data_med_RT['Acc']=='Correct']['RT'], kde = False)

In [ ]:
sns.distplot(data_med_RT[data_med_RT['Acc']=='Incorrect']['RT'], kde = False)

In [ ]:
sns.distplot(data_med_RT[data_med_RT['Cond']=='Target']['RT'], kde = False)

In [ ]:
sns.distplot(data_med_RT[(data_med_RT['Cond']=='Target') & (data_med_RT['Acc']=='Correct') ]['RT'], kde = False)

In [ ]:
sns.distplot(data_med_RT[(data_med_RT['Cond']=='Target') & (data_med_RT['Acc']=='Incorrect') ]['RT'], kde = False)

In [ ]:
sns.distplot(data_med_RT[data_med_RT['Cond']=='Lure']['RT'], kde = False)

In [ ]:
sns.distplot(data_med_RT[(data_med_RT['Cond']=='Lure') & (data_med_RT['Acc']=='Correct') ]['RT'], kde = False)

In [ ]:
sns.distplot(data_med_RT[(data_med_RT['Cond']=='Lure') & (data_med_RT['Acc']=='Incorrect') ]['RT'], kde = False)

In [ ]:
sns.distplot(data_med_RT[data_med_RT['Cond']=='Foil']['RT'], kde = False)

In [ ]:
sns.distplot(data_med_RT[(data_med_RT['Cond']=='Foil') & (data_med_RT['Acc']=='Correct') ]['RT'], kde = False)

In [ ]:
sns.distplot(data_med_RT[(data_med_RT['Cond']=='Foil') & (data_med_RT['Acc']=='Incorrect') ]['RT'], kde = False)

In [ ]:
med_RT = data_long.groupby(['pidn']).median().reset_index()
med_RT = med_RT.merge(all_data, on='pidn')
print(med_RT[med_RT['RT']<1000])

In [ ]:
sns.lmplot(x='RT', y='dprime_bps', data=med_RT)

In [ ]:
pearsonr(x=med_RT['RT'], y=med_RT['dprime_bps'])

In [ ]:
pearsonr(x=med_RT['RT'], y=med_RT['dprime_old_new'])

In [ ]:
pearsonr(x=med_RT['RT'], y=med_RT['dprime_old_lure'])

In [ ]:
sns.lmplot(x='RT', y='dprime_old_new', data=med_RT)

In [ ]:
sns.lmplot(x='RT', y='dprime_old_lure', data=med_RT)

In [ ]:
sns.lmplot(x='age', y='RT', data=med_RT)

In [ ]:
c_med_RT = data_long.groupby(['pidn', 'Acc']).median().reset_index()
c_med_RT = c_med_RT.merge(all_data, on='pidn')
print(c_med_RT)

In [ ]:
sns.lmplot(x='age', y='RT', data=(c_med_RT[c_med_RT['Acc']=='Correct']))

In [ ]:
pearsonr(x=c_med_RT[c_med_RT['Acc']=='Correct']['RT'], y=c_med_RT[c_med_RT['Acc']=='Correct']['age'])

In [ ]:
pearsonr(x=med_RT['RT'], y=med_RT['age'])

In [ ]:
pearsonr(x=c_med_RT[c_med_RT['Acc']=='Incorrect']['RT'], y=c_med_RT[c_med_RT['Acc']=='Incorrect']['age'])

In [ ]:
pearsonr(x=c_med_RT[(c_med_RT['Acc']=='Incorrect') & (c_med_RT['Cond']=='Targ')]['RT'], y=c_med_RT[(c_med_RT['Acc']=='Incorrect') & ([c_med_RT['Cond']=='Targ'])]['age'])

In [ ]:
sns.lmplot(x='age', y='RT', data=(c_med_RT[c_med_RT['Acc']=='Incorrect']))

In [ ]:
sns.lmplot(x='age', y='RT', data=(data_med_RT[data_med_RT['Cond']=='Target']))

In [ ]:
sns.lmplot(x='age', y='RT', data=(data_med_RT[(data_med_RT['Cond']=='Target') & (data_med_RT['Acc']=='Correct')]))

In [ ]:
pearsonr(x=data_med_RT[(data_med_RT['Cond']=='Target') & (data_med_RT['Acc']=='Correct')]['age'], y=data_med_RT[(data_med_RT['Cond']=='Target') & (data_med_RT['Acc']=='Correct')]['RT'])

In [ ]:
pearsonr(x=data_med_RT[(data_med_RT['Cond']=='Target') & (data_med_RT['Acc']=='Incorrect')]['age'], y=data_med_RT[(data_med_RT['Cond']=='Target') & (data_med_RT['Acc']=='Incorrect')]['RT'])

In [ ]:
pearsonr(x=data_med_RT[(data_med_RT['Cond']=='Lure') & (data_med_RT['Acc']=='Correct')]['age'], y=data_med_RT[(data_med_RT['Cond']=='Lure') & (data_med_RT['Acc']=='Correct')]['RT'])

In [ ]:
pearsonr(x=data_med_RT[(data_med_RT['Cond']=='Lure') & (data_med_RT['Acc']=='Incorrect')]['age'], y=data_med_RT[(data_med_RT['Cond']=='Lure') & (data_med_RT['Acc']=='Incorrect')]['RT'])

In [ ]:
pearsonr(x=data_med_RT[(data_med_RT['Cond']=='Foil') & (data_med_RT['Acc']=='Correct')]['age'], y=data_med_RT[(data_med_RT['Cond']=='Foil') & (data_med_RT['Acc']=='Correct')]['RT'])

In [ ]:
pearsonr(x=data_med_RT[(data_med_RT['Cond']=='Foil') & (data_med_RT['Acc']=='Incorrect')]['age'], y=data_med_RT[(data_med_RT['Cond']=='Foil') & (data_med_RT['Acc']=='Incorrect')]['RT'])

In [ ]:
sns.lmplot(x='RT', y='dprime_bps', data=data_avg_RT)

In [ ]:
sns.lmplot(x='RT', y='dprime_old_new', data=data_avg_RT)

In [ ]:
sns.lmplot(x='RT', y='dprime_old_lure', data=data_avg_RT)

In [ ]:
sns.lmplot(x='RT', y='dprime_bps', data=data_avg_RT)

In [ ]:
sns.lmplot(x='RT', y='dprime_old_new', data=data_avg_RT)

In [ ]:
sns.lmplot(x='RT', y='dprime_old_lure', data=data_avg_RT)

In [ ]:
sns.lmplot(x='age', y='RT', data=(data_med_RT[(data_med_RT['Cond']=='Target') & (data_med_RT['Acc']=='Incorrect')]))

In [ ]:
sns.lmplot(x='age', y='RT', data=(data_med_RT[(data_med_RT['Cond']=='Lure') & (data_med_RT['Acc']=='Correct')]))

In [ ]:
sns.lmplot(x='age', y='RT', data=(data_med_RT[(data_med_RT['Cond']=='Lure') & (data_med_RT['Acc']=='Incorrect')]))

In [ ]:
sns.lmplot(x='age', y='RT', data=(data_med_RT[(data_med_RT['Cond']=='Foil') & (data_med_RT['Acc']=='Correct')]))

In [ ]:
sns.lmplot(x='age', y='RT', data=(data_med_RT[(data_med_RT['Cond']=='Foil') & (data_med_RT['Acc']=='Incorrect')]))

In [ ]:
data_grouped_RT = data_long.groupby(['pidn','Acc','Cond']).mean().reset_index()
print(data_grouped_RT)

In [ ]:
data_RT = data_long.groupby(['pidn']).mean().reset_index()
print(data_RT)

In [ ]:
data_avg_RT = data_RT.merge(all_data, on='pidn')
data_avg_RT.head()

In [ ]:
sns.lmplot(x='RT', y='dprime_bps', data=data_avg_RT)

In [ ]:
pearsonr(x=data_avg_RT['RT'], y = data_avg_RT['dprime_bps'])

In [ ]:
sns.lmplot(x='RT', y='dprime_old_new', data=data_avg_RT)

In [ ]:
pearsonr(x=data_avg_RT['RT'], y = data_avg_RT['dprime_old_new'])

In [ ]:
sns.lmplot(x='RT', y='dprime_old_lure', data=data_avg_RT)

In [ ]:
pearsonr(x=data_avg_RT['RT'], y = data_avg_RT['dprime_old_lure'])

In [ ]:
row=0
data_avg_RT_c=data_grouped_RT.copy()
for i in data_avg_RT_c:
    if data_avg_RT_c.loc[row, 'Acc'] == 'Incorrect':
        data_avg_RT_c.drop(row, inplace=True)
        
    row+=1
    
data_avg_RT_c = data_avg_RT_c.groupby(['pidn']).mean().reset_index()

data_avg_RT_c.head()

In [ ]:
row=0
data_avg_RT_i=data_grouped_RT.copy()
for i in data_avg_RT_i:
    if data_avg_RT_i.loc[row, 'Acc'] == 'Correct':
        data_avg_RT_i.drop(row, inplace=True)
        
    row+=1
    
data_avg_RT_i = data_avg_RT_i.groupby(['pidn']).mean().reset_index()

data_avg_RT_i.head()

In [ ]:
data_avg_RT_i= data_avg_RT_i.merge(all_data, on='pidn')
data_avg_RT_i.head()

In [ ]:
data_avg_RT= data_avg_RT.merge(all_data, on='pidn')
data_avg_RT.head()

In [ ]:
data_avg_RT_c= data_avg_RT_c.merge(all_data, on='pidn')
data_avg_RT_c.head()

In [ ]:
sns.lmplot(x='age', y='RT', data=data_avg_RT)
plt.xlabel('Age', fontsize=14)
plt.ylabel('RT', fontsize=14)

In [ ]:
pearsonr(x=data_avg_RT['RT'], y = data_avg_RT['age'])

In [ ]:
sns.lmplot(x='age', y='RT', data=data_avg_RT_c)

In [ ]:
pearsonr(x=data_avg_RT_c['RT'], y = data_avg_RT_c['age'])

In [ ]:
pearsonr(x=data_avg_RT_i['RT'], y = data_avg_RT_i['age'])

In [ ]:
sns.lmplot(x='age', y='RT', data=data_avg_RT_i)

In [ ]:
row=0
targ_avg_RT_i=data_grouped_RT[(data_grouped_RT['Cond'] == 'Target') & (data_grouped_RT['Acc'] == 'Incorrect')]
    
targ_avg_RT_i = targ_avg_RT_i.groupby(['pidn']).mean().reset_index()

targ_avg_RT_i= targ_avg_RT_i.merge(all_data, on='pidn')
targ_avg_RT_i.head()

In [ ]:
row=0
targ_avg_RT_c=data_grouped_RT[(data_grouped_RT['Cond'] == 'Target') & (data_grouped_RT['Acc'] == 'Correct')]
    
targ_avg_RT_c = targ_avg_RT_c.groupby(['pidn']).mean().reset_index()

targ_avg_RT_c= targ_avg_RT_c.merge(all_data, on='pidn')
targ_avg_RT_c.head()

In [ ]:
row=0
lure_avg_RT_c=data_grouped_RT[(data_grouped_RT['Cond'] == 'Lure') & (data_grouped_RT['Acc'] == 'Correct')]
    
lure_avg_RT_c = lure_avg_RT_c.groupby(['pidn']).mean().reset_index()

lure_avg_RT_c= lure_avg_RT_c.merge(all_data, on='pidn')
lure_avg_RT_c.head()

In [ ]:
row=0
lure_avg_RT_i=data_grouped_RT[(data_grouped_RT['Cond'] == 'Lure') & (data_grouped_RT['Acc'] == 'Incorrect')]
    
lure_avg_RT_i = lure_avg_RT_i.groupby(['pidn']).mean().reset_index()

lure_avg_RT_i= lure_avg_RT_i.merge(all_data, on='pidn')
lure_avg_RT_i.head()

In [ ]:
row=0
foil_avg_RT_i=data_grouped_RT[(data_grouped_RT['Cond'] == 'Foil') & (data_grouped_RT['Acc'] == 'Incorrect')]
    
foil_avg_RT_i = foil_avg_RT_i.groupby(['pidn']).mean().reset_index()

foil_avg_RT_i= foil_avg_RT_i.merge(all_data, on='pidn')
foil_avg_RT_i.head()

In [ ]:
row=0
foil_avg_RT_c=data_grouped_RT[(data_grouped_RT['Cond'] == 'Foil') & (data_grouped_RT['Acc'] == 'Correct')]
    
foil_avg_RT_c = foil_avg_RT_c.groupby(['pidn']).mean().reset_index()

foil_avg_RT_c= foil_avg_RT_c.merge(all_data, on='pidn')
foil_avg_RT_c.head()

In [ ]:
sns.lmplot(x= 'age', y='RT', data=targ_avg_RT_c)

In [ ]:
pearsonr(x=targ_avg_RT_c['age'], y = targ_avg_RT_c['RT'])

In [ ]:
sns.lmplot(x= 'age', y='RT', data=targ_avg_RT_i)

In [ ]:
pearsonr(x=targ_avg_RT_i['age'], y = targ_avg_RT_i['RT'])

In [ ]:
sns.lmplot(x= 'age', y='RT', data=lure_avg_RT_i)

In [ ]:
pearsonr(x=lure_avg_RT_i['age'], y = lure_avg_RT_i['RT'])

In [ ]:
sns.lmplot(x= 'age', y='RT', data=lure_avg_RT_c)

In [ ]:
sns.lmplot(x= 'age', y='RT', data=foil_avg_RT_c)

In [ ]:
pearsonr(x=foil_avg_RT_c['age'], y = foil_avg_RT_c['RT'])

In [ ]:
pearsonr(x=foil_avg_RT_i['age'], y = foil_avg_RT_i['RT'])

In [ ]:
sns.lmplot(x= 'age', y='RT', data=foil_avg_RT_i)

In [ ]:
g=sns.distplot(data_long['RT'], kde=False)
g.set(xlim=(0, 2200))

In [ ]:
g=sns.distplot(data_long[data_long['Acc']=='Correct']['RT'], kde=False)
g.set(xlim=(0, 2200))

In [ ]:
g=sns.distplot(data_long[data_long['Acc']=='Correct']['RT'], kde=False)
g.set(xlim=(0, 2200))
#g.set(ylim=(0, 5000))

In [ ]:
g=sns.distplot(data_long[data_long['Acc']=='Incorrect']['RT'], kde=False)
g.set(xlim=(0, 2200))
g.set(ylim=(0, 700))

In [ ]:
g=sns.distplot(data_long[data_long['Acc']=='Correct']['RT'], kde=False)
g=sns.distplot(data_long[data_long['Acc']=='Incorrect']['RT'], kde=False)
g.set(xlim=(0, 2200))

In [ ]:
g=sns.distplot(data_long[(data_long['Cond']=='Target')]['RT'], kde=False)
#g=sns.distplot((data_long['Cond']=='Lure')]['RT'], kde=False)
# g=sns.distplot((data_long['Cond']=='Foil')]['RT'], kde=False)
g.set(xlim=(0, 2200))
g.set(ylim=(0, 700))
plt.title('Target Trials')

In [ ]:
g=sns.distplot(data_long[(data_long['Cond']=='Target') & (data_long['Acc']=='Correct')]['RT'], kde=False)
#g=sns.distplot((data_long['Cond']=='Lure')]['RT'], kde=False)
# g=sns.distplot((data_long['Cond']=='Foil')]['RT'], kde=False)
g.set(xlim=(0, 2200))
g.set(ylim=(0, 700))
plt.title('Target Trials')

In [ ]:
g=sns.distplot(data_long[(data_long['Cond']=='Target') & (data_long['Acc']=='Incorrect')]['RT'], kde=False)
#g=sns.distplot((data_long['Cond']=='Lure')]['RT'], kde=False)
# g=sns.distplot((data_long['Cond']=='Foil')]['RT'], kde=False)
g.set(xlim=(0, 2200))
g.set(ylim=(0, 700))
plt.title('Target Trials')

In [ ]:
g=sns.distplot(data_long[(data_long['Cond']=='Lure')]['RT'], kde=False)
# g=sns.distplot((data_long['Cond']=='Foil')]['RT'], kde=False)
g.set(xlim=(0, 2200))
g.set(ylim=(0, 700))
plt.title('Lure Trials')

In [ ]:
g=sns.distplot(data_long[(data_long['Cond']=='Foil')]['RT'], kde=False)
g.set(ylim=(0, 1000))
plt.title('Foil Trials')

In [ ]:
g=sns.distplot(data_long[(data_long['Acc']=='Correct')&(data_long['Cond']=='Target')]['RT'], kde=False)
#g=sns.distplot(data_long[(data_long['Acc']=='Correct')&(data_long['Cond']=='Lure')]['RT'], kde=False)
# g=sns.distplot(data_long[(data_long['Acc']=='Correct')&(data_long['Cond']=='Foil')]['RT'], kde=False)
g.set(xlim=(0, 2200))
g.set(ylim=(0, 700))
plt.title('Correct')

In [ ]:
g=sns.distplot(data_long[(data_long['Acc']=='Incorrect')&(data_long['Cond']=='Target')]['RT'], kde=False)
#g=sns.distplot(data_long[(data_long['Acc']=='Correct')&(data_long['Cond']=='Lure')]['RT'], kde=False)
# g=sns.distplot(data_long[(data_long['Acc']=='Correct')&(data_long['Cond']=='Foil')]['RT'], kde=False)
g.set(xlim=(0, 2200))
g.set(ylim=(0, 700))
plt.title('Incorrect')

In [ ]:
g=sns.distplot(data_long[(data_long['Acc']=='Correct')&(data_long['Cond']=='Lure')]['RT'], kde=False)
# g=sns.distplot(data_long[(data_long['Acc']=='Correct')&(data_long['Cond']=='Foil')]['RT'], kde=False)
g.set(xlim=(0, 2200))
plt.title('Correct')

In [ ]:
g=sns.distplot(data_long[(data_long['Acc']=='Incorrect')&(data_long['Cond']=='Lure')]['RT'], kde=False)
# g=sns.distplot(data_long[(data_long['Acc']=='Correct')&(data_long['Cond']=='Foil')]['RT'], kde=False)
g.set(xlim=(0, 2200))
g.set(ylim=(0, 550))
plt.title('Incorrect')

In [ ]:
g=sns.distplot(data_long[(data_long['Acc']=='Correct')&(data_long['Cond']=='Foil')]['RT'], kde=False)
g.set(xlim=(0, 2200))
plt.title('Correct')

In [ ]:
g=sns.distplot(data_long[(data_long['Acc']=='Incorrect')&(data_long['Cond']=='Foil')]['RT'], kde=False)
g.set(xlim=(0, 2200))

plt.title('Incorrect')

In [ ]:
g=sns.distplot(data_grouped_RT[data_grouped_RT['Acc']=='Correct']['RT'], kde=False)
g=sns.distplot(data_grouped_RT[data_grouped_RT['Acc']=='Incorrect']['RT'], kde=False)
g.set(xlim=(0, 2200))

In [ ]:
fig_dims = (12, 10)
fig, ax = plt.subplots(figsize=fig_dims)
g=sns.boxplot(x='Cond', y='RT', hue="Acc", data=data_grouped_RT, ax=ax,  palette='pastel', order=['Target', 'Lure', 'Foil'])
g= sns.stripplot(x='Cond', y='RT', hue = "Acc", data= data_grouped_RT, jitter=.2,split=True, ax=ax, order=['Target', 'Lure', 'Foil'])
g.set(ylim=[0, 2200])
plt.legend(bbox_to_anchor=(1.05,1),loc=2,borderaxespad=0.1, fontsize=18)
plt.xlabel('Condition', fontsize=20)
plt.ylabel('RT', fontsize=20)

In [ ]:
print(data_grouped_RT[data_grouped_RT['RT']<750])

In [ ]:
fig_dims = (12, 10)
fig, ax = plt.subplots(figsize=fig_dims)
g=sns.boxplot(x='Cond', y='RT', hue="Acc", data=data_med_RT, ax=ax,  palette='pastel', order=['Target', 'Lure', 'Foil'])
g= sns.stripplot(x='Cond', y='RT', hue = "Acc", data= data_med_RT, jitter=.2,split=True, ax=ax, order=['Target', 'Lure', 'Foil'])
g.set(ylim=[0, 2200])
plt.legend(bbox_to_anchor=(1.05,1),loc=2,borderaxespad=0.1, fontsize=18)
plt.xlabel('Condition', fontsize=20)
plt.ylabel('RT', fontsize=20)

Age

In [ ]:
g=sns.distplot(all_data['age'],kde=False)
plt.ylabel('Number of Participants', fontsize=14)
plt.xlabel('Age', fontsize=14)
g.set(ylim = [0, 65])

In [ ]:
print(statistics.mean(all_data['age']))
print(statistics.stdev(all_data['age']))
print(max(all_data['age']))

In [ ]:
g = sns.lmplot(x = 'age', y = 'prop_nr', data= all_data)
plt.ylabel('No Response (%)', fontsize=14)
plt.xlabel('Age', fontsize=14)

In [ ]:
data_long.head()

In [ ]:
data_t = data_long.merge(all_data, on=['pidn'])
data_t = data_t[data_t['Cond'] == 'Target']
data_t = data_t.groupby(['pidn']).mean().reset_index()

In [ ]:
sns.lmplot( x= 'age', y ='RT', data = data_t)


In [ ]:
pearsonr(data_t['age'], data_t['RT'])

In [ ]:
data_l = data_long.merge(all_data, on=['pidn'])
data_l = data_l[data_l['Cond'] == 'Lure']
data_l = data_l.groupby(['pidn']).mean().reset_index()

sns.lmplot( x= 'age', y ='RT', data = data_l)

In [ ]:
data_f = data_long.merge(all_data, on=['pidn'])
data_f = data_f[data_f['Cond'] == 'Foil']
data_f = data_f.groupby(['pidn']).mean().reset_index()

sns.lmplot( x= 'age', y ='RT', data = data_f)

In [ ]:
data = data_long.merge(all_data, on=['pidn'])
data=data[data['Acc']=='Correct']

data = data.groupby(['pidn']).mean().reset_index()

In [ ]:
g=sns.lmplot(x = 'age', y = 'RT', data=data) 

In [ ]:
data_i = data_long.merge(all_data, on=['pidn'])
data_i=data_i[(data_i['Acc']=='Incorrect') & (data_i['RT']!= int(0))]

data_i = data_i.groupby(['pidn']).mean().reset_index()

In [ ]:
g=sns.lmplot(x = 'age', y = 'RT', data=data_i) 

In [ ]:
pearsonr(x=data['age'], y = data['RT'])

In [ ]:
pearsonr(x=data['age'], y=data['RT'])

In [ ]:
pearsonr(x= all_data['age'], y = all_data['prop_nr'])

In [ ]:
g = sns.lmplot(x = 'age', y = 'p_correct', data = all_data)
plt.ylabel('Correct(%)')
plt.xlabel('Age')

In [ ]:
pearsonr(x= all_data['age'], y = all_data['p_correct'])

In [ ]:
g=sns.lmplot(x="age",y="old_targ_old_foil",data=all_data)
plt.xlabel('Age')
plt.ylabel('"Old"|Target - "Old"|Foil')

In [ ]:
g=sns.lmplot(x="age",y="dprime_old_new",data=all_data)
plt.xlabel('Age')
plt.ylabel('d\' Old/New')

In [ ]:
g = sns.lmplot(x = 'age', y = 'n_old_targ', data = all_data)
plt.xlabel('Age')
plt.ylabel('# Old|Target')

In [ ]:
pearsonr(x= all_data['age'], y = all_data['n_old_targ'])

In [ ]:
g = sns.lmplot(x = 'age', y = 'n_sim_lure', data = all_data)

In [ ]:
pearsonr(x= all_data['age'], y = all_data['n_sim_lure'])

In [ ]:
g = sns.lmplot(x = 'age', y = 'n_new_foil', data = all_data)

In [ ]:
pearsonr(x= all_data['age'], y = all_data['n_new_foil'])

In [ ]:
g = sns.lmplot(x='age', y = 'bps', data=all_data)
plt.xlabel('Age', fontsize=14)
plt.ylabel('BPS Score', fontsize=14)

In [ ]:
pearsonr(x= all_data['age'], y = all_data['bps'])

In [ ]:
g = sns.lmplot(x='age', y='dprime_bps', data=all_data)
plt.xlabel('Age', fontsize=14)
plt.ylabel('d\' BPS', fontsize=14)

In [ ]:
pearsonr(x= all_data['age'], y = all_data['dprime_bps'])

In [ ]:
g = sns.lmplot(x='age', y= 'old_targ_old_foil', data= all_data)
plt.xlabel('Age', fontsize=14)
plt.ylabel('Old/New Recognition', fontsize=14)

In [ ]:
pearsonr(x= all_data['age'], y = all_data['old_targ_old_foil'])

In [ ]:
g=sns.lmplot(x='age', y='dprime_old_new', data=all_data)
plt.xlabel('Age', fontsize=14)
plt.ylabel('d\' Old/New', fontsize=14)

In [ ]:
pearsonr(x= all_data['age'], y = all_data['dprime_old_new'])


In [ ]:
g = sns.lmplot(x = 'age', y = 'old_targ_old_lure', data= all_data)
plt.xlabel('Age', fontsize=14)
plt.ylabel('Old/Similar Discrimination', fontsize=14)

In [ ]:
pearsonr(x= all_data['age'], y = all_data['old_targ_old_lure'])

In [ ]:
sns.lmplot(x='age', y='dprime_old_lure', data=all_data)
plt.xlabel('Age', fontsize=14)
plt.ylabel('d\' Old/Similar', fontsize=14)

In [ ]:
pearsonr(x= all_data['age'], y = all_data['dprime_old_lure'])

In [ ]:
data_demo = all_data.copy()
data_demo.head()

In [ ]:
row = 0
age_group_new = data_demo.copy()
for a in data_demo['age']:
    if a == 59:
        a = 60
    if a >= 60:
        if a < 65:
            age_group_new.loc[row, 'age'] = '60-64'
        elif a < 70:
            age_group_new.loc[row, 'age'] = '65-69'
        elif a < 75:
            age_group_new.loc[row, 'age'] = '70-74'
        elif a < 80:
            age_group_new.loc[row, 'age'] = '75-79'
        elif a < 85:
            age_group_new.loc[row, 'age'] = '80-84'
        else:
            age_group_new.loc[row, 'age'] = '85+'
    row+=1    
print(age_group_new)

In [ ]:
print(sum(1 for a in age_group_new['age'] if a == '60-64'))
print(sum(1 for a in age_group_new['age'] if a == '65-69'))
print(sum(1 for a in age_group_new['age'] if a == '70-74'))
print(sum(1 for a in age_group_new['age'] if a == '75-79'))
print(sum(1 for a in age_group_new['age'] if a == '80-84'))
print(sum(1 for a in age_group_new['age'] if a == '85+'))

In [ ]:
g = sns.stripplot(x= age_group_new['age'], y = age_group_new['bps'], jitter = .2)
sns.boxplot(x = age_group_new['age'], y = age_group_new['bps'], palette='pastel', order = ['60-64', '65-69', '70-74', '75-79', '80-84', '85+'])
plt.xlabel('Age Group', fontsize=14)
plt.ylabel('BPS Score', fontsize=14)

In [ ]:
g = sns.stripplot(x= age_group_new['age'], y = age_group_new['dprime_bps'], jitter = .2)
sns.boxplot(x = age_group_new['age'], y = age_group_new['dprime_bps'], palette='pastel', order = ['60-64', '65-69', '70-74', '75-79', '80-84', '85+'])
plt.xlabel('Age Group', fontsize=14)
plt.ylabel('d\' BPS', fontsize=14)

In [ ]:
g = sns.stripplot(x= age_group_new['age'], y = age_group_new['dprime_old_new'], jitter = .2)
sns.boxplot(x = age_group_new['age'], y = age_group_new['dprime_old_new'], palette='pastel', order = ['60-64', '65-69', '70-74', '75-79', '80-84', '85+'])
plt.xlabel('Age Group', fontsize=14)
plt.ylabel('d\' Old/New', fontsize=14)

In [ ]:
g = sns.stripplot(x= age_group_new['age'], y = age_group_new['dprime_old_lure'], jitter = .2)
sns.boxplot(x = age_group_new['age'], y = age_group_new['dprime_old_lure'], palette='pastel', order = ['60-64', '65-69', '70-74', '75-79', '80-84', '85+'])
plt.xlabel('Age Group', fontsize=14)
plt.ylabel('d\' Old/Similar', fontsize=14)

In [ ]:
g = sns.stripplot(x= age_group_new['age'], y = age_group_new['old_targ_old_foil'], jitter = .2)
sns.boxplot(x = age_group_new['age'], y = age_group_new['old_targ_old_foil'], palette='pastel')
plt.xlabel('Age Group', fontsize=14)
plt.ylabel('Old/New Recognition', fontsize=14)

In [ ]:
g = sns.stripplot(x= age_group_new['age'], y = age_group_new['old_targ_old_lure'], jitter = .2)
sns.boxplot(x = age_group_new['age'], y = age_group_new['old_targ_old_lure'], palette='pastel')
plt.xlabel('Age Group', fontsize=14)
plt.ylabel('Old/Similar Recognition', fontsize=14)

In [ ]:
g = sns.stripplot(x= age_group_new['age'], y = age_group_new['prop_nr'], jitter = .2)
sns.boxplot(x = age_group_new['age'], y = age_group_new['prop_nr'], palette='pastel')
plt.xlabel('Age Group')
plt.ylabel('No Response (%)')

In [ ]:
g = sns.stripplot(x= age_group_new['age'], y = age_group_new['p_correct'], jitter = .2)
sns.boxplot(x = age_group_new['age'], y = age_group_new['p_correct'], palette='pastel')
plt.xlabel('Age Group')
plt.ylabel('Percent Correct')

In [ ]:
data_demo.head()
age_group_repl = data_demo.copy()
age_group_repl.loc[data_demo['age'] < 75, 'age'] = '60-74'
age_group_repl.loc[data_demo['age'] > 74, 'age']= '75-90'
age_group_repl.head()

In [ ]:
g = sns.stripplot(x= age_group_repl['age'], y = age_group_repl['bps'], jitter = .2)
sns.boxplot(x = age_group_repl['age'], y = age_group_repl['bps'], palette='pastel')
plt.xlabel('Age Group')
plt.ylabel('BPS Score')

In [ ]:
g = sns.stripplot(x= age_group_repl['age'], y = age_group_repl['dprime_bps'], jitter = .2)
sns.boxplot(x = age_group_repl['age'], y = age_group_repl['dprime_bps'], palette='pastel')
plt.xlabel('Age Group', fontsize=14)
plt.ylabel('d\' BPS', fontsize=14)

In [ ]:
g = sns.stripplot(x= age_group_repl['age'], y = age_group_repl['dprime_old_new'], jitter = .2)
sns.boxplot(x = age_group_repl['age'], y = age_group_repl['dprime_old_new'], palette='pastel')
plt.xlabel('Age Group', fontsize=14)
plt.ylabel('d\' Old/New', fontsize=14)

In [ ]:
g = sns.stripplot(x= age_group_repl['age'], y = age_group_repl['dprime_old_lure'], jitter = .2)
sns.boxplot(x = age_group_repl['age'], y = age_group_repl['dprime_old_lure'], palette='pastel')
plt.xlabel('Age Group', fontsize=14)
plt.ylabel('d\' Old\Similar', fontsize=14)

In [ ]:
g = sns.stripplot(x= age_group_repl['age'], y = age_group_repl['old_targ_old_foil'], jitter = .25)
sns.boxplot(x = age_group_repl['age'], y = age_group_repl['old_targ_old_foil'], palette='pastel')
plt.ylabel('Old/New Recognition')
plt.xlabel('Age Group')


In [ ]:
g = sns.stripplot(x= age_group_repl['age'], y = age_group_repl['old_targ_old_lure'], jitter = .2)
sns.boxplot(x = age_group_repl['age'], y = age_group_repl['old_targ_old_lure'], palette='pastel')
plt.xlabel('Age Group')
plt.ylabel('Old/Similar Recognition')

In [ ]:
g = sns.stripplot(x= age_group_repl['age'], y = age_group_repl['prop_nr'])
sns.boxplot(x = age_group_repl['age'], y = age_group_repl['prop_nr'], palette='pastel')
plt.xlabel('Age Group')
plt.ylabel('No Response(%)')

In [ ]:
g = sns.stripplot(x= age_group_repl['age'], y = age_group_repl['p_correct'])
sns.boxplot(x = age_group_repl['age'], y = age_group_repl['p_correct'], palette='pastel')
plt.xlabel('Age Group')
plt.ylabel('Percent Correct')

Education

In [ ]:
print(statistics.mean(all_data['education']))
print(statistics.stdev(all_data['education']))
print(max(all_data['education']))

In [ ]:
g=sns.distplot(all_data['education'],kde=False)
plt.ylabel('Number of Participants', fontsize=14)
plt.xlabel('Education', fontsize=14)
g.set(ylim=[0,65])


In [ ]:
educ_data = all_data.copy()
row = 0
for e in all_data['education']:
    if e == 12:
        educ_data.loc[row, 'education'] = 'HS/GED'
    elif e == 14 or e == 15:
        educ_data.loc[row, 'education']= 'Associate'
    elif e == 16 or e== 17:
        educ_data.loc[row, 'education']= 'Bachelor\'s'
    elif e== 18 or e==19:
        educ_data.loc[row, 'education']= 'Master\'s'
    else:
        educ_data.loc[row, 'education']='Doctorate'  
    row+=1
    
educ_data.head()

In [ ]:
g=sns.lmplot(x="education",y="dprime_old_new",data=all_data)
plt.xlabel('Education', fontsize=14)
plt.ylabel('d\' Old/New', fontsize=14)

In [ ]:
g=sns.lmplot(x="education",y="dprime_old_lure",data=all_data)
plt.xlabel('Education', fontsize=14)
plt.ylabel('d\' Old/Similar', fontsize=14)

In [ ]:
g=sns.lmplot(x="education",y="bps",data=all_data)
plt.xlabel('Education', fontsize=14)
plt.ylabel('BPS', fontsize=14)

In [ ]:
g=sns.lmplot(x="education",y="old_targ_old_lure",data=all_data)
plt.xlabel('Education', fontsize=14)
plt.ylabel('Old/Similar Recognition', fontsize=14)

In [ ]:
g=sns.lmplot(x="education",y="dprime_bps",data=all_data)
plt.xlabel('Education', fontsize=14)
plt.ylabel('d\' BPS', fontsize=14)

In [ ]:
g=sns.lmplot(x="education",y="old_targ_old_foil",data=all_data)
plt.xlabel('Education', fontsize=14)
plt.ylabel('Old/New Recognition', fontsize=14)

In [ ]:
g = sns.stripplot(x= educ_data['education'], y = educ_data['bps'], jitter = .2)
sns.boxplot(x = educ_data['education'], y = educ_data['bps'], palette='pastel', order= ['HS/GED', 'Associate', 'Bachelor\'s', 'Master\'s', 'Doctorate'])
plt.xlabel('Education', fontsize=14)
plt.ylabel('BPS Score', fontsize=14)

In [ ]:
g = sns.stripplot(x = educ_data['education'], y = all_data['old_targ_old_foil'], jitter = .2)
g = sns.boxplot(x = educ_data['education'], y = all_data['old_targ_old_foil'], palette = 'pastel', order= ['HS/GED', 'Associate', 'Bachelor\'s', 'Master\'s', 'Doctorate'])
plt.ylabel('Old/New Recognition', fontsize=14)
plt.xlabel('Education', fontsize=14)

In [ ]:
sns.stripplot(x = educ_data['education'], y = all_data['old_targ_old_lure'], jitter = .2)
g = sns.boxplot(x = educ_data['education'], y = all_data['old_targ_old_lure'], palette = 'pastel', order= ['HS/GED', 'Associate', 'Bachelor\'s', 'Master\'s', 'Doctorate'])
plt.xlabel('Education', fontsize=14)
plt.ylabel('Old/Similar Recognition', fontsize=14)

In [ ]:
sns.stripplot(x = educ_data['education'], y = all_data['dprime_bps'], jitter = .2)
g = sns.boxplot(x = educ_data['education'], y = all_data['dprime_bps'], palette = 'pastel', order= ['HS/GED', 'Associate', 'Bachelor\'s', 'Master\'s', 'Doctorate'])
plt.xlabel('Education', fontsize=14)
plt.ylabel('d\' BPS Score', fontsize=14)

In [ ]:
sns.stripplot(x = educ_data['education'], y = all_data['dprime_old_new'], jitter = .2)
g = sns.boxplot(x = educ_data['education'], y = all_data['dprime_old_new'], palette = 'pastel', order= ['HS/GED', 'Associate', 'Bachelor\'s', 'Master\'s', 'Doctorate'])
plt.xlabel('Education', fontsize=14)
plt.ylabel('d\' Old/New Recognition', fontsize=14)


In [ ]:
sns.stripplot(x = educ_data['education'], y = all_data['dprime_old_lure'], jitter = .2)
g = sns.boxplot(x = educ_data['education'], y = all_data['dprime_old_lure'], palette = 'pastel', order= ['HS/GED', 'Associate', 'Bachelor\'s', 'Master\'s', 'Doctorate'])
plt.xlabel('Education', fontsize=14)
plt.ylabel('d\' Old/Lure Discrimination', fontsize=14)


In [ ]:
g = sns.stripplot(x= educ_data['education'], y = all_data['prop_nr'])
sns.boxplot(x = educ_data['education'], y = all_data['prop_nr'], palette='pastel', order= ['HS/GED', 'Associate', 'Bachelor\'s', 'Master\'s', 'Doctorate'])
plt.xlabel('Education')

In [ ]:
g = sns.stripplot(x= educ_data['education'], y = all_data['p_correct'])
sns.boxplot(x = educ_data['education'], y = all_data['p_correct'], palette='pastel')
plt.xlabel('Education')
plt.ylabel('Percent Correct')

Gender

In [ ]:
sum(1 for g in all_data['gender'] if g == 'F')

In [ ]:
g = sns.stripplot(x = all_data['gender'], y = all_data['bps'])
g = sns.boxplot(x = all_data['gender'], y = all_data['bps'], palette = 'pastel')
plt.ylabel('BPS Score', fontsize=14)
plt.xlabel('Gender', fontsize=14)
g.set(ylim=[0,1])

In [ ]:
g = sns.stripplot(x = all_data['gender'], y = all_data['dprime_bps'])
g = sns.boxplot(x = all_data['gender'], y = all_data['dprime_bps'], palette = 'pastel')
plt.xlabel('Gender', fontsize=14)
plt.ylabel('d\' BPS', fontsize=14)

In [ ]:
g = sns.stripplot(x = all_data['gender'], y = all_data['dprime_old_new'])
g = sns.boxplot(x = all_data['gender'], y = all_data['dprime_old_new'], palette = 'pastel')
plt.xlabel('Gender', fontsize=14)
plt.ylabel('d\' Old/New', fontsize=14)

In [ ]:
g = sns.stripplot(x = all_data['gender'], y = all_data['dprime_old_lure'])
g = sns.boxplot(x = all_data['gender'], y = all_data['dprime_old_lure'], palette = 'pastel')
plt.xlabel('Gender', fontsize=14)
plt.ylabel('d\' Old/Similar', fontsize=14)

In [ ]:
g = sns.stripplot(x = all_data['gender'], y = all_data['old_targ_old_foil'])
g = sns.boxplot(x = all_data['gender'], y = all_data['old_targ_old_foil'], palette = 'pastel')
plt.ylabel('Old/New Recognition', fontsize=14)
plt.xlabel('Gender', fontsize=14)

In [ ]:
g = sns.stripplot(x = all_data['gender'], y = all_data['old_targ_old_lure'])
g = sns.boxplot(x = all_data['gender'], y = all_data['old_targ_old_lure'], palette = 'pastel')
plt.ylabel('Old/Similar Recognition', fontsize=14)
plt.xlabel('Gender', fontsize=14)
g.set(ylim=[0,1])

In [ ]:
g = sns.stripplot(x = all_data['gender'], y = all_data['prop_nr'])
g = sns.boxplot(x = all_data['gender'], y = all_data['prop_nr'], palette = 'pastel')
plt.ylabel('No Response (%)')

In [ ]:
g = sns.stripplot(x = all_data['gender'], y = all_data['p_correct'])
g = sns.boxplot(x = all_data['gender'], y = all_data['p_correct'], palette = 'pastel')
plt.ylabel('Percent Correct')

In [ ]:
g = sns.stripplot(x=all_data['gender'], y = (all_data['n_old_targ']+ all_data['n_old_lure']+ all_data['n_old_foil']))
g = sns.boxplot(x=all_data['gender'], y = (all_data['n_old_targ']+ all_data['n_old_lure']+ all_data['n_old_foil']), palette='pastel')
plt.ylabel('Num Old Responses')

Reaction Time

Response Summary

In [ ]:
resp_sum_count = sns.catplot(x="Cond",hue="Resp",data=data_long,kind='count', 
                       hue_order=['Old','Similar','New','No Resp'])


In [ ]:
data_long_grouped = data_long.groupby(['pidn','Cond','Resp']).count().reset_index()
data_long_grouped.head()

In [ ]:
data_long_grouped['Pr Resp']=data_long_grouped['Acc']/64
data_long_grouped.head()

In [ ]:
resp_sum_mean=sns.catplot(x="Cond",y="Acc",hue="Resp",data=data_long_grouped,kind='bar',
             order=['Target','Lure','Foil'],hue_order=['Old','Similar','New','No Resp'])
plt.ylabel('Mean Num Responses')

In [ ]:
resp_sum_prop = sns.catplot(x = 'Cond', y = 'Pr Resp', hue = 'Resp', data= data_long_grouped, kind = 'bar',
                           order=['Target','Lure','Foil'],hue_order=['Old','Similar','New','No Resp'])
plt.ylabel('Proportion Response')

In [ ]:
fig, resp_num_box = plt.subplots(figsize=(10,6))

resp_num_box =sns.boxplot(x="Cond",y="Acc",hue="Resp",data=data_long_grouped,
             order=['Target','Lure','Foil'],hue_order=['Old','Similar','New','No Resp'])
plt.ylabel('Num Responses')
plt.legend(bbox_to_anchor=(1.05,1),loc=2,borderaxespad=0.1)

In [ ]:
fig, prop_resp_box_scat = plt.subplots(figsize=(13,9))

prop_resp_box_scat=sns.stripplot(x="Cond",y="Pr Resp",hue="Resp",data=data_long_grouped,linewidth=1,jitter=.2,
             order=['Target','Lure','Foil'],hue_order=['Old','Similar','New','No Resp'],dodge=True)

prop_resp_box_scat=sns.boxplot(x="Cond",y="Pr Resp",hue="Resp",data=data_long_grouped,palette='pastel',
             order=['Target','Lure','Foil'],hue_order=['Old','Similar','New','No Resp'])
plt.legend(bbox_to_anchor=(1.05,1),loc=2,borderaxespad=0.1, fontsize=20)
plt.xlabel('Condition', fontsize=20)
plt.ylabel('Proporition Responses', fontsize=20)
plt.rc('xtick', labelsize=12)     
plt.rc('ytick', labelsize=12)

In [ ]:
fig, ax = plt.subplots(figsize=(8,5))

sns.boxplot(x="Cond",y="Pr Resp",data=data_long_grouped[data_long_grouped['Resp']=='No Resp'],palette='muted',
             order=['Target','Lure','Foil'],fliersize=0)
sns.stripplot(x="Cond", y="Pr Resp",data=data_long_grouped[data_long_grouped['Resp']=='No Resp'],
             order=['Target','Lure','Foil'],jitter=.2,linewidth=1)

plt.ylabel('Pr No Resonse', fontsize=14)
plt.xlabel('Condition', fontsize=14)

In [ ]:
g=sns.distplot(all_data['prop_nr'],kde=False) #more of these
plt.xlabel('No Response (%)', fontsize=14)
plt.ylabel('Num Participants', fontsize=14)


In [ ]:
g=sns.lmplot(x='prop_nr', y='dprime_old_new', data=all_data)

In [ ]:
g=sns.lmplot(x='prop_nr', y='dprime_bps', data=all_data)

In [ ]:
g=sns.lmplot(x='prop_nr', y='dprime_old_lure', data=all_data)

In [ ]:
g= sns.lmplot(x='age', y='prop_nr', data=all_data)

In [ ]:
sum(1 for x in all_data['prop_nr'] if x == 0)

In [ ]:
g = sns.distplot((all_data['num_nr_t']/64), kde = False)
plt.xlabel('Prop NR Target Trials', fontsize=14)
plt.ylabel('Num Participants', fontsize=14)
g.set(ylim=[0,85])

In [ ]:
g = sns.distplot(all_data['num_nr_l']/64, kde = False)
plt.xlabel('Prop NR Lure Trials', fontsize=14)
g.set(ylim=[0, 85])

In [ ]:
g = sns.distplot(all_data['num_nr_f']/64, kde=False)
plt.xlabel('Prop NR Foil Trials', size=14)
g.set(ylim=[0, 85])

In [ ]:
acc = sns.distplot(all_data['p_correct'], kde=False)
plt.xlabel('Correct(%)', fontsize=14)
plt.ylabel('Num of Participants', fontsize=14)
acc.set(ylim=[0,45])

In [ ]:
g = sns.distplot(all_data['n_old_targ']/64, kde = False)
plt.xlabel('Prop Correct Target Trials', fontsize = 14)
plt.ylabel('Num Participants', fontsize=14)
g.set(ylim=[0,40])

In [ ]:
g= sns.distplot(all_data['n_sim_lure']/64, kde = False)
plt.xlabel('Prop Correct Lure Trials', fontsize = 14)
g.set(ylim=[0,40])

In [ ]:
g = sns.distplot(all_data['n_new_foil']/64, kde = False)
plt.xlabel('Prop Correct Foil Trials', fontsize=14)
g.set(ylim=[0,40])

In [ ]:
g = sns.lmplot(x="prop_nr",y="bps",data=all_data)
g.set(ylim=(-.25, 1))


In [ ]:
pearsonr(y = all_data['bps'], x = all_data['prop_nr'])

In [ ]:
g = sns.lmplot(x="prop_nr",y="old_targ_old_lure",data=all_data)
g.set(ylim=(-.25, 1), )

In [ ]:
pearsonr(y = all_data['old_targ_old_lure'], x = all_data['prop_nr'])

In [ ]:
g = sns.lmplot(x="prop_nr",y="old_targ_old_foil",data=all_data)
g.set(ylim=(-.25, 1))

In [ ]:
pearsonr(y = all_data['old_targ_old_foil'], x = all_data['prop_nr'])

In [ ]:
g = sns.distplot(all_data['bps'], kde = False)
plt.xlabel('BPS Scores', fontsize=14)


In [ ]:
g = sns.distplot(all_data['dprime_bps'], kde = False)
plt.xlabel('d\' BPS', fontsize=14)

In [ ]:
g = sns.distplot(all_data['dprime_old_new'], kde = False)
plt.xlabel('d\' Old/New', fontsize=14)

In [ ]:
g = sns.distplot(all_data['dprime_old_lure'], kde = False)
plt.xlabel('d\' Old/Similar', fontsize=14)

In [ ]:
g = sns.distplot(all_data['old_targ_old_foil'], kde = False)
plt.xlabel('Old/New Recognition', fontsize=14)
plt.ylabel('Number of Participants', fontsize=14)

In [ ]:
g = sns.distplot(all_data['old_targ_old_lure'], kde = False)
plt.xlabel('Old/Similar Recognition', fontsize=14)
plt.ylabel('Number of Participants', fontsize=14)

In [ ]:
g = sns.lmplot(x ='bps', y = 'old_targ_old_foil', data = all_data)
g.set( ylim=(-.25, 1))
plt.xlabel('BPS Score', fontsize=14)
plt.ylabel('Old/New Recognition', fontsize=14)

In [ ]:
pearsonr(x = all_data['bps'], y = all_data['old_targ_old_foil'])

In [ ]:
g = sns.lmplot(x='bps', y = 'old_targ_old_lure', data = all_data)
g.set( ylim=(-.25, 1))
plt.xlabel('BPS Score', fontsize=14)
plt.ylabel('Old/Similar Recognition', fontsize=14)

In [ ]:
pearsonr(x = all_data['bps'], y = all_data['old_targ_old_lure'])

In [ ]:
g = sns.lmplot(x= 'old_targ_old_foil', y = 'old_targ_old_lure', data=all_data)
g.set(ylim=(-.25, 1))
plt.xlabel('Old/New Recognition', fontsize=14)
plt.ylabel('Old/Similar Recognition', fontsize=14)

In [ ]:
pearsonr(x = all_data['old_targ_old_foil'], y = all_data['old_targ_old_lure'])

In [ ]:
g = sns.lmplot(x= 'dprime_bps', y = 'dprime_old_new', data=all_data)
plt.xlabel('d\' BPS', fontsize=14)
plt.ylabel('d\' Old/New Recognition', fontsize=14)
g.set(ylim=[-1.6, 4])

In [ ]:
pearsonr(x = all_data['dprime_bps'], y = all_data['dprime_old_new'])

In [ ]:
g = sns.lmplot(x= 'dprime_bps', y = 'dprime_old_lure', data=all_data)
plt.xlabel('d\' BPS', fontsize=14)
plt.ylabel('d\' Old/Similar Recognition', fontsize=14)
g.set(ylim=[-1.6, 4])

In [ ]:
pearsonr(x = all_data['dprime_bps'], y = all_data['dprime_old_lure'])

In [ ]:
g = sns.lmplot(x= 'dprime_old_new', y = 'dprime_old_lure', data=all_data)
g.set(ylim=[-1.6, 4])
plt.xlabel('d\' Old/New Recognition', fontsize=14)
plt.ylabel('d\' Old/Similar Recognition', fontsize=14)

In [ ]:
pearsonr(x = all_data['dprime_old_new'], y = all_data['dprime_old_lure'])

Neuropsych Correlations


In [ ]:
neuro_data = np_data.dropna().merge(all_data, on = 'pidn')

neuro_data.head()

In [ ]:
print(statistics.mean(neuro_data['mmse']))
print(statistics.stdev(neuro_data['mmse'])) 
print(min(neuro_data['mmse']))
print(max(neuro_data['mmse']))

In [ ]:
g= sns.lmplot(x = 'mmse_norm', y= 'dprime_bps', data= neuro_data)

In [ ]:
pearsonr(x = neuro_data['mmse_norm'], y = neuro_data['dprime_bps'])

In [ ]:
g = sns.lmplot(x='mmse_norm', y = 'dprime_old_new', data=neuro_data)

In [ ]:
pearsonr(x=neuro_data['mmse_norm'], y=neuro_data['dprime_old_new'])

In [ ]:
g = sns.lmplot(x='mmse_norm', y='dprime_old_lure', data=neuro_data)

In [ ]:
pearsonr(x=neuro_data['mmse_norm'], y=neuro_data['dprime_old_lure'])

In [ ]:
g = sns.lmplot(x='ir_comp', y = 'dprime_bps', data=neuro_data)
plt.xlabel('Immediate Recall', fontsize=14)
plt.ylabel('d\' BPS', fontsize=14)

In [ ]:
pearsonr(x=neuro_data['ir_comp'], y=neuro_data['dprime_bps'])

In [ ]:
g= sns.lmplot(x='ir_comp', y='dprime_old_lure', data=neuro_data)
plt.xlabel('Immediate Recall', fontsize=14)
plt.ylabel('d\' Old/Similar', fontsize=14)

In [ ]:
g= sns.lmplot(x='ir_comp', y='dprime_old_new', data=neuro_data)
plt.xlabel('Immediate Recall', fontsize=14)
plt.ylabel('d\' Old/New', fontsize=14)

In [ ]:
pearsonr(x=neuro_data['ir_comp'], y=neuro_data['dprime_old_new'])

In [ ]:
g=sns.lmplot(x='hvltpr', y='dprime_bps', data=neuro_data)

In [ ]:
pearsonr(x=neuro_data['hvltpr'], y=neuro_data['dprime_bps'])

In [ ]:
g=sns.lmplot(x='bvmtpr', y='dprime_bps', data=neuro_data)

In [ ]:
pearsonr(x=neuro_data['bvmtpr'], y=neuro_data['dprime_bps'])

In [ ]:
g=sns.lmplot(x='lmpr', y='dprime_bps', data=neuro_data)

In [ ]:
pearsonr(x=neuro_data['lmpr'], y=neuro_data['dprime_bps'])

In [ ]:
g=sns.lmplot(x='dr_comp', y='old_targ_old_foil', data=neuro_data)
plt.xlabel('Delayed Recall', fontsize=14)
plt.ylabel('Old/New Recognition', fontsize=14)

In [ ]:
g=sns.lmplot(x='dr_comp', y='old_targ_old_lure', data=neuro_data)
plt.xlabel('Delayed Recall', fontsize=14)
plt.ylabel('Old/Similar Recognition', fontsize=14)

In [ ]:
g=sns.lmplot(x='dr_comp', y='bps', data=neuro_data)
plt.xlabel('Delayed Recall', fontsize=14)
plt.ylabel('BPS', fontsize=14)

In [ ]:
pearsonr(x=neuro_data['dr_comp'], y=neuro_data['bps'])

In [ ]:
pearsonr(x=neuro_data['dr_comp'], y=neuro_data['old_targ_old_foil'])

In [ ]:
pearsonr(x=neuro_data['dr_comp'], y=neuro_data['old_targ_old_lure'])

In [ ]:
g=sns.lmplot(x='dr_comp', y='dprime_old_new', data=neuro_data)
plt.xlabel('Delayed Recall', fontsize=14)
plt.ylabel('d\' Old/New', fontsize=14)

In [ ]:
pearsonr(x=neuro_data['dr_comp'], y=neuro_data['dprime_old_new'])

In [ ]:
g=sns.lmplot(x='dr_comp', y='dprime_old_lure', data=neuro_data)
plt.xlabel('Delayed Recall', fontsize=14)
plt.ylabel('d\' Old/Similar', fontsize=14)

In [ ]:
pearsonr(x=neuro_data['dr_comp'], y=neuro_data['dprime_old_lure'])

In [ ]:
g=sns.lmplot(x='dr_comp', y='dprime_bps', data=neuro_data)
plt.xlabel('Delayed Recall', fontsize=14)
plt.ylabel('d\' BPS', fontsize=14)

In [ ]:
pearsonr(x=neuro_data['dr_comp'], y=neuro_data['dprime_bps'])

In [ ]:
g=sns.lmplot(x='hvltpr', y='dprime_old_new', data=neuro_data)

In [ ]:
pearsonr(x=neuro_data['hvltpr'], y=neuro_data['dprime_old_new'])

In [ ]:
g=sns.lmplot(x='bvmtpr', y='dprime_old_new', data=neuro_data)

In [ ]:
pearsonr(x=neuro_data['bvmtpr'], y=neuro_data['dprime_old_new'])

In [ ]:
g=sns.lmplot(x='lmpr', y='dprime_old_new', data=neuro_data)

In [ ]:
pearsonr(x=neuro_data['lmpr'], y=neuro_data['dprime_old_new'])

In [ ]:
g=sns.lmplot(x='hvltpr', y='dprime_old_lure', data=neuro_data)

In [ ]:
pearsonr(x=neuro_data['hvltpr'], y=neuro_data['dprime_old_lure'])

In [ ]:
g=sns.lmplot(x='bvmtpr', y='dprime_old_lure', data=neuro_data)

In [ ]:
pearsonr(x=neuro_data['bvmtpr'], y=neuro_data['dprime_old_lure'])

In [ ]:
g=sns.lmplot(x='lmpr', y='dprime_old_lure', data=neuro_data)

In [ ]:
pearsonr(x=neuro_data['lmpr'], y=neuro_data['dprime_old_lure'])

In [ ]:
g=sns.lmplot(x='hvltr4_norm', y='dprime_bps', data=neuro_data)
plt.xlabel('HLVTR Delay', fontsize=14)
plt.ylabel('d\' BPS', fontsize=14)

In [ ]:
pearsonr(x=neuro_data['hvltr4_norm'], y=neuro_data['dprime_bps'])

In [ ]:
g=sns.lmplot(x='hvltr4', y='bps', data=neuro_data)
plt.xlabel('HVLT Delay', fontsize=14)
plt.ylabel('BPS', fontsize=14)

In [ ]:
g=sns.lmplot(x='hvltr4_norm', y='dprime_bps', data=neuro_data)
plt.xlabel('HVLT Delay', fontsize=14)
plt.ylabel('d\' BPS', fontsize=14)

In [ ]:
pearsonr(x=neuro_data['dprime_bps'], y=neuro_data['hvltr4'])

In [ ]:
g=sns.lmplot(x='bvmtr4_norm', y='dprime_bps', data=neuro_data)
plt.xlabel('BVMT Delay', fontsize=14)
plt.ylabel('d\' BPS', fontsize=14)

In [ ]:
pearsonr(x= neuro_data['bvmtr4'], y = neuro_data['dprime_bps'])

In [ ]:
g=sns.lmplot(x='lm2_norm', y='dprime_bps', data=neuro_data)
plt.xlabel('LM Delay', fontsize=14)
plt.ylabel('d\' BPS', fontsize=14)

In [ ]:
pearsonr(x=neuro_data['hvltr4'], y=neuro_data['bps'])

In [ ]:
pearsonr(x=neuro_data['bvmtr4'], y=neuro_data['bps'])

In [ ]:
pearsonr(x=neuro_data['lm2'], y=neuro_data['dprime_bps'])

In [ ]:
g=sns.lmplot(x='bvmtr4_norm', y='dprime_bps', data=neuro_data)
plt.xlabel('BVMT Delay', fontsize=14)
plt.ylabel('d\' BPS', fontsize=14)

In [ ]:
pearsonr(x=neuro_data['bvmtr4_norm'], y=neuro_data['dprime_bps'])


In [ ]:
g=sns.lmplot(x='lm2_norm', y='dprime_bps', data=neuro_data)
plt.xlabel('LM Delay', fontsize=14)
plt.ylabel('d\' BPS', fontsize=14)

In [ ]:
pearsonr(x=neuro_data['lm2_norm'], y=neuro_data['dprime_bps'])

In [ ]:
g=sns.lmplot(x='hvltr4_norm', y='dprime_old_new', data=neuro_data)
plt.xlabel('HVLT Delay', fontsize=14)
plt.ylabel('d\' Old/New', fontsize=14)

In [ ]:
pearsonr(x=neuro_data['hvltr4_norm'], y=neuro_data['dprime_old_new'])

In [ ]:
g=sns.lmplot(x='bvmtr4_norm', y='dprime_old_new', data=neuro_data)
plt.xlabel('BVMT Delay', fontsize=14)
plt.ylabel('d\' Old/New', fontsize=14)

In [ ]:
pearsonr(x=neuro_data['bvmtr4_norm'], y=neuro_data['dprime_old_new'])

In [ ]:
g=sns.lmplot(x='lm2_norm', y='dprime_old_new', data=neuro_data)
plt.xlabel('LM Delay', fontsize=14)
plt.ylabel('d\' Old/New', fontsize=14)

In [ ]:
pearsonr(x=neuro_data['lm2_norm'], y=neuro_data['dprime_old_new'])

In [ ]:
g=sns.lmplot(x='hvltr4_norm', y='dprime_old_lure', data=neuro_data)
plt.xlabel('HVLT Delay', fontsize=14)
plt.ylabel('d\' Old/Similar', fontsize=14)

In [ ]:
pearsonr(x=neuro_data['hvltr4_norm'], y=neuro_data['dprime_old_lure'])

In [ ]:
g=sns.lmplot(x='bvmtr4_norm', y='dprime_old_lure', data=neuro_data)
plt.xlabel('BVMT Delay', fontsize=14)
plt.ylabel('d\' Old/Similar', fontsize=14)

In [ ]:
pearsonr(x=neuro_data['bvmtr4_norm'], y=neuro_data['dprime_old_lure'])

In [ ]:
g=sns.lmplot(x='lm2_norm', y='dprime_old_lure', data=neuro_data)
plt.xlabel('LM Delay', fontsize=14)
plt.ylabel('d\' Old/Similar', fontsize=14)

In [ ]:
pearsonr(x=neuro_data['lm2_norm'], y=neuro_data['dprime_old_lure'])

In [ ]:
g=sns.lmplot(x='dr_comp', y='dprime_old_lure', data=neuro_data)
plt.xlabel('Delayed Recall', fontsize=14)
plt.ylabel('d\' Old/Similar', fontsize=14)

In [ ]:
pearsonr(x=neuro_data['dr_comp'], y=neuro_data['dprime_old_lure'])

In [ ]:
g=sns.lmplot(x='dr_comp', y='old_targ_old_lure', data=neuro_data)
plt.xlabel('Delayed Recall', fontsize=14)
plt.ylabel('Old/New Recognition', fontsize=14)

In [ ]:
g=sns.lmplot(x='dr_comp', y='old_targ_old_lure', data=neuro_data)
plt.xlabel('Delayed Recall', fontsize=14)
plt.ylabel('Old/Similar Recognition', fontsize=14)

In [ ]:
pearsonr(x=neuro_data['hvltr4_norm'], y=neuro_data['old_targ_old_lure'])

In [ ]:
pearsonr(x=neuro_data['bvmtr4_norm'], y=neuro_data['old_targ_old_lure'])

In [ ]:
pearsonr(x=neuro_data['lm2_norm'], y=neuro_data['old_targ_old_lure'])

In [ ]:
g=sns.lmplot(x='bvmtr4_norm', y='old_targ_old_lure', data=neuro_data)
plt.xlabel('BVMT Delayed Recall', fontsize=14)
plt.ylabel('Old/New Recognition', fontsize=14)

In [ ]:
g=sns.lmplot(x='lm2_norm', y='old_targ_old_lure', data=neuro_data)
plt.xlabel('LM Delayed Recall', fontsize=14)
plt.ylabel('Old/New Recognition', fontsize=14)

In [ ]:
pearsonr(x=neuro_data['dr_comp'], y=neuro_data['old_targ_old_foil'])

In [ ]:
g=sns.lmplot(x='dr_comp', y='bps', data=neuro_data)
plt.xlabel('Delayed Recall', fontsize=14)
plt.ylabel('BPS Score', fontsize=14)

In [ ]:
pearsonr(x=neuro_data['dr_comp'], y=neuro_data['bps'])

In [ ]:
g=sns.lmplot(x='dr_comp', y='old_targ_old_lure', data=neuro_data)
plt.xlabel('Delayed Recall', fontsize=14)
plt.ylabel('Old/Similar Recognition', fontsize=14)

In [ ]:
g=sns.lmplot(x='hvltr4_norm', y='old_targ_old_lure', data=neuro_data)
plt.xlabel('HVLT Delay', fontsize=14)
plt.ylabel('Old/Similar Recognition', fontsize=14)

In [ ]:
g=sns.lmplot(x='bvmtr4_norm', y='old_targ_old_lure', data=neuro_data)
plt.xlabel('BVMT Delay', fontsize=14)
plt.ylabel('Old/Similar Recognition', fontsize=14)

In [ ]:
g=sns.lmplot(x='lm2_norm', y='old_targ_old_lure', data=neuro_data)
plt.xlabel('LM Delay', fontsize=14)
plt.ylabel('Old/Similar Recognition', fontsize=14)

In [ ]:
pearsonr(x=neuro_data['dr_comp'], y=neuro_data['old_targ_old_lure'])

In [ ]:
pearsonr(x=neuro_data['hvltr4_norm'], y=neuro_data['old_targ_old_lure'])

In [ ]:
pearsonr(x=neuro_data['bvmtr4_norm'], y=neuro_data['old_targ_old_lure'])

In [ ]:
pearsonr(x=neuro_data['lm2_norm'], y=neuro_data['old_targ_old_lure'])

In [ ]:
g=sns.lmplot(x='recog_comp', y='dprime_bps', data=neuro_data)
plt.xlabel('Recognition', fontsize=14)
plt.ylabel('d\' BPS', fontsize=14)

In [ ]:
pearsonr(x=neuro_data['recog_comp'], y=neuro_data['dprime_bps'])

In [ ]:
g=sns.lmplot(x='recog_comp', y='dprime_old_new', data=neuro_data)
plt.xlabel('Recognition', fontsize=14)
plt.ylabel('d\' Old/New', fontsize=14)

In [ ]:
pearsonr(x=neuro_data['recog_comp'], y=neuro_data['dprime_old_new'])

In [ ]:
g=sns.lmplot(x='recog_comp', y='dprime_old_lure', data=neuro_data)
plt.xlabel('Recognition', fontsize=14)
plt.ylabel('d\' Old/Similar', fontsize=14)

In [ ]:
pearsonr(x=neuro_data['recog_comp'], y=neuro_data['dprime_old_lure'])

In [ ]:

def not_used(pidn):
        ## calculate traditional recognition score corrected for the number of trials responded to 

    #subtract the proportion of old|foil (corrected for no response) 
    #from the proportion of old|targ (corrected for no response)
    recog_c = prop_resp_old_targ - prop_resp_old_foil
    
     ## calculate BPS score corrected for the number of trials responded to 

    #subtract the proportion of sim|foil (corrected for no responses) 
    #from the proportion of sim|lure (corrected for no responses)
    bps_score_c = prop_resp_sim_lure - prop_resp_sim_foil
    #print(bps_score_c)
    
    ## calculate proportion of old|foil for foil trials responded to

    #divide the number of old|foil by the number of foil trials responded to 
    prop_resp_old_foil = num_old_foil/(num_foil_resp)
    #print(prop_resp_old_foil)

    
  

    ## calculate proportion of similar|foil for foil trials responded to

    #divide the number of similar|foil by the number of foil trials responded to
    prop_resp_sim_foil = num_sim_foil/(num_foil_resp)
    #print(prop_resp_sim_foil)

    
    #divide the number of new|foil by the number of foil trials responded to
    prop_resp_new_foil = num_new_foil/(num_foil_resp)
    #print(prop_resp_new_foil)

    
     ## calculate the proportion of similar|lure for lure trials responded to 

    #divide number of similar|lure by the number of lure trials responded to
    prop_resp_sim_lure = num_sim_lure/(num_lure_resp)
    #print(prop_resp_sim_lure)

  

    ## calculate proportion of new|lure for lure trials responded to 

    #divide the number of new|lure by the number of lure trials responded to
    prop_resp_new_lure = num_new_lure/(num_lure_resp)
    #print(prop_resp_new_lure)
    
    ## calculate the number of Target trials that were responded to 
    #subtract the number of no response to target trials from the number of target trials
    num_targ_resp = num_targ_trials - num_nr_targ
    #print(num_targ_resp)

    ## calculate the number of lure trials that were responded to 
    #subtract the number of no response to lure trials from the number of lure trials
    num_lure_resp = num_lure_trials - num_nr_lure
    #print(num_lure_resp)

    ## calculate the number of foil trials that were responded to 
    #subtract the number of no response to foil trials from the number of foil trials
    num_foil_resp = num_foil_trials - num_nr_foil
    #print(num_foil_resp) 
    
    ## calculate the total number of responses

    num_resp = num_targ_resp + num_lure_resp + num_foil_resp
    #print(num_resp)

     ## calculate the proportion of target trials responded to 
    #divide the number of target trials responded to by the total number of target trials
    prop_targ_resp = num_targ_resp/num_targ_trials
    #print(prop_targ_resp)
    prop_nr_targ = num_nr_targ/num_targ_trials
    all_data.loc[row, 'Prop NR Targ'] = prop_nr_targ
    
    
    ## calculate the proportion of lure trials that were responded to
    #divide the number of lure trials responded to by the total number of lure trials
    prop_lure_resp = num_lure_resp/num_lure_trials
    #print(prop_lure_resp)
    prop_nr_lure = num_nr_lure/num_lure_trials
    all_data.loc[row, 'Prop NR Lure'] = prop_nr_lure

    ## calculate the proportion of foil trials that were responded to 
    # divide the number of foil trials responded to by the total number of foil trials
    prop_foil_resp = num_foil_resp/num_foil_trials
    #print(prop_foil_resp)
    prop_nr_foil = num_nr_foil/num_foil_trials
    all_data.loc[row, 'Prop NR Foil'] = prop_nr_foil

    ## calculate the proportion of trials responded to 

    prop_resp = num_resp/(num_targ_trials + num_lure_trials + num_foil_trials)
   # print(prop_resp)

    #Divide the number of old|target by the total number of target trials
    prop_old_targ = num_old_targ/num_targ_trials
    all_data.loc[row, 'Prop Old|Targ'] = prop_old_targ

    ## calculate the proportion of Old|Target for Target Trials that were responded to 

    #divide the number of old|target by the number of target trials responded to
    prop_resp_old_targ = num_old_targ/(num_targ_resp)
    #print(prop_resp_old_targ)

    #divide the number of similar|target by the number of target trials that were responded to 
    prop_resp_sim_targ = num_sim_targ/(num_targ_resp)
    #print(prop_resp_sim_targ)
    
       ## calculate the proportion of New|Target for target trials responded to 

    #divide the number of New|Target by the number of target trials responded to 
    prop_resp_new_targ = num_new_targ/(num_targ_resp)
    #print(prop_resp_new_targ)
    
      ## calculate the proportion of Old|Lure for lure trials responded to 

    #divide number of Old|Lure by the number of lure trials responded to
    prop_resp_old_lure = num_old_lure/(num_lure_resp)
    #print(prop_resp_old_lure)
    
    return 0

In [23]:
print(data_long[data_long['RT']<400])

       Trial   Img    Cond  LBin     Resp        Acc   RT  pidn
6          7  137b    Lure     5  No Resp  Incorrect    0   449
12        13  146a  Target     0  No Resp  Incorrect    0   449
16        17  136a  Target     0  No Resp  Incorrect    0   449
17        18  139a  Target     0      Old    Correct    4   449
18        19  054b    Lure     5      Old  Incorrect    5   449
23        24  060b    Lure     5  No Resp  Incorrect    0   449
25        26  088b    Lure     4  No Resp  Incorrect    0   449
44        45  190b    Lure     4  No Resp  Incorrect    0   449
45        46  072a    Foil     0  Similar  Incorrect    4   449
68        69  049a  Target     0  No Resp  Incorrect    0   449
100      101  172a    Foil     0  No Resp  Incorrect    0   449
104      105  096b    Lure     1  No Resp  Incorrect    0   449
121      122  078a    Foil     0  No Resp  Incorrect    0   449
133      134  132b    Lure     5  No Resp  Incorrect    0   449
148      149  151b    Lure     5  No Res

In [1]:
data_corr=pd.DataFrame()
np_data_corr=pd.DataFrame()
row=0
missing=0
has_data=0
n_stim_per=64
study_len = n_stim_per*2
test_len = n_stim_per*3
demo_data = pd.read_csv('/Users/madisonhunt/Desktop/Oak/stark_task/stark_demographics.csv')
pidn_list = demo_data['pidn']
age_list = demo_data['np_age']
ed_list = demo_data['yrs_educ']
gender_list = demo_data['gender']

for pidn in pidn_list:

    if pidn < 685:
        filename = '/Users/madisonhunt/Desktop/Oak/stark_task/Data/BPSOlog_0{}.txt'.format(pidn)
#         print(filename)
    elif pidn > 685:
        filename = '/Users/madisonhunt/Desktop/Oak/stark_task/Data/MSTlog_0{}.txt'.format(pidn)
#         print(filename)
        if pidn > 1000: 
            filename = '/Users/madisonhunt/Desktop/Oak/stark_task/Data/MSTlog_{}.txt'.format(pidn)
    #         print(filename)

    if exists(filename):

        has_data +=1
        data_corr.loc[row, 'pidn'] = demo_data.loc[row, 'pidn']
        data_corr.loc[row, 'mmse'] = demo_data.loc[row, 'mmse']
        data_corr.loc[row, 'hvltr4'] = demo_data.loc[row, 'hvltr4']
        data_corr.loc[row, 'hvltpr'] = demo_data.loc[row, 'hvltrpr']
        data_corr.loc[row, 'bvmtr4'] = demo_data.loc[row, 'bvmtr4']
        data_corr.loc[row, 'bvmtpr'] = demo_data.loc[row, 'bvmtrpr']
        data_corr.loc[row, 'lm2'] = demo_data.loc[row, 'lm2']
        data_corr.loc[row, 'lmpr'] = demo_data.loc[row, 'lmpr']
        data_corr.loc[row, 'mmse_norm'] = demo_data.loc[row, 'mmse_norm']
        data_corr.loc[row, 'hvltr4_norm'] = demo_data.loc[row, 'hvltr4_norm']
        data_corr.loc[row, 'bvmtr4_norm'] = demo_data.loc[row, 'bvmtr4_norm']
        data_corr.loc[row, 'lm2_norm'] = demo_data.loc[row, 'lm2_norm']
        data_corr.loc[row, 'ir_comp'] = demo_data.loc[row, 'immediate_recall_comp']
        data_corr.loc[row, 'dr_comp'] = demo_data.loc[row, 'delayed_recall_comp']
        data_corr.loc[row, 'recog_comp'] = demo_data.loc[row, 'recognition_comp']
        

        s_starts, t_starts = findlines(filename)
        if len(s_starts) & len(t_starts):
            study_starts = s_starts[-1]
            test_starts = t_starts[-1]
            pt_data =  pd.read_csv(filename, delimiter="\t", skiprows= test_starts, nrows=test_len, header = 0)     

            data_corr.loc[row, 'pidn'] = pidn 
            data_corr.loc[row, 'age'] = age_list[row]
            data_corr.loc[row, 'education'] = ed_list[row]
            if gender_list[row] == 0:
                data_corr.loc[row, 'gender'] = 'F'
            else:
                data_corr.loc[row,'gender'] = 'M'
                
            data_corr.loc[row, 'prop_nr'] = all_data.loc[row, 'prop_nr']
            ## for no NR
#             data_corr.loc[row, 'num_targ_tr']  = (all_data.loc[row, 'num_targ_tr'] - all_data.loc[row, 'num_nr_t'])     
#             data_corr.loc[row, 'num_foil_tr'] = (all_data.loc[row, 'num_foil_tr'] - all_data.loc[row, 'num_nr_f']) 
#             data_corr.loc[row, 'num_lure_tr'] = (all_data.loc[row,'num_lure_tr'] - all_data.loc[row,'num_nr_l'])

            data_corr.loc[row, 'num_targ_tr']  = len(data_long[(data_long['pidn']==pidn) & (data_long['Cond']=='Target')])
            data_corr.loc[row, 'num_lure_tr'] = len(data_long[(data_long['pidn']==pidn) & (data_long['Cond']=='Lure')])
            data_corr.loc[row, 'num_foil_tr'] = len(data_long[(data_long['pidn']==pidn) & (data_long['Cond']=='Foil')])
            data_corr.loc[row, 'p_old_targ'] = (all_data.loc[row, 'n_old_targ']/data_corr.loc[row, 'num_targ_tr']) 
            data_corr.loc[row, 'p_sim_targ'] = all_data.loc[row, 'n_sim_targ']/data_corr.loc[row, 'num_targ_tr']
            data_corr.loc[row, 'p_new_targ'] = all_data.loc[row, 'n_new_targ']/data_corr.loc[row, 'num_targ_tr']
            data_corr.loc[row, 'p_old_lure'] = all_data.loc[row, 'n_old_lure']/data_corr.loc[row, 'num_lure_tr']
            data_corr.loc[row, 'p_sim_lure'] = all_data.loc[row, 'n_sim_lure']/data_corr.loc[row, 'num_lure_tr']
            data_corr.loc[row, 'p_new_lure']= all_data.loc[row, 'n_new_lure']/data_corr.loc[row, 'num_lure_tr']
            data_corr.loc[row, 'p_old_foil']= all_data.loc[row, 'n_old_foil']/data_corr.loc[row, 'num_foil_tr']
            data_corr.loc[row, 'p_sim_foil'] = all_data.loc[row, 'n_sim_foil']/data_corr.loc[row, 'num_foil_tr']
            data_corr.loc[row, 'p_new_foil'] = all_data.loc[row, 'n_new_foil']/data_corr.loc[row, 'num_foil_tr']
            data_corr.loc[row, 'p_correct'] = (all_data.loc[row, 'n_old_targ']+all_data.loc[row, 'n_sim_lure']+all_data.loc[row, 'n_new_foil'])/(data_corr.loc[row, 'num_targ_tr']+data_corr.loc[row,'num_lure_tr']+data_corr.loc[row,'num_foil_tr'])
            data_corr.loc[row, 'bps'] = calc_bps(data_corr.loc[row, 'p_sim_lure'], data_corr.loc[row, 'p_sim_foil'])
            [data_corr.loc[row, 'old_targ_old_foil'], data_corr.loc[row, 'old_targ_old_lure'],
             data_corr.loc[row,'old_lure_sim_lure']] = recog_scores(data_corr.loc[row, 'p_old_targ'], data_corr.loc[row, 'p_old_foil'], data_corr.loc[row,'p_old_lure'],  data_corr.loc[row, 'p_sim_lure'])
            data_corr.loc[row, 'dprime_old_new'] = dprime(snodgrass(all_data.loc[row, 'n_old_targ'], data_corr.loc[row, 'num_targ_tr']), snodgrass(all_data.loc[row, 'n_old_foil'], data_corr.loc[row, 'num_foil_tr']))
            data_corr.loc[row,'dprime_old_lure'] = dprime(snodgrass(all_data.loc[row, 'n_old_targ'], data_corr.loc[row, 'num_targ_tr']), snodgrass(all_data.loc[row, 'n_old_lure'], data_corr.loc[row, 'num_lure_tr']))
            data_corr.loc[row, 'dprime_bps'] = dprime(snodgrass(all_data.loc[row, 'n_sim_lure'], data_corr.loc[row, 'num_lure_tr']), snodgrass(all_data.loc[row, 'n_sim_foil'], data_corr.loc[row, 'num_foil_tr']))
            row +=1
        else: print(pidn)
    else:
#         print(pidn)
        missing +=1
# print(data_corr['dprime_old_new'])


NameError: name 'pd' is not defined

In [25]:
data_corrected = data_corr[data_corr['dprime_old_new']>.5]


#confirmed now absent

# create factor variable reflecting nr status for a given person


data_corrected.loc[data_corrected['prop_nr']>=.20, 'nr_group'] = 'nr_.20'
data_corrected.loc[(data_corrected['prop_nr']>=.15)&(data_corrected['prop_nr']<.20), 'nr_group'] = 'nr_.15'
data_corrected.loc[(data_corrected['prop_nr']>=.10)&(data_corrected['prop_nr']<.15), 'nr_group'] = 'nr_.10'
data_corrected.loc[(data_corrected['prop_nr']>=.05)&(data_corrected['prop_nr']<.10), 'nr_group'] = 'nr_.05'
data_corrected.loc[(data_corrected['prop_nr']<.05), 'nr_group'] = 'nr_low'

In [26]:
data_corrected=data_corrected.merge(data_med_RT, on='pidn')

NameError: name 'data_med_RT' is not defined

In [ ]:
data_corrected['outliers'] = zscore(data_corrected['RT'])

In [ ]:
print(data_corrected[abs(data_corrected['outliers']) >2.5]['pidn'])
print(data_corrected[abs(data_corrected['outliers']) >2.5]['Cond'])
print(data_corrected[abs(data_corrected['outliers']) >2.5]['Resp'])

In [ ]:
data_out= data_corrected[data_corrected['outliers']>-2.5]
print(data_out)

In [ ]:
g= sns.lmplot(x='RT', y='dprime_bps_x', data=data_out[(data_out['Cond']=='Lure') & (data_out['Resp']=='Similar')], hue='nr_group', hue_order=['nr_low', 'nr_.05', 'nr_.10', 'nr_.15', 'nr_.20'], fit_reg=False)
g=sns.regplot(x="RT", y="dprime_bps_x", data=data_out, scatter=False)
plt.ylabel('d\' BPS')

In [ ]:
g= sns.lmplot(x='RT', y='dprime_bps_x', data=data_out[(data_out['Cond']=='Lure') & (data_out['Resp']=='New')], hue='nr_group', hue_order=['nr_low', 'nr_.05', 'nr_.10', 'nr_.15', 'nr_.20'], fit_reg=False)
g=sns.regplot(x="RT", y="dprime_bps_x", data=data_out, scatter=False)
plt.ylabel('d\' BPS')

In [ ]:
g= sns.lmplot(x='RT', y='dprime_bps_x', data=data_corrected[(data_corrected['Cond']=='Lure') & (data_corrected['Resp']=='Old')], hue='nr_group', hue_order=['nr_low', 'nr_.05', 'nr_.10', 'nr_.15', 'nr_.20'], fit_reg=False)
g=sns.regplot(x="RT", y="dprime_bps_x", data=data_corrected, scatter=False)
plt.ylabel('d\' BPS')


In [ ]:
g= sns.lmplot(x='RT', y='dprime_bps_x', data=data_corrected[(data_corrected['Cond']=='Lure') & (data_corrected['Resp']=='Similar')], hue='nr_group', hue_order=['nr_low', 'nr_.05', 'nr_.10', 'nr_.15', 'nr_.20'], fit_reg=False)
g=sns.regplot(x="RT", y="dprime_bps_x", data=data_corrected, scatter=False)
plt.ylabel('d\' BPS')

In [ ]:
g= sns.lmplot(x='RT', y='dprime_bps_x', data=data_corrected[(data_corrected['Cond']=='Lure') & (data_corrected['Resp']=='New')], hue='nr_group', hue_order=['nr_low', 'nr_.05', 'nr_.10', 'nr_.15', 'nr_.20'], fit_reg=False)
g=sns.regplot(x="RT", y="dprime_bps_x", data=data_corrected, scatter=False)
plt.ylabel('d\' BPS')

In [ ]:
g= sns.lmplot(x='RT', y='dprime_bps_x', data=data_corrected[data_corrected['Cond']=='Lure'], hue='Acc')
plt.ylabel('d\' BPS')

In [ ]:
am_data=pd.read_csv('/Users/madisonhunt/Desktop/md_np_am_data.csv', usecols=['pidn','associative_dprime', 'overall_dprime'])

In [ ]:
data = data_corrected.merge(am_data, on=['pidn'])

In [ ]:
data.head()

In [ ]:
g=sns.lmplot(x='overall_dprime', y='dprime_old_new_x', data=data, hue='nr_group',fit_reg=False, hue_order=['nr_low', 'nr_.05','nr_.10' ,'nr_.15','nr_.20'])
g=sns.regplot(x="overall_dprime", y="dprime_old_new_x", data=data, scatter=False, ax=g.axes[0, 0])
plt.ylabel('d\' O/N')
plt.xlabel('Overall d\'')

In [ ]:
data=data.dropna()

In [ ]:
pearsonr(x=data['overall_dprime'], y=data['dprime_old_new_x'])

In [ ]:
g=sns.lmplot(x='overall_dprime', y='dprime_bps_x', data=data, hue='nr_group',fit_reg=False, hue_order=['nr_low', 'nr_.05','nr_.10' ,'nr_.15','nr_.20'])
g=sns.regplot(x="overall_dprime", y="dprime_bps_x", data=data, scatter=False, ax=g.axes[0, 0])
plt.ylabel('d\' BPS')
plt.xlabel('Overall d\'')

In [ ]:
pearsonr(x=data['overall_dprime'], y=data['dprime_bps_x'])

In [ ]:
g=sns.lmplot(x='overall_dprime', y='dprime_old_lure_x', data=data, hue='nr_group',fit_reg=False, hue_order=['nr_low', 'nr_.05','nr_.10' ,'nr_.15','nr_.20'])
g=sns.regplot(x="overall_dprime", y="dprime_old_lure_x", data=data, scatter=False, ax=g.axes[0, 0])
plt.ylabel('d\' O/S')
plt.xlabel('Overall d\'')

In [ ]:
pearsonr(x=data['overall_dprime'], y=data['dprime_old_lure_x'])

In [ ]:
g=sns.lmplot(x='associative_dprime', y='dprime_old_new_x', data=data, hue='nr_group',fit_reg=False, hue_order=['nr_low', 'nr_.05','nr_.10' ,'nr_.15','nr_.20'])
g=sns.regplot(x="associative_dprime", y="dprime_old_new_x", data=data, scatter=False, ax=g.axes[0, 0])
plt.xlabel('Associative d\'')
plt.ylabel('d\' O/N')

In [ ]:
pearsonr(x=data['associative_dprime'], y=data['dprime_old_new_x'])

In [ ]:
pearsonr(x=data['associative_dprime'], y=data['dprime_bps_x'])

In [ ]:
pearsonr(x=data['associative_dprime'], y=data['dprime_old_lure_x'])

In [ ]:
g=sns.lmplot(x='associative_dprime', y='dprime_bps_x', data=data, hue='nr_group',fit_reg=False, hue_order=['nr_low', 'nr_.05','nr_.10' ,'nr_.15','nr_.20'])
g=sns.regplot(x="associative_dprime", y="dprime_bps_x", data=data, scatter=False, ax=g.axes[0, 0])
plt.xlabel('Associative d\'')
plt.ylabel('d\' BPS')

In [ ]:
g=sns.lmplot(x='associative_dprime', y='dprime_old_lure_x', data=data, hue='nr_group',fit_reg=False, hue_order=['nr_low', 'nr_.05','nr_.10' ,'nr_.15','nr_.20'])
g=sns.regplot(x="associative_dprime", y="dprime_old_lure_x", data=data, scatter=False, ax=g.axes[0, 0])
plt.xlabel('Associative d\'')
plt.ylabel('d\' O/S')

In [ ]:

g= sns.lmplot(x='recog_comp', y='dprime_old_new_x', hue='nr_group', data=data_corrected, fit_reg=False, hue_order=['nr_low', 'nr_.05','nr_.10' ,'nr_.15','nr_.20'
                                                                                                               ] )
g=sns.regplot(x="recog_comp", y="dprime_old_new_x", data=data_corrected, scatter=False, ax=g.axes[0, 0])
plt.xlabel('Recognition')
plt.ylabel('d\' O/N')

In [ ]:
g= sns.lmplot(x='recog_comp', y='dprime_bps_x', hue='nr_group', data=data_corrected, fit_reg=False,hue_order=['nr_low', 'nr_.05','nr_.10' ,'nr_.15','nr_.20'
                                                                                                               ] )
g=sns.regplot(x="recog_comp", y="dprime_bps_x", data=data_corrected, scatter=False, ax=g.axes[0, 0])
plt.xlabel('Recognition')
plt.ylabel('d\' BPS')

In [ ]:
g= sns.lmplot(x='recog_comp', y='dprime_old_lure_x', hue='nr_group', data=data_corrected, fit_reg=False , hue_order=['nr_low', 'nr_.05','nr_.10' ,'nr_.15','nr_.20'
                                                                                                               ])
g=sns.regplot(x="recog_comp", y="dprime_old_lure_x", data=data_corrected, scatter=False, ax=g.axes[0, 0])
plt.xlabel('Recognition')
plt.ylabel('d\' O/S')

In [ ]:
g= sns.lmplot(x='ir_comp', y='dprime_old_new_x', hue='nr_group', data=data_corrected, fit_reg=False, hue_order=['nr_low', 'nr_.05','nr_.10' ,'nr_.15','nr_.20'
                                                                                                               ] )
g=sns.regplot(x="ir_comp", y="dprime_old_new_x", data=data_corrected, scatter=False, ax=g.axes[0, 0])
plt.xlabel('Immediate Recall')
plt.ylabel('d\' O/N')

In [ ]:
g= sns.lmplot(x='ir_comp', y='dprime_bps_x', hue='nr_group', data=data_corrected, fit_reg=False,hue_order=['nr_low', 'nr_.05','nr_.10' ,'nr_.15','nr_.20'
                                                                                                               ] )
g=sns.regplot(x="ir_comp", y="dprime_bps_x", data=data_corrected, scatter=False, ax=g.axes[0, 0])
plt.xlabel('Immediate Recall')
plt.ylabel('d\' BPS')

In [ ]:
g= sns.lmplot(x='ir_comp', y='dprime_old_lure_x', hue='nr_group', data=data_corrected, fit_reg=False , hue_order=['nr_low', 'nr_.05','nr_.10' ,'nr_.15','nr_.20'
                                                                                                               ])
g=sns.regplot(x="ir_comp", y="dprime_old_lure_x", data=data_corrected, scatter=False, ax=g.axes[0, 0])
plt.xlabel('Immediate Recall')
plt.ylabel('d\' O/S')

In [ ]:
g= sns.lmplot(x='dr_comp', y='dprime_old_new_x', hue='nr_group', data=data_corrected, fit_reg=False, hue_order=['nr_low', 'nr_.05','nr_.10' ,'nr_.15','nr_.20'
                                                                                                               ] )
g=sns.regplot(x="dr_comp", y="dprime_old_new_x", data=data_corrected, scatter=False, ax=g.axes[0, 0])
plt.xlabel('Delayed Recall')
plt.ylabel('d\' O/N')

In [ ]:
pearsonr(x=data_corrected['recog_comp'], y=data_corrected['dprime_old_new_x'])

In [ ]:
pearsonr(x=data_corrected['recog_comp'], y=data_corrected['dprime_old_lure_x'])

In [ ]:
pearsonr(x=data_corrected['recog_comp'], y=data_corrected['dprime_bps_x'])

In [ ]:
g= sns.lmplot(x='dr_comp', y='dprime_bps_x', hue='nr_group', data=data_corrected, fit_reg=False,hue_order=['nr_low', 'nr_.05','nr_.10' ,'nr_.15','nr_.20'
                                                                                                               ] )
g=sns.regplot(x="dr_comp", y="dprime_bps_x", data=data_corrected, scatter=False, ax=g.axes[0, 0])
plt.xlabel('Delayed Recall')
plt.ylabel('d\' BPS')

In [ ]:
g= sns.lmplot(x='dr_comp', y='dprime_old_lure_x', hue='nr_group', data=data_corrected, fit_reg=False , hue_order=['nr_low', 'nr_.05','nr_.10' ,'nr_.15','nr_.20'
                                                                                                               ])
g=sns.regplot(x="dr_comp", y="dprime_old_lure_x", data=data_corrected, scatter=False, ax=g.axes[0, 0])
plt.xlabel('Delayed Recall')
plt.ylabel('d\' O/S')

In [ ]:
g= sns.lmplot(x='education_x', y='dprime_old_new_x', hue='nr_group', data=data_corrected, fit_reg=False, hue_order=['nr_low', 'nr_.05','nr_.10' ,'nr_.15','nr_.20'
                                                                                                               ] )
g=sns.regplot(x="education_x", y="dprime_old_new_x", data=data_corrected, scatter=False, ax=g.axes[0, 0])
plt.xlabel('Education')
plt.ylabel('d\' O/N')

In [ ]:
g= sns.lmplot(x='education_x', y='dprime_bps_x', hue='nr_group', data=data_corrected, fit_reg=False,hue_order=['nr_low', 'nr_.05','nr_.10' ,'nr_.15','nr_.20'
                                                                                                               ] )
g=sns.regplot(x="education_x", y="dprime_bps_x", data=data_corrected, scatter=False, ax=g.axes[0, 0])
plt.xlabel('Education')
plt.ylabel('d\' BPS')

In [ ]:
g= sns.lmplot(x='education_x', y='dprime_old_lure_x', hue='nr_group', data=data_corrected, fit_reg=False , hue_order=['nr_low', 'nr_.05','nr_.10' ,'nr_.15','nr_.20'
                                                                                                               ])
g=sns.regplot(x="education_x", y="dprime_old_lure_x", data=data_corrected, scatter=False, ax=g.axes[0, 0])
plt.xlabel('Education')
plt.ylabel('d\' O/S')

In [ ]:
g= sns.lmplot(x='age_x', y='dprime_old_new_x', hue='nr_group', data=data_corrected, fit_reg=False, hue_order=['nr_low', 'nr_.05','nr_.10' ,'nr_.15','nr_.20'
                                                                                                               ] )
g=sns.regplot(x="age_x", y="dprime_old_new_x", data=data_corrected, scatter=False, ax=g.axes[0, 0])
plt.xlabel('Age')
plt.ylabel('d\' O/N')

In [ ]:
g= sns.lmplot(x='age_x', y='dprime_bps_x', hue='nr_group', data=data_corrected, fit_reg=False,hue_order=['nr_low', 'nr_.05','nr_.10' ,'nr_.15','nr_.20'
                                                                                                               ] )
g=sns.regplot(x="age_x", y="dprime_bps_x", data=data_corrected, scatter=False, ax=g.axes[0, 0])
plt.xlabel('Age')
plt.ylabel('d\' BPS')

In [ ]:
g= sns.lmplot(x='age_x', y='dprime_old_lure_x', hue='nr_group', data=data_corrected, fit_reg=False , hue_order=['nr_low', 'nr_.05','nr_.10' ,'nr_.15','nr_.20'
                                                                                                               ])
g=sns.regplot(x="age_x", y='dprime_old_lure_x', data=data_corrected, scatter=False, ax=g.axes[0, 0])
plt.xlabel('Age')
plt.ylabel('d\' O/S')

In [ ]:
data_corrected.head()

In [ ]:
# plt.figure(figsize=(20, 10))
g=sns.lmplot(x='RT', y='p_old_targ_x', data=data_corrected[(data_corrected['Cond']=='Target')], hue='Acc', height=7)
plt.xlabel('Median RT', size=16)
plt.ylabel('Correct (%)', size=16)
g=sns.lmplot(x='RT', y='p_sim_lure_x', data=data_corrected[data_corrected['Cond']=='Lure'] , hue='Acc', height=7 )
plt.xlabel('Median RT', size=16)
plt.ylabel('Correct (%)', size=16)
g=sns.lmplot(x='RT', y='p_new_foil_x', data=data_corrected[data_corrected['Cond']=='Foil'] , hue='Acc', height=7 )
plt.xlabel('Median RT', size=16)
plt.ylabel('Correct (%)', size=16)

In [ ]:
g=sns.lmplot(x='RT', y='p_old_targ_x', data=data_corrected[(data_corrected['Cond']=='Target')& (data_corrected['Acc']=='Correct')], hue='nr_group', height=7, hue_order=['nr_low', 'nr_.05', 'nr_.10', 'nr_.15', 'nr_.20'], fit_reg=False)
g=sns.regplot(x="RT", y="p_old_targ_x", data=data_corrected[(data_corrected['Cond']=='Target')& (data_corrected['Acc']=='Correct')], scatter=False, ax=g.axes[0, 0])
plt.xlabel('Median RT', size=16)
plt.ylabel('Correct (%)', size=16)


In [ ]:
pearsonr(x=data_corrected[(data_corrected['Cond']=='Target')& (data_corrected['Acc']=='Correct')]['RT'], y= data_corrected[(data_corrected['Cond']=='Target')& (data_corrected['Acc']=='Correct')]['p_old_targ_x'])

In [ ]:
pearsonr(x=data_corrected[(data_corrected['Cond']=='Target')& (data_corrected['Acc']=='Incorrect')]['RT'], y= data_corrected[(data_corrected['Cond']=='Target')& (data_corrected['Acc']=='Incorrect')]['p_old_targ_x'])

In [ ]:
g=sns.lmplot(x='RT', y='p_old_targ_x', data=data_corrected[(data_corrected['Cond']=='Target')& (data_corrected['Acc']=='Incorrect')], hue='nr_group', height=7, hue_order=['nr_low', 'nr_.05', 'nr_.10', 'nr_.15', 'nr_.20'], fit_reg=False)
g=sns.regplot(x="RT", y="p_old_targ_x", data=data_corrected[(data_corrected['Cond']=='Target')& (data_corrected['Acc']=='Incorrect')], scatter=False, ax=g.axes[0, 0])
plt.xlabel('Median RT', size=16)
plt.ylabel('Correct (%)', size=16)


In [ ]:
g=sns.lmplot(x='RT', y='p_sim_lure_x', data=data_corrected[(data_corrected['Cond']=='Lure')& (data_corrected['Acc']=='Correct')], hue='nr_group', height=7, hue_order=['nr_low', 'nr_.05', 'nr_.10', 'nr_.15', 'nr_.20'], fit_reg=False)
g=sns.regplot(x="RT", y="p_sim_lure_x", data=data_corrected[(data_corrected['Cond']=='Lure')& (data_corrected['Acc']=='Correct')], scatter=False, ax=g.axes[0, 0])
plt.xlabel('Median RT', size=16)
plt.ylabel('Correct (%)', size=16)


In [ ]:
g=sns.lmplot(x='RT', y='p_sim_lure_x', data=data_corrected[(data_corrected['Cond']=='Lure')& (data_corrected['Acc']=='Incorrect')], hue='nr_group', height=7, hue_order=['nr_low', 'nr_.05', 'nr_.10', 'nr_.15', 'nr_.20'], fit_reg=False)
g=sns.regplot(x="RT", y="p_sim_lure_x", data=data_corrected[(data_corrected['Cond']=='Lure')& (data_corrected['Acc']=='Incorrect')], scatter=False, ax=g.axes[0, 0])
plt.xlabel('Median RT', size=16)
plt.ylabel('Correct (%)', size=16)


In [ ]:
pearsonr(x=data_corrected[(data_corrected['Cond']=='Lure')& (data_corrected['Acc']=='Correct')]['RT'], y= data_corrected[(data_corrected['Cond']=='Lure')& (data_corrected['Acc']=='Correct')]['p_sim_lure_x'])

In [ ]:
pearsonr(x=data_corrected[(data_corrected['Cond']=='Lure')& (data_corrected['Acc']=='Incorrect')]['RT'], y= data_corrected[(data_corrected['Cond']=='Lure')& (data_corrected['Acc']=='Incorrect')]['p_sim_lure_x'])

In [ ]:
g=sns.lmplot(x='RT', y='p_new_foil_x', data=data_corrected[(data_corrected['Cond']=='Foil')& (data_corrected['Acc']=='Correct')], hue='nr_group', height=7, hue_order=['nr_low', 'nr_.05', 'nr_.10', 'nr_.15', 'nr_.20'], fit_reg=False)
g=sns.regplot(x="RT", y="p_new_foil_x", data=data_corrected[(data_corrected['Cond']=='Foil')& (data_corrected['Acc']=='Correct')], scatter=False, ax=g.axes[0, 0])
plt.xlabel('Median RT', size=16)
plt.ylabel('Correct (%)', size=16)


In [ ]:
g=sns.lmplot(x='RT', y='p_new_foil_x', data=data_corrected[(data_corrected['Cond']=='Foil')& (data_corrected['Acc']=='Incorrect')], hue='nr_group', height=7, hue_order=['nr_low', 'nr_.05', 'nr_.10', 'nr_.15', 'nr_.20'], fit_reg=False)
g=sns.regplot(x="RT", y="p_new_foil_x", data=data_corrected[(data_corrected['Cond']=='Foil')& (data_corrected['Acc']=='Incorrect')], scatter=False, ax=g.axes[0, 0])
plt.xlabel('Median RT', size=16)
plt.ylabel('Correct (%)', size=16)


In [ ]:
pearsonr(x=data_corrected[(data_corrected['Cond']=='Foil')& (data_corrected['Acc']=='Correct')]['RT'], y= data_corrected[(data_corrected['Cond']=='Foil')& (data_corrected['Acc']=='Correct')]['p_new_foil_x'])

In [ ]:
pearsonr(x=data_corrected[(data_corrected['Cond']=='Foil')& (data_corrected['Acc']=='Incorrect')]['RT'], y= data_corrected[(data_corrected['Cond']=='Foil')& (data_corrected['Acc']=='Incorrect')]['p_new_foil_x'])

In [ ]:
Age

In [ ]:
data_corrected.head()

In [ ]:
g=sns.lmplot(x='age_x', y='p_old_targ_x', data=data_corrected[(data_corrected['Cond']=='Target')& (data_corrected['Acc']=='Correct')], hue='nr_group', hue_order=['nr_low', 'nr_.05', 'nr_.10', 'nr_.15', 'nr_.20'], fit_reg=False)
g=sns.regplot(x="age_x", y="p_old_targ_x", data=data_corrected[(data_corrected['Cond']=='Target')& (data_corrected['Acc']=='Correct')], scatter=False, ax=g.axes[0, 0])
plt.xlabel('Age', size=16)
plt.ylabel('Correct (%)', size=16)
g.set(ylim=[0,1.1])


In [ ]:
g=sns.lmplot(x='age_x', y='p_sim_lure_x', data=data_corrected[(data_corrected['Cond']=='Lure')& (data_corrected['Acc']=='Correct')], hue='nr_group',  hue_order=['nr_low', 'nr_.05', 'nr_.10', 'nr_.15', 'nr_.20'], fit_reg=False)
g=sns.regplot(x="age_x", y="p_sim_lure_x", data=data_corrected[(data_corrected['Cond']=='Lure')& (data_corrected['Acc']=='Correct')], scatter=False, ax=g.axes[0, 0])
plt.xlabel('Age', size=16)
plt.ylabel('Correct (%)', size=16)
g.set(ylim=[0,1.1])


In [ ]:
g=sns.lmplot(x='age_x', y='p_new_foil_x', data=data_corrected[(data_corrected['Cond']=='Foil')& (data_corrected['Acc']=='Correct')], hue='nr_group', hue_order=['nr_low', 'nr_.05', 'nr_.10', 'nr_.15', 'nr_.20'], fit_reg=False)
g=sns.regplot(x="age_x", y="p_new_foil_x", data=data_corrected[(data_corrected['Cond']=='Foil')& (data_corrected['Acc']=='Correct')], scatter=False, ax=g.axes[0, 0])
plt.xlabel('Age', size=16)
plt.ylabel('Correct (%)', size=16)
g.set(ylim=[0,1.1])

In [ ]:
g=sns.lmplot(x='age_x', y='RT', data=data_corrected[(data_corrected['Cond']=='Target')& (data_corrected['Acc']=='Correct')], hue='nr_group', hue_order=['nr_low', 'nr_.05', 'nr_.10', 'nr_.15', 'nr_.20'], fit_reg=False)
g=sns.regplot(x="age_x", y="RT", data=data_corrected[(data_corrected['Cond']=='Target')& (data_corrected['Acc']=='Correct')], scatter=False, ax=g.axes[0, 0])
plt.xlabel('Age', size=16)
plt.ylabel('RT', size=16)



In [ ]:
g=sns.lmplot(x='age_x', y='RT', data=data_corrected[(data_corrected['Cond']=='Lure')& (data_corrected['Acc']=='Correct')], hue='nr_group',  hue_order=['nr_low', 'nr_.05', 'nr_.10', 'nr_.15', 'nr_.20'], fit_reg=False)
g=sns.regplot(x="age_x", y="RT", data=data_corrected[(data_corrected['Cond']=='Lure')& (data_corrected['Acc']=='Correct')], scatter=False, ax=g.axes[0, 0])
plt.xlabel('Age', size=16)
plt.ylabel('RT', size=16)



In [ ]:
g=sns.lmplot(x='age_x', y='RT', data=data_corrected[(data_corrected['Cond']=='Foil')& (data_corrected['Acc']=='Correct')], hue='nr_group', hue_order=['nr_low', 'nr_.05', 'nr_.10', 'nr_.15', 'nr_.20'], fit_reg=False)
g=sns.regplot(x="age_x", y="RT", data=data_corrected[(data_corrected['Cond']=='Foil')& (data_corrected['Acc']=='Correct')], scatter=False, ax=g.axes[0, 0])
plt.xlabel('Age', size=16)
plt.ylabel('RT', size=16)


In [ ]:
pearsonr(x=data_corrected[(data_corrected['Cond']=='Target')& (data_corrected['Acc']=='Correct')]['age_x'], y = data_corrected[(data_corrected['Cond']=='Target')& (data_corrected['Acc']=='Correct')]['RT'])

In [ ]:
pearsonr(x=data_corrected[(data_corrected['Cond']=='Foil')& (data_corrected['Acc']=='Correct')]['age_x'], y = data_corrected[(data_corrected['Cond']=='Foil')& (data_corrected['Acc']=='Correct')]['RT'])

In [ ]:
g=sns.lmplot(x='age_x', y='RT', data=data_corrected[(data_corrected['Cond']=='Target')& (data_corrected['Acc']=='Incorrect')], hue='nr_group', hue_order=['nr_low', 'nr_.05', 'nr_.10', 'nr_.15', 'nr_.20'], fit_reg=False)
g=sns.regplot(x="age_x", y="RT", data=data_corrected[(data_corrected['Cond']=='Target')& (data_corrected['Acc']=='Incorrect')], scatter=False, ax=g.axes[0, 0])
plt.xlabel('Age', size=16)
plt.ylabel('RT', size=16)



In [ ]:
g=sns.lmplot(x='age_x', y='RT', data=data_corrected[(data_corrected['Cond']=='Lure')& (data_corrected['Acc']=='Incorrect')], hue='nr_group',  hue_order=['nr_low', 'nr_.05', 'nr_.10', 'nr_.15', 'nr_.20'], fit_reg=False)
g=sns.regplot(x="age_x", y="RT", data=data_corrected[(data_corrected['Cond']=='Lure')& (data_corrected['Acc']=='Incorrect')], scatter=False, ax=g.axes[0, 0])
plt.xlabel('Age', size=16)
plt.ylabel('RT', size=16)



In [ ]:
g=sns.lmplot(x='age_x', y='RT', data=data_corrected[(data_corrected['Cond']=='Foil')& (data_corrected['Acc']=='Incorrect')], hue='nr_group', hue_order=['nr_low', 'nr_.05', 'nr_.10', 'nr_.15', 'nr_.20'], fit_reg=False)
g=sns.regplot(x="age_x", y="RT", data=data_corrected[(data_corrected['Cond']=='Foil')& (data_corrected['Acc']=='Incorrect')], scatter=False, ax=g.axes[0, 0])
plt.xlabel('Age', size=16)
plt.ylabel('RT', size=16)


In [ ]:
pearsonr(x=data_corrected[(data_corrected['Cond']=='Foil')& (data_corrected['Acc']=='Incorrect')]['age_x'], y = data_corrected[(data_corrected['Cond']=='Foil')& (data_corrected['Acc']=='Incorrect')]['RT'])

In [ ]:
pearsonr(x=data_corrected[(data_corrected['Cond']=='Lure')& (data_corrected['Acc']=='Incorrect')]['age_x'], y = data_corrected[(data_corrected['Cond']=='Lure')& (data_corrected['Acc']=='Incorrect')]['RT'])

In [ ]:
# plt.figure(figsize=(20, 10))
g=sns.lmplot(x='RT', y='dprime_old_lure_x', data=data_corrected[data_corrected['Acc']=='Incorrect'], hue='Cond', height=7 )
plt.xlabel('Median RT', size=16)
plt.ylabel('d\' Old/Sim', size=16)

In [ ]:
# plt.figure(figsize=(20, 10))
g=sns.lmplot(x='RT', y='dprime_bps_x', data=data_corrected[data_corrected['Acc']=='Correct'], hue='Cond', height=7 )
plt.xlabel('Median RT', size=16)
plt.ylabel('d\' BPS', size=16)

In [ ]:
# plt.figure(figsize=(20, 10))
g=sns.lmplot(x='RT', y='dprime_bps_x', data=data_corrected[data_corrected['Acc']=='Incorrect'], hue='Cond', height=7 )
plt.xlabel('Median RT', size=16)
plt.ylabel('d\' BPS', size=16)

In [ ]:
# plt.figure(figsize=(20, 10))
g=sns.lmplot(x='RT', y='dprime_old_new_x', data=data_corrected[data_corrected['Acc']=='Correct'], hue='Cond', height=7 )
plt.xlabel('Median RT', size=16)
plt.ylabel('d\' Old/New', size=16)

In [ ]:
# plt.figure(figsize=(20, 10))
g=sns.lmplot(x='RT', y='dprime_old_new_x', data=data_corrected[data_corrected['Acc']=='Incorrect'], hue='Cond', height=7 )
plt.xlabel('Median RT', size=16)
plt.ylabel('d\' Old/New', size=16)

In [ ]:
g = sns.distplot(data_corr['p_correct'], kde = False)
plt.xlabel('Correct (%)', fontsize=14)
g.set(ylim=[0,45])

In [ ]:
g = sns.distplot(data_corr['p_old_targ'], kde = False)
plt.xlabel('Prop Correct Target Trials', fontsize=14)
plt.ylabel('Number of Participants', fontsize=14)
g.set(ylim=[0,45])

In [ ]:
g = sns.distplot(data_corr['p_sim_lure'], kde = False)
plt.xlabel('Prop Correct Lure Trials', fontsize=14)

g.set(ylim=[0,45])

In [ ]:
g = sns.distplot(data_corr['p_new_foil'], kde = False)
plt.xlabel('Prop Correct Foil Trials', fontsize=14)

g.set(ylim=[0,45])

In [ ]:
g = sns.distplot(data_corr['dprime_bps'], kde = False)
plt.xlabel('d\' BPS', fontsize=14)

In [ ]:
g = sns.distplot(data_corr['dprime_old_new'], kde = False)
plt.xlabel('d\' Old/New', fontsize=14)

In [ ]:
g = sns.distplot(data_corr['dprime_old_lure'], kde = False)
plt.xlabel('d\' Old/Similar', fontsize=14)

In [ ]:
g = sns.distplot(data_corr['bps'], kde = False)
plt.xlabel('BPS Score', fontsize=14)



In [ ]:
g = sns.distplot(data_corr['old_targ_old_foil'], kde = False)
plt.xlabel('Old/New Recognition', fontsize=14)


In [ ]:
g = sns.distplot(data_corr['old_targ_old_lure'], kde = False)
plt.xlabel('Old/Similar Recognition', fontsize=14)


In [ ]:
g=sns.lmplot(x='bps', y='old_targ_old_foil', data=data_corr)
plt.xlabel('BPS Score', fontsize=14)
plt.ylabel('Old/New Recognition', fontsize=14)

In [ ]:
g=sns.lmplot(x='bps', y='old_targ_old_lure', data=data_corr)
plt.xlabel('BPS Score', fontsize=14)
plt.ylabel('Old/Similar Recognition', fontsize=14)


In [ ]:
pearsonr(x=data_corr['bps'], y=data_corr['old_targ_old_lure'])

In [ ]:
g=sns.lmplot(x='old_targ_old_foil', y='old_targ_old_lure', data=data_corr)
plt.xlabel('Old/New Recognition', fontsize=14)
plt.ylabel('Old/Similar Recognition', fontsize=14)


In [ ]:
pearsonr(x=data_corr['old_targ_old_lure'], y=data_corr['old_targ_old_foil'])

In [ ]:
g=sns.lmplot(x='dprime_bps', y='dprime_old_new', data=data_corr)
plt.xlabel('d\' BPS', fontsize=14)
plt.ylabel('d\' Old/New', fontsize=14)

In [ ]:
g=sns.lmplot(x='dprime_bps', y='dprime_old_lure', data=data_corr)
plt.xlabel('d\' BPS', fontsize=14)
plt.ylabel('d\' Old/Similar', fontsize=14)

In [ ]:
g=sns.lmplot(x='dprime_old_new', y='dprime_old_lure', data=data_corr)
plt.xlabel('d\' Old/New', fontsize=14)
plt.ylabel('d\' Old/Similar', fontsize=14)

In [ ]:
pearsonr(x=data_corr['dprime_bps'], y=data_corr['dprime_old_new'])

In [ ]:
pearsonr(x=data_corr['dprime_bps'], y=data_corr['dprime_old_lure'])

In [ ]:
pearsonr(x=data_corr['dprime_old_new'], y=data_corr['dprime_old_lure'])

In [ ]:
g=sns.lmplot(x='age', y='old_targ_old_foil', data=data_corr)
plt.xlabel('Age', fontsize=14)
plt.ylabel('Old/New Recognition', fontsize=14)

In [ ]:
g=sns.lmplot(x='age', y='bps', data=data_corr)
plt.xlabel('Age', fontsize=14)
plt.ylabel('BPS', fontsize=14)

In [ ]:
g=sns.lmplot(x='age', y='old_targ_old_lure', data=data_corr)
plt.xlabel('Age', fontsize=14)
plt.ylabel('Old/Similar Recognition', fontsize=14)

In [ ]:
g=sns.lmplot(x='age', y='dprime_old_new', data=data_corr)
plt.xlabel('Age', fontsize=14)
plt.ylabel('d\' Old/New', fontsize=14)

In [ ]:
g=sns.lmplot(x='age', y='dprime_bps', data=data_corr)
plt.xlabel('Age', fontsize=14)
plt.ylabel('d\' BPS', fontsize=14)

In [ ]:
g=sns.lmplot(x='age', y='dprime_old_lure', data=data_corr)
plt.xlabel('Age', fontsize=14)
plt.ylabel('d\' Old/Similar', fontsize=14)

In [ ]:
g=sns.lmplot(x='education', y='old_targ_old_foil', data=data_corr)
plt.xlabel('Education', fontsize=14)
plt.ylabel('Old/New Recognition', fontsize=14)

In [ ]:
g=sns.lmplot(x='education', y='bps', data=data_corr)
plt.xlabel('Education', fontsize=14)
plt.ylabel('BPS', fontsize=14)

In [ ]:
g=sns.lmplot(x='education', y='old_targ_old_lure', data=data_corr)
plt.xlabel('Education', fontsize=14)
plt.ylabel('Old/Similar Recognition', fontsize=14)

In [ ]:
g=sns.lmplot(x='education', y='dprime_old_new', data=data_corr)
plt.xlabel('Education', fontsize=14)
plt.ylabel('d\' Old/New', fontsize=14)

In [ ]:
g=sns.lmplot(x='education', y='dprime_bps', data=data_corr)
plt.xlabel('Education', fontsize=14)
plt.ylabel('d\' BPS', fontsize=14)

In [ ]:
g=sns.lmplot(x='education', y='dprime_old_lure', data=data_corr)
plt.xlabel('Education', fontsize=14)
plt.ylabel('d\' Old/Lure', fontsize=14)

In [ ]:
g=sns.lmplot(x='dr_comp', y='old_targ_old_foil', data=data_corr)
plt.xlabel('Delayed Recall', fontsize=14)
plt.ylabel('Old/New Recognition', fontsize=14)

In [ ]:
g=sns.lmplot(x='dr_comp', y='bps', data=data_corr)
plt.xlabel('Delayed Recall', fontsize=14)
plt.ylabel('BPS', fontsize=14)

In [ ]:
g=sns.lmplot(x='dr_comp', y='old_targ_old_lure', data=data_corr)
plt.xlabel('Delayed Recall', fontsize=14)
plt.ylabel('Old/Similar Recognition', fontsize=14)

In [ ]:
g=sns.lmplot(x='dr_comp', y='dprime_old_new', data=data_corr)
plt.xlabel('Delayed Recall', fontsize=14)
plt.ylabel('d\' Old/New', fontsize=14)

In [ ]:
g=sns.lmplot(x='dr_comp', y='dprime_bps', data=data_corr)
plt.xlabel('Delayed Recall', fontsize=14)
plt.ylabel('d\' BPS', fontsize=14)

In [ ]:
g=sns.lmplot(x='dr_comp', y='dprime_old_lure', data=data_corr)
plt.xlabel('Delayed Recall', fontsize=14)
plt.ylabel('d\' Old/Lure', fontsize=14)

In [ ]:
g=sns.lmplot(x='hvltr4_norm', y='old_targ_old_foil', data=data_corr)
plt.xlabel('HVLT Delayed', fontsize=14)
plt.ylabel('Old/New Recognition', fontsize=14)

In [ ]:
g=sns.lmplot(x='bvmtr4_norm', y='old_targ_old_foil', data=data_corr)
plt.xlabel('BVMT Delayed', fontsize=14)
plt.ylabel('Old/New Recognition', fontsize=14)

In [ ]:
g=sns.lmplot(x='lm2_norm', y='old_targ_old_foil', data=data_corr)
plt.xlabel('LM Delayed', fontsize=14)
plt.ylabel('Old/New Recognition', fontsize=14)

In [ ]:
g=sns.lmplot(x='lm2_norm', y='dprime_old_new', data=data_corr)
plt.xlabel('LM Delayed', fontsize=14)
plt.ylabel('d\' Old/New', fontsize=14)

In [ ]:
g=sns.lmplot(x='bvmtr4_norm', y='dprime_old_new', data=data_corr)
plt.xlabel('BVMT Delayed', fontsize=14)
plt.ylabel('d\' Old/New', fontsize=14)

In [ ]:
g=sns.lmplot(x='hvltr4_norm', y='dprime_old_new', data=data_corr)
plt.xlabel('HVLT Delayed', fontsize=14)
plt.ylabel('d\' Old/New', fontsize=14)

In [ ]:
g=sns.lmplot(x='hvltr4_norm', y='bps', data=data_corr)
plt.xlabel('HVLT Delayed', fontsize=14)
plt.ylabel('BPS', fontsize=14)

In [ ]:
g=sns.lmplot(x='bvmtr4_norm', y='bps', data=data_corr)
plt.xlabel('BVMT Delayed', fontsize=14)
plt.ylabel('BPS', fontsize=14)

In [ ]:
g=sns.lmplot(x='lm2_norm', y='bps', data=data_corr)
plt.xlabel('LM Delayed', fontsize=14)
plt.ylabel('BPS', fontsize=14)

In [ ]:
g=sns.lmplot(x='lm2_norm', y='dprime_bps', data=data_corr)
plt.xlabel('LM Delayed', fontsize=14)
plt.ylabel('d\' BPS', fontsize=14)

In [ ]:
g=sns.lmplot(x='bvmtr4_norm', y='dprime_bps', data=data_corr)
plt.xlabel('BVMT Delayed', fontsize=14)
plt.ylabel('d\' BPS', fontsize=14)

In [ ]:
g=sns.lmplot(x='hvltr4_norm', y='dprime_bps', data=data_corr)
plt.xlabel('HVLT Delayed', fontsize=14)
plt.ylabel('d\' BPS', fontsize=14)

In [ ]:
g=sns.lmplot(x='hvltr4_norm', y='old_targ_old_lure', data=data_corr)
plt.xlabel('HVLT Delayed', fontsize=14)
plt.ylabel('Old/Similar Recognition', fontsize=14)

In [ ]:
g=sns.lmplot(x='bvmtr4_norm', y='old_targ_old_lure', data=data_corr)
plt.xlabel('BVMT Delayed', fontsize=14)
plt.ylabel('Old/Similar Recognition', fontsize=14)

In [ ]:
g=sns.lmplot(x='lm2_norm', y='old_targ_old_lure', data=data_corr)
plt.xlabel('LM Delayed', fontsize=14)
plt.ylabel('Old/Similar Recognition', fontsize=14)

In [ ]:
g=sns.lmplot(x='bvmtr4_norm', y='dprime_old_lure', data=data_corr)
plt.xlabel('BVMT Delayed', fontsize=14)
plt.ylabel('d\' Old/Similar', fontsize=14)

In [ ]:
g=sns.lmplot(x='hvltr4_norm', y='dprime_old_lure', data=data_corr)
plt.xlabel('HVLT Delayed', fontsize=14)
plt.ylabel('d\' Old/Similar', fontsize=14)

In [ ]:
g=sns.lmplot(x='lm2_norm', y='dprime_old_lure', data=data_corr)
plt.xlabel('LM Delayed', fontsize=14)
plt.ylabel('d\' Old/Similar', fontsize=14)

In [ ]:
g=sns.lmplot(x='ir_comp', y='dprime_old_new', data=data_corr)
plt.xlabel('Immediate Recall', fontsize=14)
plt.ylabel('d\' Old/New', fontsize=14)

In [ ]:
g=sns.lmplot(x='ir_comp', y='dprime_old_lure', data=data_corr)
plt.xlabel('Immediate Recall', fontsize=14)
plt.ylabel('d\' Old/Similar', fontsize=14)

In [ ]:
g=sns.lmplot(x='ir_comp', y='dprime_bps', data=data_corr)
plt.xlabel('Immediate Recall', fontsize=14)
plt.ylabel('d\' BPS', fontsize=14)

In [ ]:
pearsonr(x=data_corr['ir_comp'], y=data_corr['dprime_bps'])

In [ ]:
g=sns.lmplot(x='recog_comp', y='dprime_old_new', data=data_corr)
plt.xlabel('Recognition', fontsize=14)
plt.ylabel('d\' Old/New', fontsize=14)

In [ ]:
g=sns.lmplot(x='recog_comp', y='dprime_old_lure', data=data_corr)
plt.xlabel('Recognition', fontsize=14)
plt.ylabel('d\' Old/Lure', fontsize=14)

In [ ]:
g=sns.lmplot(x='recog_comp', y='dprime_bps', data=data_corr)
plt.xlabel('Recognition', fontsize=14)
plt.ylabel('d\' BPS', fontsize=14)

In [ ]:
row = 0
age_corr_new = data_corr.copy()
for a in age_corr_new['age']:
    if a == 59:
        a = 60
    if a >= 60:
        if a < 65:
            age_corr_new.loc[row, 'age'] = '60-64'
        elif a < 70:
            age_corr_new.loc[row, 'age'] = '65-69'
        elif a < 75:
            age_corr_new.loc[row, 'age'] = '70-74'
        elif a < 80:
            age_corr_new.loc[row, 'age'] = '75-79'
        elif a < 85:
            age_corr_new.loc[row, 'age'] = '80-84'
        else:
            age_corr_new.loc[row, 'age'] = '85+'
    row+=1    
age_corr_new.head()

In [ ]:
g = sns.stripplot(x= age_corr_new['age'], y = age_corr_new['old_targ_old_lure'], jitter = .2)
sns.boxplot(x = age_corr_new['age'], y = age_corr_new['old_targ_old_lure'], palette='pastel')
plt.xlabel('Age Group')
plt.ylabel('Old/Similar Recognition')

In [ ]:
g = sns.stripplot(x= age_corr_new['age'], y = age_corr_new['old_targ_old_foil'], jitter = .2)
sns.boxplot(x = age_corr_new['age'], y = age_corr_new['old_targ_old_foil'], palette='pastel')
plt.xlabel('Age Group')
plt.ylabel('Old/New Recognition')

In [ ]:
g = sns.stripplot(x= age_corr_new['age'], y = age_corr_new['bps'], jitter = .2)
sns.boxplot(x = age_corr_new['age'], y = age_corr_new['bps'], palette='pastel')
plt.xlabel('Age Group')
plt.ylabel('BPS')

In [ ]:
g = sns.stripplot(x= age_corr_new['age'], y = age_corr_new['dprime_bps'], jitter = .2)
sns.boxplot(x = age_corr_new['age'], y = age_corr_new['dprime_bps'], palette='pastel')
plt.xlabel('Age Group')
plt.ylabel('d\' BPS')

In [ ]:
g = sns.stripplot(x= age_corr_new['age'], y = age_corr_new['dprime_old_new'], jitter = .2)
sns.boxplot(x = age_corr_new['age'], y = age_corr_new['dprime_old_new'], palette='pastel')
plt.xlabel('Age Group')
plt.ylabel('d\' Old/New')

In [ ]:
g = sns.stripplot(x= age_corr_new['age'], y = age_corr_new['dprime_old_lure'], jitter = .2)
sns.boxplot(x = age_corr_new['age'], y = age_corr_new['dprime_old_lure'], palette='pastel')
plt.xlabel('Age Group')
plt.ylabel('d\' Old/Similar')

In [ ]:
age_corr_repl = data_corr.copy()
age_corr_repl.loc[data_corr['age'] < 75, 'age'] = '60-74'
age_corr_repl.loc[data_corr['age'] > 74, 'age']= '75-90'
age_corr_repl.head()

In [ ]:
g = sns.stripplot(x= age_corr_repl['age'], y = age_corr_repl['old_targ_old_foil'], jitter = .2)
sns.boxplot(x = age_corr_repl['age'], y = age_corr_repl['old_targ_old_foil'], palette='pastel')
plt.xlabel('Age Group')
plt.ylabel('Old/New Recognition')

In [ ]:
g = sns.stripplot(x= age_corr_repl['age'], y = age_corr_repl['old_targ_old_lure'], jitter = .2)
sns.boxplot(x = age_corr_repl['age'], y = age_corr_repl['old_targ_old_lure'], palette='pastel')
plt.xlabel('Age Group')
plt.ylabel('Old/Similar Recognition')

In [ ]:
g = sns.stripplot(x= age_corr_repl['age'], y = age_corr_repl['bps'], jitter = .2)
sns.boxplot(x = age_corr_repl['age'], y = age_corr_repl['bps'], palette='pastel')
plt.xlabel('Age Group')
plt.ylabel('BPS')

In [ ]:
g = sns.stripplot(x= age_corr_repl['age'], y = age_corr_repl['dprime_old_new'], jitter = .2)
sns.boxplot(x = age_corr_repl['age'], y = age_corr_repl['dprime_old_new'], palette='pastel')
plt.xlabel('Age Group')
plt.ylabel('d\' Old/New')

In [ ]:
g = sns.stripplot(x= age_corr_repl['age'], y = age_corr_repl['dprime_old_lure'], jitter = .2)
sns.boxplot(x = age_corr_repl['age'], y = age_corr_repl['dprime_old_lure'], palette='pastel')
plt.xlabel('Age Group')
plt.ylabel('d\' Old/Similar')

In [ ]:
g = sns.stripplot(x= age_corr_repl['age'], y = age_corr_repl['dprime_bps'], jitter = .2)
sns.boxplot(x = age_corr_repl['age'], y = age_corr_repl['dprime_bps'], palette='pastel')
plt.xlabel('Age Group')
plt.ylabel('d\' BPS')

In [ ]:
educ_corr_data = data_corr.copy()
row = 0
for e in data_corr['education']:
    if e == 12:
        educ_corr_data.loc[row, 'education'] = 'HS/GED'
    elif e == 14 or e == 15:
        educ_corr_data.loc[row, 'education']= 'Associate'
    elif e == 16 or e== 17:
        educ_corr_data.loc[row, 'education']= 'Bachelor\'s'
    elif e== 18 or e==19:
        educ_corr_data.loc[row, 'education']= 'Master\'s'
    else:
        educ_corr_data.loc[row, 'education']='Doctorate'  
    row+=1
    
educ_corr_data.head()

In [ ]:
g = sns.stripplot(x= educ_corr_data['education'], y = educ_corr_data['old_targ_old_foil'], jitter = .2)
sns.boxplot(x = educ_corr_data['education'], y = educ_corr_data['old_targ_old_foil'], palette='pastel')
plt.xlabel('Education')
plt.ylabel('Old/New Recogntion')

In [ ]:
g = sns.stripplot(x= educ_corr_data['education'], y = educ_corr_data['bps'], jitter = .2)
sns.boxplot(x = educ_corr_data['education'], y = educ_corr_data['bps'], palette='pastel')
plt.xlabel('Education')
plt.ylabel('BPS')

In [ ]:
g = sns.stripplot(x= educ_corr_data['education'], y = educ_corr_data['old_targ_old_lure'], jitter = .2)
sns.boxplot(x = educ_corr_data['education'], y = educ_corr_data['old_targ_old_lure'], palette='pastel')
plt.xlabel('Education')
plt.ylabel('Old/Similar Recogntion')

In [ ]:
g = sns.stripplot(x= educ_corr_data['education'], y = educ_corr_data['dprime_old_new'], jitter = .2)
sns.boxplot(x = educ_corr_data['education'], y = educ_corr_data['dprime_old_new'], palette='pastel')
plt.xlabel('Education')
plt.ylabel('d\' Old/New')

In [ ]:
g = sns.stripplot(x= educ_corr_data['education'], y = educ_corr_data['dprime_old_lure'], jitter = .2)
sns.boxplot(x = educ_corr_data['education'], y = educ_corr_data['dprime_old_lure'], palette='pastel')
plt.xlabel('Education')
plt.ylabel('d\' Old/Similar')

In [ ]:
g = sns.stripplot(x= educ_corr_data['education'], y = educ_corr_data['dprime_bps'], jitter = .2)
sns.boxplot(x = educ_corr_data['education'], y = educ_corr_data['dprime_bps'], palette='pastel')
plt.xlabel('Education')
plt.ylabel('d\' BPS')

In [ ]:
g = sns.stripplot(x= educ_corr_data['gender'], y = educ_corr_data['old_targ_old_foil'], jitter = .2)
sns.boxplot(x = educ_corr_data['gender'], y = educ_corr_data['old_targ_old_foil'], palette='pastel')
plt.xlabel('Gender')
plt.ylabel('Old/New Recognition')

In [ ]:
g = sns.stripplot(x= educ_corr_data['gender'], y = educ_corr_data['old_targ_old_lure'], jitter = .2)
sns.boxplot(x = educ_corr_data['gender'], y = educ_corr_data['old_targ_old_lure'], palette='pastel')
plt.xlabel('Gender')
plt.ylabel('Old/Similar Recognition')

In [ ]:
g = sns.stripplot(x= educ_corr_data['gender'], y = educ_corr_data['bps'], jitter = .2)
sns.boxplot(x = educ_corr_data['gender'], y = educ_corr_data['bps'], palette='pastel')
plt.xlabel('Gender')
plt.ylabel('BPS')

In [ ]:
g = sns.stripplot(x= educ_corr_data['gender'], y = educ_corr_data['dprime_bps'], jitter = .2)
sns.boxplot(x = educ_corr_data['gender'], y = educ_corr_data['dprime_bps'], palette='pastel')
plt.xlabel('Gender', fontsize=14)
plt.ylabel('d\' BPS', fontsize=14)

In [ ]:
g = sns.stripplot(x= educ_corr_data['gender'], y = educ_corr_data['dprime_old_new'], jitter = .2)
sns.boxplot(x = educ_corr_data['gender'], y = educ_corr_data['dprime_old_new'], palette='pastel')
plt.xlabel('Gender', fontsize=14)
plt.ylabel('d\' Old/New', fontsize=14)

In [ ]:
g = sns.stripplot(x= educ_corr_data['gender'], y = educ_corr_data['dprime_old_lure'], jitter = .2)
sns.boxplot(x = educ_corr_data['gender'], y = educ_corr_data['dprime_old_lure'], palette='pastel')
plt.xlabel('Gender', fontsize=14)
plt.ylabel('d\' Old/Similar', fontsize=14)

Lure Bin Analysis

In [ ]:
data_long.head()